In [2]:
import os, random, time
import matplotlib.pyplot as plt, matplotlib.ticker as ticker, numpy as np, scipy as sp, scipy.constants as ct, scipy.optimize as sciopt
%matplotlib tk
plt.rcParams.update({'font.size': 20})
from collections import defaultdict
from math import floor, ceil, log, pi, sqrt
from scipy.integrate import solve_ivp
from scipy.interpolate import interp1d, InterpolatedUnivariateSpline
from scipy.signal import savgol_filter, find_peaks, hann, hilbert
from scipy.optimize import minimize
from scipy.misc import derivative
from numpy.fft import rfft, rfftfreq, irfft

import PyLTSpice
from PyLTSpice.LTSpice_RawRead import LTSpiceRawRead
from PyLTSpice.LTSpiceBatch import LTCommander

import VoltageGeneration

Found Numpy. WIll be used for storing data


In [2]:
import matplotlib.animation as animation
from IPython.display import HTML
from IPython.display import set_matplotlib_formats
set_matplotlib_formats('pdf', 'svg')

# Ideal Electrode Voltages

Takes .npy files from backwards ITVG that contains 20 electrode voltages to create a given potential at a given location.

Given number of DAC steps and length of dt, this creates a dictionary of all the ideal v(t) interpolated plots for each electrode (ideal_v). 
Also saves a .npy file with the voltage inputs for each electrode

In [2]:
filepath = 'backwards_ITVG_output/sta_trig/'
exp = 'sta_trig/'
N0 = 10
dt = 1e-6
buffer1 = 0
buffer2 = 0

basepath = '/Users/lukeqi/Desktop/School/MIT/UROP/SuperUROP/IonMotion/'
filenames = sorted(os.listdir(''.join([basepath, filepath])), key = lambda x: float(x.split('_')[-1][:-4]))
filedata = [np.load(''.join([basepath, filepath]) + fname, allow_pickle = True).tolist() for fname in filenames]

Ntot = buffer1+N0+buffer2
T0 = dt*Ntot
_electrode_ordering = [
    "S6", "S7", "S8", "S9", "S10", "S11", "S12", "S13", "S14",
    "S24", "S25",
    "N6", "N7", "N8", "N9", "N10", "N11", "N12", "N13", "N14",
]
    
d = defaultdict(list)
for f in filedata:
    [d[k].append(v) for k, v in f.items()]

%matplotlib tk
ideal_v = {}
ideal_v_deconv = {}
voltages = {}
plt.figure('voltages')
for k, v in d.items():
    vevents = np.concatenate(([v[0]], v, [v[-1]]))
    tevents = np.concatenate(([0], dt*np.linspace(buffer1, N0+buffer1, len(v)), [T0]))
    
    ## TWEAK VOLTAGE SETS
#     vevents = savgol_filter(vevents, 19, 3) ## 19

    ## PLOT VOLTAGE SETS
    mark = '.'
    if k[0] == 'S':
        mark = 'x'
    plt.plot([t*1e6 for t in tevents], vevents, marker=mark, label=k)
    ideal_v[k] = interp1d(np.append(tevents, 1), np.append(vevents, vevents[-1]))
    ideal_v_deconv[k] = interp1d(np.concatenate((tevents, 2*T0-tevents[::-1])), np.concatenate((vevents, vevents[::-1])))
    voltages[k] = ideal_v[k].__call__([(n+0.5)*dt for n in range(Ntot)])
#     voltages[k] = ideal_v_deconv[k].__call__([(n+0.5)*dt for n in range(Ntot*2)])

plt.legend(loc='best', prop={'size': 12})
plt.ylabel('Electrode Voltage (V)')
plt.xlabel('Time (us)')
# np.save(basepath + '/'.join((exp, 'STA_crosstalk_better.npy')), voltages)


Text(0.5, 0, 'Time (us)')

In [ ]:
#### Reformat from dict of lists to lists of dict ####
# basepath = '/Users/lukeqi/Desktop/School/MIT/UROP/SuperUROP/IonMotion/'
# filepath = 'loading/'
# npyfile = 'NominalVolts_20191217_shimtest_1p20_-360um.npy'
# npyfile = 'loading_more_robust.npy'
# load_volt = np.load(''.join((basepath, filepath, npyfile)), allow_pickle=1).tolist()

# volt_array = []
# for i in range(len(list(load_volt.values())[0])):
#     d = {}
#     d.update([(k, v[i]) for k, v in load_volt.items()])
#     volt_array.append(d)
# np.save(''.join((basepath, filepath, npyfile[:-4], '_4Jules.npy')), volt_array)

#### Tanh Interpolation between split and double well ####
filepath = 'backwards_ITVG_output/stringer_full/'
exp = 'stringer'
basepath = '/Users/lukeqi/Desktop/School/MIT/UROP/SuperUROP/IonMotion/'
v1 = np.load(''.join([basepath, filepath]) + 'NominalVolts_500.npy', allow_pickle = True).tolist()
v2 = np.load(''.join([basepath, filepath]) + 'NominalVolts_5000.npy', allow_pickle = True).tolist()
v3 = np.load(''.join([basepath, filepath]) + 'NominalVolts_6199.npy', allow_pickle = True).tolist()
v4 = np.load(''.join([basepath, filepath]) + 'NominalVolts_10000.npy', allow_pickle = True).tolist()

n_transform = 50
v_transform = defaultdict(list)
v_transform2 = defaultdict(list)
for el in v1:
    v_transform[el] = [v1[el] + (v2[el]-v1[el])*qtanhN(i, (0, n_transform), qspan=(0, 1), N=4) for i in range(n_transform)]  
    v_transform2[el] = [v3[el] + (v4[el]-v3[el])*qtanhN(i, (0, n_transform), qspan=(0, 1), N=4) for i in range(n_transform)]  
    
for i in range(n_transform):
    v = [v[i] for v in v_transform.values()]
    v2 = [v[i] for v in v_transform2.values()]
    save_voltage_config(vg, v, ''.join([basepath, filepath, 'NominalVolts_{}.npy'.format(i+1000)]))
    save_voltage_config(vg, v2, ''.join([basepath, filepath, 'NominalVolts_{}.npy'.format(i+7500)]))


# Transport Protocol Methods

In [4]:
def qtanhN(t, tspan, N=5, qspan=(0, 120)):
    ''' t is a specific time
        tspan is a tuple with the start and end time'''
    t0, tf = tspan
    y0, yf = qspan
    T = tf-t0
    if (t-t0) < T:
        return y0 + max((yf-y0)/2*(np.tanh(N*(2*(t-t0)-T)/T)+np.tanh(N))/np.tanh(N), 0)
    else:
        return yf
    
def qsin(t, tspan, qspan=(0, 120)):
    ''' t is a specific time
        tspan is a tuple with the start and end time'''
    t0, tf = tspan
    y0, yf = qspan
    T = tf-t0
    if t < t0:
        return y0
    elif (t-t0) < T:
        return y0 + max((yf-y0)/2*(1-np.cos(pi*(t-t0)/T)), 0)
    else:
        return yf
    
def get_freq_from_alpha(a, m_=40*ct.u):
    ''' make sure alpha is in V/um^2'''
    return np.sign(a)*(2*np.abs(a)*1e12*ct.e/m_)**0.5/(2*pi)

def get_alpha_from_freq(f, m_=40*ct.u):
    return (2*pi*f)**2/(2*ct.e/m_)

def qsta(t, tspan, qspan=(0, 120), f=1):
    ''' t is a specific time
        tspan is a tuple with the start and end time'''
    t0, tf = tspan
    y0, yf = qspan
    T, ti = tf-t0, t-t0 
    s = ti/T
    if s < 0:
        return y0
    elif s <= 1:
        return y0 + (yf-y0)*((1/(2*pi*f)**2)*(60*s - 180*s**2 + 120*s**3) + 10*s**3 - 15*s**4 + 6*s**5)
    else:
        return yf
    
def qsta2(t, tspan, qspan=(0, 120), f=1):
    ''' t is a specific time
        tspan is a tuple with the start and end time'''
    t0, tf = tspan
    y0, yf = qspan
    T, ti = tf-t0, t-t0 
    s = ti/T
    if s < 0:
        return y0
    elif s <= 1:
        return y0 + (yf-y0)*((1/(2*pi*f)**2)*(2520*s**3-12600*s**4+22680*s**5-17640*s**6+5040*s**7)
                             + 126*s**5-420*s**6+540*s**7-315*s**8+70*s**9)
    else:
        return yf
    
def qtrig(t, tspan, qspan=(0, 120), f=1.2e6, mu=1):
    ''' args should be f (axialfreq in Hz), mu (mass ratio)'''
    t0, tf = tspan
    y0, yf = qspan
    T, ti, d = tf-t0, t-t0, yf-y0
    s = ti/T
    Op, On = 2*np.pi*f*(1+1/mu+(1-1/mu+1/mu**2)**0.5)**0.5, 2*np.pi*f*(1+1/mu-(1-1/mu+1/mu**2)**0.5)**0.5
    b3 = -49*((T*Op)**2 - (5*np.pi)**2)*((T*On)**2 - (5*np.pi)**2)/(2048*(T*T*On*Op)**2)
    b4 = 5*((T*Op)**2 - (7*np.pi)**2)*((T*On)**2 - (7*np.pi)**2)/(2048*(T*T*On*Op)**2)
    
    if s < 0:
        return y0
    elif s <= 1:
        return y0 + d*(0.5 + (-9/16+2*b3+5*b4)*np.cos(np.pi*s) + 1/16*(1-48*b3-96*b4)*np.cos(3*np.pi*s) + b3*np.cos(5*np.pi*s) + b4*np.cos(7*np.pi*s))
    else:
        return yf
    
def d_t(t, tspan, qspan=(0, 120)):
    ''' t is a specific time
        tspan is a tuple with the start and end time'''
    t0, tf = tspan
    y0, yf = qspan
    T, ti = tf-t0, t-t0 
    s = ti/T
    if s < 0:
        return y0
    elif s <= 1:
        return y0 + (yf-y0)*(10*s**3 - 15*s**4 + 6*s**5)
    else:
        return yf
    
def gaussian(x, mu, sig):
    return np.exp(-(x-mu)*(x-mu)/(2*sig*sig))

# PyLTSpice Initialization

In [3]:
def set_vinput(volt_array, dt, st, sr):
    ''' writes PWL input to 'input.txt' '''    
    N = len(volt_array)
    varray = [v/16+2.5 for v in volt_array] ## fastino
#     varray = volt_array ## zotino
    
    tevents = [0]
    vevents = [varray[0]]
    for i in range(1,N):
        ti = dt*i
        vi = vevents[-1]
        tevents.append(ti)
        vevents.append(vi)
        
        tdelay = abs(varray[i] - vi)/sr
        if tdelay > dt:
            print('WARNING: SLEW RATE LONGER THAN TIME STEP')
            tevents.append(ti + dt)
            vevents.append(vi + (varray[i] - vi)*dt/tdelay)
        else:
            tevents.append(ti + tdelay)
            vevents.append(varray[i])
    tevents.append(N*dt)
    vevents.append(varray[-1])
    
    with open(circuit_directory_path + 'input.txt', 'w+') as f:
        [f.write('\t'.join((str(t), str(v))) + '\n') for t, v in zip(tevents, vevents)]

    return tevents, vevents

def run_spice_sim(endsim, st, attempts=1):
    instruction = '.TRAN' + ' '.join((str(st), str(endsim)))
    LTC.netlist.insert(-2, instruction)
    LTC.write_netlist()
    rawfile, logfile = LTC.run()
    LTC.reset_netlist()

    if rawfile:
        for i in range(attempts):
            try:
                LTR = LTSpiceRawRead(rawfile)
                break
            except UnicodeDecodeError:
                LTC.netlist.insert(-2, instruction)
                LTC.write_netlist()
                rawfile, logfile = LTC.run()
                LTC.reset_netlist()
                LTR = LTSpiceRawRead(rawfile)
        os.remove(logfile)
    else:
        print('SIMULATION FAILED')
        with open(logfile, 'r') as f:
            [print(line) for line in f.readlines()]
        os.remove(logfile)
        return None
    return LTR

## Intializing SPICE Parameters

In [8]:
circuit_name = 'FastinoAmplifier_V1.0.cir' ## Zotino vs FastinoAmplifier

circuit_directory_path = '/Users/lukeqi/Desktop/School/MIT/UROP/SuperUROP/IonMotion/lib_circuits/'
LTC = LTCommander(circuit_directory_path + circuit_name)
endsim = T0
st = max(dt/20, 100e-9) ## maximum time step of the SPICE transient sim. no lower than 100ns
slew_rate = 20/1E-6 ## slew rate in V/s
gain_err = 0.01 ## 1%
calibration_err = 0.1 ## +- 100mV

print(dt/st, 'steps per dt')

10.0 steps per dt


## Circuitry Testing

In [38]:
#### Compare Fastino step response with measured data ####
with open('fastino_step_chamb_filters_amped.txt', 'r') as f: ## this is Jules's measured data
    data = np.array([l.replace('\n', '').replace(',', '\t').split('\t') for l in f.readlines()][0:])
    t_step = np.array(data[:, 0], dtype='float').reshape(-1)
    v_step = np.array(data[:, 1], dtype='float').reshape(-1)

with open('lib_circuits/fastino_step_chamb_filters_amped_SPICE.txt', 'r') as f:
    data = np.concatenate([l.replace('\n', '').replace(',', '\t').split('\t') for l in f.readlines()][1:])
    data = np.array([d for d in data if d[:4] != 'Step']).reshape(-1, 2)
    t_step_sim = np.array(data[:, 0], dtype='float').reshape(-1)*1e9
    v_step_sim = np.array(data[:, 1], dtype='float').reshape(-1)
    v_step_func = interp1d(t_step_sim, v_step_sim)

plt.figure('time domain')
plt.plot(t_step, v_step, 'k.', label='oscilloscope data')
plt.plot(t_step, v_step_func.__call__(t_step), label='SPICE simulation')
plt.xlabel('time (ns)')
plt.ylabel('fastino response (V)')
plt.title('Fastino Amped Step Response')
plt.legend(shadow=1, loc='best')

In [4]:
#### Compare Fastino step response with measured data ####
with open('../IonMotion_material/Fall20/fastino_amped_step.txt', 'r') as f: ## this is Jules's measured data
    data = np.array([l.replace('\n', '').replace(',', '\t').split('\t') for l in f.readlines()][0:])
    t_step = np.array(data[:, 0], dtype='float').reshape(-1)
    v_step = np.array(data[:, 1], dtype='float').reshape(-1)

with open('../IonMotion_material/Fall20/fastino_amped_SPICE.txt', 'r') as f:
    data = np.array([l.replace('\n', '').split('\t') for l in f.readlines()[1:]]).T
    t_step_sim = np.array(data[0], dtype='float')*1e9
    v_step_sim = np.array(data[1], dtype='float')

plt.figure('time domain')
plt.plot(t_step, v_step, 'k', label='oscilloscope data')
plt.plot(t_step_sim, v_step_sim, label='SPICE simulation')
plt.xlabel('time (ns)')
plt.ylabel('fastino response (V)')
plt.title('Fastino Amped Step Response')
plt.legend(shadow=1, loc='best')

In [17]:
N = len(t_step)*8 # zeropadding = sinc interpolation
w_step = np.fft.rfftfreq(N, (t_step[1]-t_step[0])*1e-9)
v_step_fft = np.fft.rfft(v_step, N)
v_step_fftmag = np.abs(v_step_fft)
v_step_fftphs = np.angle(v_step_fft)

v_stepsim_fft = np.fft.rfft(v_step_func.__call__(t_step), N)
v_stepsim_fftmag = np.abs(v_stepsim_fft)
v_stepsim_fftphs = np.angle(v_stepsim_fft)

plt.figure('frequency domain')
# plt.loglog(w_step[2:], v_step_fftmag[2:], marker='.', label='oscilloscope data')
# plt.loglog(w_step[2:], v_stepsim_fftmag[2:], marker='.', label='SPICE simulation')
ratio = v_step_fftmag[2:]/v_stepsim_fftmag[2:]
plt.semilogx(w_step[2:], 20*np.log(ratio), marker='.', label='oscilloscope data divided by SPICE')
plt.legend(loc='best', shadow=1)
plt.xlabel('Hz')

Text(0.5, 0, 'Hz')

In [3]:
xfer_data = np.loadtxt("filter_board_and_octagon_xfer_combined.txt")
freq_data = xfer_data[:,0] * 1e6
gains_data = np.abs(xfer_data[:,2]/xfer_data[:,1])
plt.semilogx(freq_data, 10*np.log10(gains_data), 'm', label = "Filterbored")


In [4]:
endsim = 120e-6
tevents, vevents = set_vinput(np.linspace(-40, 40, 100), 1e-6, st, slew_rate)
LTR = run_spice_sim(endsim, st, 3)
analysis_output = LTR.get_trace('V(ion)').get_wave(0)
t_spice = LTR.get_trace('time').get_time_axis(0)

plt.plot([t*1E6 for t in tevents], [16*(v-2.5) for v in vevents], label='input') ## fastino
plt.plot(t_spice*1E6, analysis_output, label='output')
plt.legend(loc='best')
plt.show()

NameError: name 'set_vinput' is not defined

## Run SPICE Simulations on non-predistorted voltages

In [ ]:
plotting = True
addnoise = False
endbuffer = 10

# voltages = np.load(basepath + '/'.join((exp, 'STA_crosstalk_better.npy')), allow_pickle = True).tolist()
v_of_t = {}
for k, v in voltages.items():
    endsim = dt*(len(v) + endbuffer)
    tevents, vevents = set_vinput(v, dt, st, slew_rate)
    
    LTR = run_spice_sim(endsim, st, 3)
    
    analysis_output = LTR.get_trace('V(ion)').get_wave(0)
    t_spice = LTR.get_trace('time').get_time_axis(0)
    print('\t'.join((k, str(v), str(t_spice[-1]))), '\n')
    
    if addnoise:
        v_of_t[k] = interp1d(t_spice, np.array(analysis_output)*(1+random.choice([-gain_err, gain_err]))
                         + random.choice([-calibration_err, calibration_err]))
    else:
        v_of_t[k] = interp1d(t_spice, analysis_output)
    
    if plotting:
        plt.figure(k, (19, 9))
        plt.title('Electrode %s' % (k, ))
        plt.xlabel('Time [us]')
        plt.ylabel('Voltage [V]')
        plt.grid()
        
        plt.plot([t*1E6 for t in tevents], [16*(v-2.5) for v in vevents], label='input') ## fastino
#         plt.plot([t*1E6 for t in tevents], vevents, label='input') ## zotino
        plt.plot(t_spice*1E6, analysis_output, label='output')
        t_view = np.linspace(0, T0, 1000)
        plt.plot(t_view*1E6, ideal_v[k].__call__(t_view), 'k--', label='ideal')
        plt.legend(loc='best')
        
        plt.tight_layout()
plt.show()

# Predistortion

The following tries to predistort the DAC input voltages in order to match the electrode output with the desired v(t).

First the impulse response of the circuit is determined. Then the ideal v(t) is deconvoluted with the system's impulse response to determine the ideal input x(t). The x(t) is sampled with the dt and N defined earlier to obtain an initial guess for the ideal DAC input voltages. A local minimizer is then run with the initial guess to obtain a (hopefully) globally optimum input voltage

## Deconvolution

In [ ]:
#### impulse response of circuit  
periods = 1
endbuffer = (2*periods-1)*Ntot
step = True
if step:
    with open(circuit_directory_path + 'input.txt', 'w+') as f:
        [f.write('\t'.join((str(t), str(v))) + '\n') for t, v in zip([0, 1E-12], [2.5, 1/16+2.5])]
else:
    with open(circuit_directory_path + 'input.txt', 'w+') as f:
        [f.write('\t'.join((str(t), str(v))) + '\n') for t, v in zip([0, 1E-12, 100E-9], [2.5, 1/16+2.5, 2.5])]
LTR = run_spice_sim(T0 + endbuffer*dt, st, 3)
analysis_output = LTR.get_trace("V(ion)").get_wave(0)
t_spice = LTR.get_trace('time').get_time_axis(0)
ended = float(t_spice[-1])

#### FFT of impulse response
N = 2**16
M = int(100*N*ended)
t_fft = np.linspace(0, ended, M)
dx = t_fft[1] - t_fft[0]
print(M, N/M, 1/dx)
if step:
    h_step_t = InterpolatedUnivariateSpline(t_spice, analysis_output).__call__(t_fft)
    h_t = np.gradient(h_step_t, dx)*dx
else:
    h_t = InterpolatedUnivariateSpline(t_spice, analysis_output).__call__(t_fft)
h_fft = np.fft.rfft(h_t, N)
h_w = np.fft.rfftfreq(N, dx)
h_mag = np.abs(h_fft)
h_phi = np.unwrap(np.angle(h_fft))

h_phase = -h_phi/h_w
h_group = -np.gradient(h_phi, h_w[1]-h_w[0])

# plt.plot(t_fft, h_t)
plt.figure('impulse mag')
plt.loglog(h_w, h_mag, marker='.', label='with PI-filter')
plt.legend(loc='best', shadow=1)
plt.figure('impulse phase')
plt.plot(h_w, h_phase, ls='--', label='phase delay PI-filter', color='tab:orange')
plt.plot(h_w, h_group, lw=3, label='group delay PI-filter', color='tab:orange')
plt.legend(loc='best', shadow=1)


In [ ]:
#### transfer function of circuit  
periods = 1
endbuffer = (2*periods-1)*Ntot

ended = T0 + endbuffer*dt
N = 2**16
M = int(100*N*ended)
print(M, N/M)
t_fft = np.linspace(0, ended, M)
dx = t_fft[1] - t_fft[0]
h_w = np.fft.rfftfreq(N, dx)

with open(circuit_directory_path + 'FastinoAmplifier_transfer.txt', 'r') as f:
    data = [l.replace('\n', '').replace(',', '\t').split('\t') for l in f.readlines()][1:]
    freq = np.concatenate(([0], np.array([float(d[0]) for d in data])))
    real = np.concatenate(([data[0][1]], np.array([float(d[1]) for d in data])))
    imag = np.concatenate(([data[0][2]], np.array([float(d[2]) for d in data])))
    real_ = interp1d(freq, real).__call__(h_w)
    imag_ = interp1d(freq, imag).__call__(h_w)

h_fft = real_ + 1j*imag_
h_phi = np.unwrap(np.angle(h_fft))
h_phase = -h_phi/h_w
h_group = -np.gradient(h_phi, h_w[1]-h_w[0])

# plt.figure('impulse mag')
# plt.loglog(h_w, np.abs(h_fft), marker='.', label='with PI-filter, SPICE transfer')
plt.figure('impulse phase')
plt.plot(h_w, h_phase, ls='--', label='phase delay PI-filter SPICE', color='tab:green')
plt.plot(h_w, h_group, lw=3, label='group delay PI-filter SPICE', color='tab:green')
plt.legend(loc='best', shadow=1)


In [ ]:
#### frequency domain playground
SNR = np.array([1E6 if w < 120e3 else 500 if w < 360e3 else 10 for w in h_w])

k, v = 'S9', ideal_v_deconv['S9']
#### FFT of ideal v(t)
v_t = v.__call__(t_fft%(2*T0))
vavg = np.average(v_t)
v_fft = np.fft.rfft(v_t - vavg, N)

denom = ((h_fft)*(1+1/(np.abs(h_fft)**2*SNR)))

#### Wiener deconvolution
x_fft = v_fft/denom
x_t = np.fft.irfft(x_fft, N)
x_t = x_t[:M] + vavg

tevents, vevents = set_vinput(x_t, dx, st, slew_rate)
LTR = run_spice_sim(t_fft[-1]+0.1E-6, st, 3)
analysis_output = LTR.get_trace("V(ion)").get_wave(0)
t_spice = LTR.get_trace('time').get_time_axis(0)

x2_t = interp1d(tevents, [16*(v-2.5) for v in vevents], kind='zero').__call__(t_fft)
x2_fft = np.fft.rfft(x2_t, N)
y_t = interp1d(t_spice, analysis_output).__call__(t_fft) ## SPICE output
y_fft = np.fft.rfft(y_t, N)
h_new_fft = y_fft/x2_fft
# plt.plot(t_fft*1E6, x2_t)
# plt.plot(t_fft*1E6, y_t)

y2_fft = h_fft*x2_fft
y2_t = np.fft.irfft(y2_fft, N)[:M] ## convolved output
# plt.plot(y2_t)

# plt.figure('freq domain', (19, 9))
# plt.semilogy(h_w, np.abs(y_fft), marker='.', color='k', label='SPICE output')
# plt.plot(h_w, np.imag(y_fft), marker='.', color='tab:gray', ls='--')
# plt.semilogy(h_w, np.abs(y2_fft), marker='.', color='tab:blue', label='convolved output')
# plt.plot(h_w, np.imag(y2_fft), marker='.', color='tab:cyan', ls='--')
# plt.legend(loc='best', shadow=1)

# plt.figure('impulse mag')
# plt.loglog(h_w, np.abs(h_fft), marker='.')
# plt.loglog(h_w, np.abs(h_new_fft), marker='.')
# plt.figure('impulse phase')
# plt.semilogx(h_w, np.angle(h_fft), marker='.')
# plt.semilogx(h_w, np.angle(h_new_fft), marker='.')

plt.figure(k, (19, 9))
plt.plot(t_fft[:int(len(t_fft)/periods)]*1e6, v_t[:int(len(t_fft)/periods)], 'k--', label='ideal')
plt.plot(t_fft[:int(len(t_fft)/periods)]*1e6, x2_t[:int(len(t_fft)/periods)], 'r-.', label='deconvolved')
plt.plot(t_fft[:int(len(t_fft)/periods)]*1e6, y_t[:int(len(t_fft)/periods)], label='SPICE output')
plt.plot(t_fft[:int(len(t_fft)/periods)]*1e6, y2_t[:int(len(t_fft)/periods)], label='convolved output')
plt.title(k)
plt.tight_layout()
plt.legend(loc='best', shadow=1)
plt.show()


### Run Wiener Deconvolution

In [ ]:
x_n = {}
SNR = np.array([1E6 if w < 120e3 else 500 if w < 360e3 else 10 for w in h_w])
for k, v in ideal_v_deconv.items():
    if k not in x_n.keys():
        #### FFT of ideal v(t)
        v_t = v.__call__(t_fft%(2*T0))
        vavg = np.average(v_t)
        v_t -= vavg
        v_fft = np.fft.rfft(v_t, N)
        v_t += vavg

        #### Wiener deconvolution
        x_fft = v_fft/(h_fft*(1+1/(np.abs(h_fft)**2*SNR)))
        x_t = np.fft.irfft(x_fft, N)
        x_t = x_t[:M] + vavg

        #### SPICE sim of deconvolved input
        tevents, vevents = set_vinput(x_t, dx, st, slew_rate)
        LTR = run_spice_sim(t_fft[-1], st, 3)
        analysis_output = LTR.get_trace("V(ion)").get_wave(0)
        t_spice = LTR.get_trace('time').get_time_axis(0)

        plt.figure(k, (19, 9))
        plt.plot(t_fft[:int(len(t_fft)/periods)]*1e6, x_t[:int(len(t_fft)/periods)], 'r-.')
        plt.plot(t_fft[:int(len(t_fft)/periods)]*1e6, v_t[:int(len(t_fft)/periods)], 'k--')
        plt.plot(t_spice[:int(len(t_spice)/periods)]*1E6, analysis_output[:int(len(t_spice)/periods)])
        plt.title(k)
        plt.tight_layout()

        x_n[k] = interp1d(t_fft, x_t).__call__([(n+0.5)*dt for n in range(Ntot+endbuffer)])

## Local Optimization

In [ ]:
def float_to_16bit(n):
    ''' n is an array of floats
    DAC resolution is 16-bit from -10V to 10V '''
    bit16 = 2**16-1
    if type(n) == 'float':
        return int((n+10)/20*bit16)/bit16*20-10
    return [int((i+10)/20*bit16)/bit16*20-10 for i in n]

def spice_sim_cost(x, *args):
    ''' x is a list of input voltages of length N
        returns cost function between SPICE output and idealized v(t) '''    
    endsim = args[2][-1]+0.5*dt
    if args[3]:
        x = np.concatenate((args[3][0]*[args[1].__call__(0)], x, args[3][1]*[args[1].__call__(1)]))
    set_vinput(x, dt, st, slew_rate)
    
    LTR = run_spice_sim(endsim, st, 25)
    analysis_output = LTR.get_trace('V(ion)').get_wave(0)
    t_spice = LTR.get_trace('time').get_time_axis(0)     
    
    #### cost function ####
    xarray = interp1d(t_spice, analysis_output).__call__(args[2])
    yarray = args[1].__call__(args[2])
    cost = sum((xarray - yarray)**2/args[4]**2)*2
    
    dx = args[2][1]-args[2][0]
    dxdt = dx*np.gradient(xarray, dx)
    dydt = dx*np.gradient(yarray, dx)
    cost += sum((dxdt - dydt)**2/np.std(dydt)**2)*0.5 ## also cost function of the first derivative
    
    print('\t'.join((args[0], str(x), '{:.2f}\t{:.4f}'.format(t_spice[-1]*1E6, cost))), '\n')
    return cost

In [ ]:
ti, tf = (279, 300)
begin, end = (1, 1) ## Ntot = (tf - ti + 1) - (begin + end)
t_cost = np.linspace(0, 25e-6, 1000)

vguess = {}
ideal_v_tweak = {}
t_tweak = dt*np.linspace(ti, tf, 100)
for k, v in ideal_v_deconv.items():
    vguess[k] = np.concatenate((begin*[v.__call__(ti*dt)], x_n[k][ti+begin:tf-end+1], end*[v.__call__(tf*dt)]))
    ideal_v_tweak[k] = interp1d(np.append(t_tweak-ti*dt, 1), np.append(v.__call__(t_tweak%(2*T0)), v.__call__(tf*dt%(2*T0))))

newvoltages = {}
run_minimize = True
for k, v in ideal_v_tweak.items():
    if run_minimize:
        vguessi = vguess[k][begin:-end]
        std = np.std(v.__call__(t_cost))
        print('standard dev in V: ', std)
        if std:
            x_ideal = minimize(spice_sim_cost, vguessi, args=(k, v, t_cost, (begin, end), std), bounds=[(-40,40)]*(len(vguessi)), 
                               method='SLSQP', options={'disp':True, 'ftol':1e-4, 'eps':min(1.0, std/100), 'maxiter':50})
            print(x_ideal)
#             x_ideal2 = minimize(spice_sim_cost, x_ideal.x, args=(k, v, t_cost, (begin, end), std), bounds=[(-40,40)]*(len(vguessi)), 
#                                method='SLSQP', options={'disp':True, 'ftol':1e-6, 'eps':min(0.1, std/10), 'maxiter':100})
#             print(x_ideal2)
            newvoltages[k] = float_to_16bit(np.concatenate((vguess[k][:begin], x_ideal.x, vguess[k][-end:])))
    else:
        tevents, vevents = set_vinput(vguess[k], dt, st, slew_rate)
        LTR = run_spice_sim(t_cost[-1], st, 3)
        analysis_output = LTR.get_trace("V(ion)").get_wave(0)
        t_spice = LTR.get_trace('time').get_time_axis(0)
        
        plt.figure(k, (19, 9))
        plt.title('Electrode %s' % (k, ))
        plt.xlabel('Time [us]')
        plt.ylabel('Voltage [V]')
        plt.grid()
        plt.plot([t*1E6 for t in tevents], [16*(v-2.5) for v in vevents], label='input')
        plt.plot(t_spice*1E6, analysis_output, label='output')
        plt.plot(t_spice*1E6, ideal_v_tweak[k].__call__(t_spice), 'k--', label='ideal')
        plt.legend(loc='best')

        newvoltages[k] = vguess[k]

# np.save(basepath + '/'.join((exp, 'voltages_predist.npy')), newvoltages)

### Fine Adjustments

In [ ]:
#### make finer adjustments to improve optimization
print(newvoltages)
exp
# newvoltages = {'S6': np.array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]), 'S7': [0.5693141069657432, 0.1902800030518037, 8.908979934386206, 16.645914396887157, 15.019607843137255, 9.888303959716183, 3.91256580453193, 1.401541161211565, 0.045014114595254284, 0.11154345006485045, 0.03280689707789719, 0.21408407721065004, 0.21408407721065004], 'S8': [0.04898146028839534, 0.000457770656900891, -1.9023422598611432, -3.7038223849851226, -3.497520408941787, -2.326543068589303, -1.1042954146639197, -0.4258793011367974, -0.18661783779659835, -0.14267185473411104, -0.15060654612039315, -0.1377889677271682, -0.1377889677271682], 'S9': [-0.00015259021896696368, -0.03616388189517039, 1.2230106050202174, 2.28503852903029, 2.1647974364843208, 1.4622720683604165, 0.6395056076905465, 0.2537575341420606, 0.07400625619897738, 0.06576638437476134, 0.056610971236743524, 0.07125963225757204, 0.07125963225757204], 'S10': [0.04898146028839534, 0.000457770656900891, -1.9023422598611432, -3.7038223849851226, -3.4978255893797208, -2.326848249027237, -1.1042954146639197, -0.4258793011367974, -0.18661783779659835, -0.14267185473411104, -0.15060654612039315, -0.1377889677271682, -0.1377889677271682], 'S11': [0.5693141069657432, 0.1881437399862662, 8.906538490882735, 16.64621957732509, 15.015640497444114, 9.889524681467918, 3.9128709849698637, 1.4033722438391685, 0.0474555580987257, 0.12344548714427361, 0.033417257953765045, 0.21408407721065004, 0.21408407721065004], 'S12': np.array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]), 'S13': np.array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]), 'S14': np.array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]), 'S24': np.array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]), 'S25': np.array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])}
# newvoltages['S10'] = newvoltages['S8']
# newvoltages['S11'] = newvoltages['S7']
np.save(basepath + '/'.join((exp, 'voltages_predist_better2.npy')), newvoltages)


## Evaluating Predistorted Voltages

### Run SPICE Simulations on predistorted voltages

In [ ]:
plotting = False
addnoise = False
crosstalk = True
endbuffer = 23

newvoltages = np.load(basepath + '/'.join((exp, 'predistorted_long.npy')), allow_pickle = True).tolist()
v_of_t_predist={}
for k, v in newvoltages.items(): ## loop through each electrode
    if addnoise:
        v = np.array(v)*(1+random.choice([-gain_err, gain_err])) + random.choice([-calibration_err, calibration_err])

    endsim=dt*(len(v) + endbuffer) ## simulate a little longer so the voltages can reach their steady state
    tevents, vevents = set_vinput(v, dt, st, slew_rate)

    LTR = run_spice_sim(endsim, st, 3)
    analysis_output = LTR.get_trace("V(ion)").get_wave(0)
    t_spice = LTR.get_trace('time').get_time_axis(0)
    print('\t'.join((k, str(v), str(t_spice[-1]))), '\n')

    v_of_t_predist[k] = interp1d(t_spice, analysis_output)


    if plotting:
        plt.figure(k, (19, 9))
        plt.title('Electrode %s' % (k, ))
        plt.xlabel('Time [us]')
        plt.ylabel('Voltage [V]')
        plt.grid()

        plt.plot([t*1E6 for t in tevents], [16*(v-2.5) for v in vevents], label='input')
        plt.plot(t_spice*1E6, analysis_output, label='output')
#         plt.plot(t_spice*1E6, ideal_v_tweak[k].__call__(t_spice), 'k--', label='ideal')
        plt.legend(loc='best')

        plt.tight_layout()
plt.show()

v_of_t_crosstalk = {}
if crosstalk:
    t_eval = np.linspace(0, endsim-0.5e-6, 1000)
    varray = np.array([v.__call__(t_eval) for k, v in v_of_t_predist.items()])
    for i, v in enumerate(v_of_t_predist.values()):
        v_of_t_crosstalk[list(v_of_t_predist.keys())[i]] = interp1d(t_eval, varray[i]+0.01*(sum(varray[:i])+sum(varray[i+1:])))
        

In [ ]:
(0.05*(sum(varray[0:3])+sum(varray[4:])))

### Plots v(t) for all electrodes

In [ ]:
t_view=np.linspace(0E-6, 34e-6, 1000)
plt.figure(100)
for k, v in ideal_v.items():
    plt.plot(t_view*1e6, v.__call__(t_view), label=k)
plt.legend(loc='best', prop={'size': 12})
plt.ylabel('voltage (V)')

# plt.figure(101)
# for k, v in v_of_t.items():
#     plt.plot(t_view*1e6, v.__call__(t_view), label=k)
# plt.legend(loc='best', prop={'size': 12})
# plt.ylabel('Electrode Voltage (V)')
# plt.xlabel('Step')    
    
plt.figure(102)
for k, v in v_of_t_predist.items():
    plt.plot(t_view*1e6, v.__call__(t_view))

# ITVG Initialization

The following creates a VoltageGeneration object with gridfiles, ion mass and charge, and electrode ordering.

Takes about 5 minutes to import all the grid files.

In [16]:
reload_vg = False
amu = ct.m_u
e = ct.e
Ca = 40*amu
Sr = 88*amu

_electrode_ordering = [
    "S6", "S7", "S8", "S9", "S10", "S11", "S12", "S13", "S14",
    "S24", "S25",
    "N6", "N7", "N8", "N9", "N10", "N11", "N12", "N13", "N14",
] ## Ordering for the SLT configuration

_electrode_ordering2 = [
    "S1", "S2", "S3", "S4", "S8", "S9", "S10", "S11", "S12",
    "S13", "S14", "S19", "S20", "S21",
    "S22", "S23", "S24", "S25",
    "N1", "N2", "N3", "N4", "N8", "N9", "N10", "N11", "N12",
    "N13", "N14", "N19", "N20", "N21",
] ## Ordering for the maxbeta configuration

def num_to_electrode_name(num):
#     return _electrode_ordering[num-1] ## SLT
    return _electrode_ordering2[maxbeta_electrode_ordering.index(num)] ## MaxBeta

## WHICH ONE SHOULD I USE??
# artiq_electrode_ordering = [12, 13, 14, 15, 16, 17, 18, 19, 20, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]
artiq_electrode_ordering = [i+1 for i in range(20)]

maxbeta_electrode_ordering = [27, 25, 23, 21, 19, 17, 15, 13, 11,
                            9, 7, 5, 3, 1,
                            32, 31, 30, 29,
                            2, 4, 6, 8, 10, 12, 14, 16, 18,
                            20, 22, 24, 26, 28] ## maps electrode name : grid file number

path = '/Users/lukeqi/Desktop/School/MIT/UROP/SuperUROP/GridFiles'
name = 'AngleTrap_Skinny_Potential_reduced'
name2 = 'MaxBeta_Potential'
name3 = 'MaxBeta_Production_Potential'

electrode_grouping = [[x] for x in artiq_electrode_ordering] ## Lincoln Standard
electrode_grouping2 = [[1], [2], [3], [4], [5], [6], [7], [8], [9], [10], [11]] ## Max Beta
electrode_grouping3 = [[x] for x in maxbeta_electrode_ordering] ## Full Max Beta
d_cons = []

start_time = time.time()

if reload_vg:
#     vg = VoltageGeneration.VoltageGeneration(path, name, electrode_grouping, fit_ranges=[5e-6, 10e-6, 5e-6], rf_electrodes=[21], m=Sr, rf_omega=2.*np.pi*49.2e6, order=4, f_cons=d_cons, v_rf=[50])
#     vg2 = VoltageGeneration.VoltageGeneration(path, name2, electrode_grouping2, rf_electrodes=[12], m=Ca, rf_omega=2.*np.pi*43e6, order=4, f_cons=d_cons, v_rf=[70])
    vg3 = VoltageGeneration.VoltageGeneration(path, name3, electrode_grouping=electrode_grouping3, rf_electrodes=[33], m=Ca, rf_omega=2.*np.pi*45.247e6, order=4, f_cons=d_cons, v_rf=[50])

print("Imported grid files in {}".format(time.time() - start_time))

Imported grid files in 9.179115295410156e-05


# Forward ITVG

Uses ITVG in the forward direction to determine the potential from electrode outputs. This is done by calling ITVG to determine potential coefficients at every time given all the electrode voltages.

This is done with a Taylor expansion around a given point, so the resulting coefficients are quite sensitive to the expansion point. Thus multiple runs need to be done in order to find an accurate converging result. 

[the length units are in um, the time units are in sec]

## Run Forward ITVG

### Moving Harmonic Oscillator

In [ ]:
#### Moving Harmonic Oscillator
num_guess = 1
endITVG = 10e-6
volt_gen = vg3
N_final = int(endITVG*1e6*5)+1
t_anal = np.linspace(0, (endITVG), N_final)
print(N_final, 'calls per iteration')

## DON'T FORGET TO CHANGE THE FIRST GUESS
# q_ideal = np.array([qsta2(t, (0e-6, 50e-6), qspan=(0, 120), f=50e-6*1.1e6) for t in np.linspace(0, 50, 100)*1e-6])
# q1 = [120+q_ideal, 240+q_ideal, [360]*len(q_ideal)*4, 360-q_ideal, 240-q_ideal, 120-q_ideal, q_ideal, [120]*len(q_ideal)*2]
# q1 = interp1d(np.append(np.linspace(0, 600, 1200)*1e-6, 1), np.append(np.concatenate(q1), q1[-1][-1]))
# q2 = [[-120]*len(q_ideal)*2, -120+q_ideal, -q_ideal, -120-q_ideal, -240-q_ideal, [-360]*len(q_ideal)*4, -360+q_ideal, -240+q_ideal]
# q2 = interp1d(np.append(np.linspace(0, 600, 1200)*1e-6, 1), np.append(np.concatenate(q2), q2[-1][-1]))
# q_firstguess = q2.__call__(t_anal)
q_firstguess = np.array([qtrig(t, (1e-6, 21e-6), qspan=(120, 240), f=1.2e6) for t in t_anal])
q_firstguess = np.array([186.5 for t in t_anal])

## DON'T FORGET TO CHOOSE PROPER VOLTAGES
bigarray = np.array([v.__call__(t_anal) for k, v in ideal_v.items()]).T

el_config = []
for row in bigarray:
    el_config.append([[[i+1], v] for i, v in enumerate(row)])

q = []
coeff = []
for n in range(num_guess):
    print('\t'.join(('iter', 't [us]', 'R [um]', 'f [Hz]', 'shift [um]')))
    if not n:
        qg = q_firstguess
    else:
        qg = q_guess
        
    expansion_point = [[[0., i, 50.]] for i in qg]
        
    for i in range(len(t_anal)):
        coeffs = volt_gen.compute_total_potential_axes(expansion_point[i], el_config[i], printing=False)
        coeff.append(coeffs)
        shift = -coeffs[1]/(2*coeffs[6])
        q.append(qg[i] + shift)
        print('\t'.join((str(i), '{:.2f}\t{:.2f}\t{:.0f}\t{:.3f}'.format(t_anal[i]*1E6, qg[i], get_freq_from_alpha(coeffs[6], Sr), shift))))
    q_guess = q[n*N_final:(n+1)*N_final]

coeff_final = np.array(coeff[N_final*(num_guess-1):N_final*num_guess])
q_final = np.array(q[N_final*(num_guess-1):N_final*num_guess])
a_final = coeff_final.T[6]

In [ ]:
# print(vg.function_map)
# print(np.array(coeff)[:, 34:35])
# print(a_final)

with open(basepath + '/'.join((exp, '-60um-60um_v.txt')), 'w+') as f:
    f.writelines(['\t'.join((str(t), str(q))) + '\n' for t, q in zip(t_anal, q_final)])


### Splitting Potential

In [ ]:
#### Splitting Potential
num_guess = 1
endITVG = 300e-6
volt_gen = vg
N_final = int(endITVG*1e6*3)+1
t_anal = np.linspace(0, (endITVG), N_final)
print(N_final, 'calls per iteration')

## DON'T FORGET TO CHOOSE PROPER VOLTAGES
bigarray = np.array([v.__call__(t_anal) for k, v in ideal_v.items()]).T

el_config = [] ## compute_total_potential_axes() requires a special formatting for voltages
for row in bigarray:
    el_config.append([[[i+1], v] for i, v in enumerate(row)])

# y_span_um = 420
# n_y_points = 840
# y_points = np.linspace(-y_span_um, y_span_um, n_y_points)
# y_test_points = np.array([np.zeros(n_y_points), y_points, np.zeros(n_y_points)]).T + [[0.0, 0, 50.0]]
# y_pot = np.array([volt_gen.compute_potential(y_test_points, [[el[0][0], el[1]] for el in e]) for e in el_config[::1]])
# q1 = [y_points[find_peaks(-yi)[0][-1]] for yi in y_pot]

coeff = []
for n in range(num_guess):
    print('\t'.join(('iter', 't [us]', 'a [V/um^2]', 'b [V/um^4]')))        
    for i in range(len(t_anal)):
        coeffs = volt_gen.compute_total_potential_axes([[0.0, 0, 50.0]], el_config[i], printing=False)
        coeff.append(coeffs)
        print('\t'.join((str(i), '{:.2f}\t{:.2E}\t{:.2E}'.format(t_anal[i]*1E6, coeffs[6], coeffs[29]))))

coeff_final = np.array(coeff[N_final*(num_guess-1):N_final*num_guess])
q_final = np.full(len(t_anal), 0)
a_final = coeff_final.T[6]
b_final = coeff_final.T[29]
g_final = coeff_final.T[1]
c_final = coeff_final.T[15]

# plt.plot(q1)


### Potential Animation

In [ ]:
def total_potential_cut(vg, voltages, trap_center, tilt_angle_deg, x_bounds, z_bounds):
    el_config = vg.ungroup_configuration(voltages)
    for el in vg.rf_electrodes:
        el_config.append([el, 1])
    
    tilt_angle_rad = tilt_angle_deg/180.0*np.pi
    
    max_r = max(
        np.abs(x_bounds[0] - trap_center[0]),
        np.abs(x_bounds[1] - trap_center[0]),
        np.abs(z_bounds[0] - trap_center[2]),
        np.abs(z_bounds[1] - trap_center[2])
    )
    
    r_points = np.linspace(-max_r, max_r, 201)
    
    test_coords = []
    r_points_clipped = []
    
    for i, r_value in enumerate(r_points):
        test_coord = [
            trap_center[0] + r_value*np.sin(tilt_angle_rad),
            trap_center[1],
            trap_center[2] + r_value*np.cos(tilt_angle_rad),
        ]
    
        if test_coord[0] >= x_bounds[0] and test_coord[0] <= x_bounds[1] and test_coord[2] >= z_bounds[0] and test_coord[2] <= z_bounds[1]:
            test_coords.append(test_coord)
            r_points_clipped.append(r_value)
    
    r_points_clipped = np.array(r_points_clipped)
    pot_values = np.zeros(len(test_coords))
    
    for i, test_coord in enumerate(test_coords):
        pot_values[i] = vg.compute_potential([test_coord], el_config)
    
    return r_points_clipped, pot_values

In [ ]:
#### Splitting potential shape
down = 1
R = [[0.0, 0, 50.0]]
volt_gen = vg3

## 

# x_bounds = [-20.0, 20.0]
# z_bounds = [40.0, 80.0]
# pot_values = np.array([total_potential_cut(volt_gen, b, R[0], 0.0, x_bounds, z_bounds) for b in bigarray[::down]])
# r_values = np.transpose(pot_values, (1, 0, 2))[0][0]
# pot_values = np.transpose(pot_values, (1, 0, 2))[1]

##

y_span_um = 780
n_y_points = 780
y_points = np.linspace(-y_span_um, y_span_um, n_y_points)
y_test_points = np.array([np.zeros(n_y_points), y_points, np.zeros(n_y_points)]).T + R
y_pot = np.array([volt_gen.compute_potential(y_test_points, [[el[0][0], el[1]] for el in e]) for e in el_config[::down]])

## change what you're plotting
xaxis = y_points
yaxis = y_pot
adaptive_yaxis = False

%matplotlib inline
fig, ax = plt.subplots()
# ax.set_ylim(min(yaxis.flatten()), max(yaxis.flatten()))
ax.set_ylim(-3, 3)
line, = ax.plot(xaxis, yaxis[0])

def animate(i):
    if adaptive_yaxis:
#         ax.set_ylim(min(yaxis[i]), max(yaxis[i]))
        ax.set_ylim(yaxis[i][find_peaks(-yaxis[i])[0][0]], max(yaxis[i]))
    line.set_ydata(yaxis[i])  # update the data
    return line,

#Init only required for blitting to give a clean slate.
def init():
    line.set_ydata(np.ma.array(xaxis, mask=True))
    return line,

ani = animation.FuncAnimation(fig, animate, np.arange(1, int(len(t_anal)/down)-1), init_func=init,
    interval=25, blit=True)
HTML(ani.to_jshtml())

y_points *= 1e-6

In [ ]:
HTML(ani.to_jshtml())

In [ ]:
el_config = [[[[27], 5.0], [[25], 5.0], [[23], 5.0], [[21], 5.0], [[19], 5.0], [[17], 0.201225475752411], [[15], 1.0014838159514627], [[13], 3.3225474499293868], [[11], 2.128938105718141], [[9], 2.270692808882413], [[7], 0.6374714275053842], [[5], 0.1466649766144879], [[3], 5.0], [[1], 5.0], [[32], 7.391058969545981], [[31], 2.3287339860098673], [[30], 2.6046570657691115], [[29], 7.110452367105831], [[2], 5.0], [[4], 5.0], [[6], 0.21145891139496492], [[8], 1.6029101362525104], [[10], 3.65055730601964], [[12], 2.2783729685730587], [[14], 2.7977210059094646], [[16], 0.30471347506795593], [[18], 5.0], [[20], 5.0], [[22], 5.0], [[24], 5.0], [[26], 5.0], [[28], 5.0]]]
el_config[0].append([[vg3.rf_electrodes[0]], 1]) ## adds rf electrode
y_pot = np.array([vg3.compute_potential(y_test_points, [[el[0][0], el[1]] for el in e]) for e in el_config[::down]])
plt.plot([y*1e6 for y in y_points], y_pot[0])

peaks = find_peaks(-y_pot[0])[0]
plt.plot([y_points[p]*1e6 for p in peaks], [y_pot[0][p] for p in peaks], 'rs')

ion_min = y_points[peaks[-1]]*1e6
print(get_freq_from_alpha(np.gradient(np.gradient(y_pot[0]))[peaks[-1]], m_=Ca)/2)


In [ ]:
ypot = y_pot[25]
peaks = find_peaks(-ypot)[0]
plt.plot(y_points, ypot)
plt.plot(y_points[peaks], ypot[peaks], marker='x', ls='')
print(y_points[peaks])
print(coeffs[34])

### Local Coefficient Determination

In [ ]:
#### Splitting Potential
endITVG = 3029.5e-6
volt_gen = vg
N_final = int(endITVG*1e6*0.1)+1
t_anal = np.linspace(0, (endITVG), N_final)
print(N_final, 'calls per iteration')

## DON'T FORGET TO CHOOSE PROPER VOLTAGES
bigarray = np.array([v.__call__(t_anal) for k, v in ideal_v.items()]).T

el_config = [] ## compute_total_potential_axes() requires a special formatting for voltages
for row in bigarray:
    el_config.append([[[i+1], v] for i, v in enumerate(row)])
    
R = [[0.0, 0, 50.0]]
y_span_um = 420
n_y_points = 5000
y_points = np.linspace(-y_span_um, y_span_um, n_y_points)
y_test_points = np.array([np.zeros(n_y_points), y_points, np.zeros(n_y_points)]).T + R
y_pot = np.array([volt_gen.compute_potential(y_test_points, [[el[0][0], el[1]] for el in e]) for e in el_config])
q_array = [y_points[find_peaks(-ypoti)[0]] for ypoti in y_pot]

coeff = []
print('\t'.join(('iter', 't [us]', 'a [V/um^2]', 'b [V/um^4]')))        
for i in range(len(t_anal)):
    coeffs = volt_gen.compute_total_potential_axes([[0.0, q_array[i][0], 50.0]], el_config[i], printing=False)
    coeff.append(coeffs)
    print('\t'.join((str(i), '{:.2f}\t{:.2E}\t{:.2E}'.format(t_anal[i]*1E6, coeffs[6], coeffs[29]))))

coeff_final = np.array(coeff)
q_final = np.array([q[0] for q in q_array])
a_final = coeff_final.T[6]
b_final = coeff_final.T[29]
g_final = coeff_final.T[1]
c_final = coeff_final.T[15]

d_array = -g_final/(2*a_final)
# print(d_array)
axial_array = get_freq_from_alpha(a_final - 3*c_final*d_array + 6*b_final*d_array**2, m_=40*amu)
print(axial_array)
plt.plot(axial_array)
# print([c[[34, 1, 6, 15, 29]] for c in coeff_final])

## Evaluate Forward ITVG Output

### Transport Output

In [ ]:
fig, ax = plt.subplots()
ax.plot(t_anal*1e6, q_firstguess, 'k', linestyle='--', marker='o')
ax.plot(t_anal*1e6, q_final, linestyle='--', marker='o')

t_view = np.linspace(0, t_anal[-1], int(1e7*t_anal[-1]))
t_view = t_anal
ax2 = ax.twinx()
dx=t_view[1]-t_view[0]

q_points = InterpolatedUnivariateSpline(t_anal, q_final, k=1).__call__(t_view)
q_ideal = InterpolatedUnivariateSpline(t_anal, q_firstguess, k=1).__call__(t_view)
dqdt=np.gradient(q_points, dx)
d2qdt2=np.gradient(dqdt, dx)
dqdt_g=np.gradient(q_firstguess, dx)
d2qdt2_g=np.gradient(dqdt_g, dx)
ax2.plot(t_view*1e6, d2qdt2_g, 'k')
ax2.plot(t_view*1e6, d2qdt2)


In [ ]:
plt.plot(t_anal*1e6, get_freq_from_alpha(a_final, m_=Sr))

In [ ]:
#### Plots error ratio
dq = q_final - q_firstguess
dddq = np.gradient(np.gradient(dq, dx), dx)
plt.figure('error')
plt.plot(t_anal*1e6, dq)
# plt.plot(t_anal*1e6, dddq)

#### Error in frequency domain
N = 2**16 ## whether or not you zero pad makes little difference
t_fft, dx = np.linspace(0, 100e-6, 4096, retstep=1)
q_w = np.fft.rfftfreq(N, dx)

dddq_t = interp1d(np.concatenate([t_view, [1]]), np.concatenate([dddq, [dddq[-1]]])).__call__(t_fft)
dddq_fft = np.fft.rfft(dddq_t, N)
dddq_fftmag = np.abs(dddq_fft)
dddq_fftphs = np.angle(dddq_fft)
plt.figure('error frequency')
plt.semilogy(q_w, dddq_fftmag, marker='.')

In [ ]:
qfiles = ['-60um-60um.txt', '0um-120um.txt', '120um-240um.txt', '-60um-60um_v.txt']
t_arr = []
q_arr = []
for q in qfiles:
    with open(basepath + exp + q, 'r') as f:
        data = np.array([l.replace('\n', '').replace(',', '\t').split('\t') for l in f.readlines()])
        t_arr.append(np.array(data[:, 0:1], dtype=float).T.reshape(-1))
        q_arr.append(np.array(data[:, 1:2], dtype=float).T.reshape(-1))

qguess = [np.array([qtrig(t, (0e-6, 20e-6), qspan=qspani, f=1.2e6) for t in ti]) for ti, qspani in zip(t_arr, [(-60, 60), (0, 120), (120, 240), (-60, 60)])]
N = 2**16 ## whether or not you zero pad makes little difference
t_fft, dx = np.linspace(0, 200e-6, 1000, retstep=1)
q_w = np.fft.rfftfreq(N, dx)

for ti, qi, qgi, qf in zip(t_arr, q_arr, qguess, qfiles):
    dq = qi- qgi
    dddq = np.gradient(np.gradient(dq, dx), dx)
    dddq_t = interp1d(np.concatenate([t_view, [1]]), np.concatenate([dddq, [dddq[-1]]])).__call__(t_fft)
    dddq_fft = np.fft.rfft(dddq_t, N)
    dddq_fftmag = np.abs(dddq_fft)
    dddq_fftphs = np.angle(dddq_fft)
    plt.semilogy(q_w, dddq_fftmag, marker='.', label=qf)
#     plt.plot(q_w, dddq_fft, marker='.', label=qf)

plt.legend(loc='best', shadow=1)
    
    

### Splitting Output

In [ ]:
maxb = np.max(b_final)
plt.figure('ITVG output')
plt.plot(t_anal*1e6, a_final/a_final[0], label='α(t)/α(0)')
plt.plot(t_anal*1e6, b_final/maxb, label='β(t)/βmax')
# plt.plot(t_anal*1e6, g_final/g_final[0])
# plt.plot(t_anal*1e6, c_final/c_final[0])
plt.legend(loc='best', shadow=1)
plt.title('ITVG output')
plt.xlabel('time (us)')
print('max beta: ', maxb)


In [ ]:
#### Splitting Animation ####
down = 2
y_eval = np.linspace(-360, 360, 1000)
phi_t = np.array([c[34] + c[1]*y_eval + c[6]*y_eval**2 + c[15]*y_eval**3 + c[29]*y_eval**4 for c in coeff])
anharm_t = np.array([100*(c[1]*y_eval + c[15]*y_eval**3)/p for c, p in zip(coeff, phi_t)]) ## percent error

phi_t = phi_t[::down]
anharm_t = anharm_t[::down]

## change what you're plotting
xaxis = y_eval
yaxis = phi_t
adaptive_yaxis = True

%matplotlib inline
fig, ax = plt.subplots()
ax.set_ylim(min(yaxis.flatten()), max(yaxis.flatten()))
line, = ax.plot(xaxis, yaxis[0])

def animate(i):
    if adaptive_yaxis:
        ax.set_ylim(min(yaxis[i]), max(yaxis[i]))
    line.set_ydata(yaxis[i])  # update the data
    return line,

#Init only required for blitting to give a clean slate.
def init():
    line.set_ydata(np.ma.array(yaxis, mask=True))
    return line,

ani = animation.FuncAnimation(fig, animate, np.arange(1, int(len(t_anal)/down)-1), init_func=init,
    interval=25, blit=True)
HTML(ani.to_jshtml())

### Save Output

In [ ]:
q_file = 'q(t).txt'
coeff_file = 'Φ(t).txt'

with open(basepath + '/'.join((exp, q_file)), 'w+') as f:
    f.writelines([str(qi) + ' ' for qi in q_final])
    
with open(basepath + '/'.join((exp, coeff_file)), 'w+') as f:
    [f.writelines([str(c) + ' ' for c in coeff] + ['\n']) for coeff in coeff_final]

## Load protocol from text file

In [ ]:
experiment = 'experiment13/'
os.chdir(basepath + experiment)

with open('q(t)2.txt', 'r') as f:
    q_final = np.array([float(x) for x in f.readlines()[0].strip().split(' ')])
    
with open('Φ(t)2.txt', 'r') as f:
    coeff_final = np.array([[float(i) for i in x.split()] for x in f.readlines()])

t_anal = np.linspace(0, 15E-6, len(q_final))
print(len(q_final), len(coeff_final))

#  Ion Simulation

This starts the simulation of ion trajectory given potential shape versus time.

The first cell initializes constants and the ions: the number, charge, and mass of each ion.

The second cell defines helper methods for the simulation. `phi(t, x)` is the electric potential function $\Phi \left( x, t \right)$. `p_derivative(func, ms, var=1, point=[])` implements are partial derivative at a given point to determine the electrostatic force on the ions. `get_dist_matrix(x)` determines the distance between each ion given their positions `x`; used to determine the Coulomb interaction. `heat_sim(tff, qi, t_eval, ms, showplot)` runs an ODE solver given the system ODE and potential $\Phi$ to determine each ion's trajectory. `get_heat(yvalues, omega)` determines the ion's heat gained in quanta using the method described by Hucul.

In [19]:
def heat_sim(tff, q_protocol, t_eval, ms=1E-8, showplot=False, a=None, b=None, g=None, c=None, deg=2):
    ''' simulates ion motion and returns heat gained
        parameters: final integration time, motion protocol, time window for heat calculation 
        all length units must be in um'''
    if a is not None:
        alpha = InterpolatedUnivariateSpline(np.concatenate((t_anal,[tff*1.1])), np.append(a*1e12, a[-1]*1e12), k=deg)
    else:
        alpha = InterpolatedUnivariateSpline(np.concatenate((t_anal,[tff*1.1])), [get_alpha_from_freq(omega_0/(2*pi))]*(len(t_anal)+1), k=deg)
    if b is not None:
        beta = InterpolatedUnivariateSpline(np.concatenate((t_anal,[tff*1.1])), np.append(b*1e24, b[-1]*1e24), k=deg)
    else:
        beta = InterpolatedUnivariateSpline(np.concatenate((t_anal,[tff*1.1])), [0]*(len(t_anal)+1), k=deg)
    if g is not None:
        gamma = InterpolatedUnivariateSpline(np.concatenate((t_anal,[tff*1.1])), np.append(g*1e6, g[-1]*1e6), k=deg)
    else:
        gamma = InterpolatedUnivariateSpline(np.concatenate((t_anal,[tff*1.1])), [0]*(len(t_anal)+1), k=deg)
    if c is not None:
        cubic = InterpolatedUnivariateSpline(np.concatenate((t_anal,[tff*1.1])), np.append(c*1e18, c[-1]*1e18), k=deg)
    else:
        cubic = InterpolatedUnivariateSpline(np.concatenate((t_anal,[tff*1.1])), [0]*(len(t_anal)+1), k=deg)
    qi = InterpolatedUnivariateSpline(np.concatenate((t_anal,[tff*1.1])), np.concatenate((q_protocol/1e6, [q_protocol[-1]/1e6])), k=deg)
    
    def fn(t, v):
        x = v[:Nion]
        dxdt = v[Nion:]
        dist = get_dist_matrix(x)
        
        eom = [] ## equation of motion
        for i in range(Nion): ## iterate over all the ions
            eom.append(-q_ions[i]/m[i]*p_derivative(phi, point=[t, x[i]-qi.__call__(t), alpha.__call__(t), beta.__call__(t), m[i], gamma.__call__(t), cubic.__call__(t)]) +
                       k0*q_ions[i]/m[i]*sum([q_ions[j]*abs(dist[i][j])/(dist[i][j]**3) for j in range(Nion) if j!=i]))
        
        return np.concatenate([dxdt, eom])
        
    outsol = solve_ivp(fn, (0, tff), np.concatenate([x0, x0dot]), method='RK45', dense_output=True, t_eval=t_eval, max_step=ms)    
    h = get_heat(outsol.y[0]-qi.__call__(t_eval), 2*pi*get_freq_from_alpha(alpha.__call__(tff)/1e12, m[0]), m[0])
    
    if showplot:
        t_view = np.linspace(0, tff, int(1E9*tff))
        q = qi.__call__(t_view)
        x = outsol.sol(t_view)[0]
    #     dx=t_view[1]-t_view[0]
    #     dqdt=np.gradient(q,dx)
    #     d2qdt2=np.gradient(dqdt, dx)
    #     d3qdt3=np.gradient(d2qdt2, dx)

        fig, ax = plt.subplots()
        plt1, = ax.plot(t_view*1E6, q*1E6, 'k--')
        ax.set_ylabel('potential minimum position (um)', color='k')
        ax2=ax.twinx()
        plt2, = ax2.plot(t_view*1E6, (x-q)*1E9)
        ax2.set_ylabel('ion oscillation (nm)')
        plt3 = ax2.axvline(t_eval[0]*1E6, color='r', linestyle='--')
        ax2.axvline(t_eval[-1]*1E6, color='r', linestyle='--')
    #     ax.plot(t_view*1E6, dqdt)
    #     ax.plot(t_view*1E6, d2qdt2)
    #     ax.plot(t_view*1E6, d3qdt3, 'g')
        ax.set_xlabel('time (us)')
        plt.legend((plt1, plt2, plt3), ('potential minimum', 'ion motion', 'heat calculation window'), loc='best')
        
        for n in range(1, Nion):
            x = outsol.sol(t_view)[n]
            ax2.plot(t_view*1E6, (x-q)*1E9)
        
    return outsol, h, outsol.y, qi.__call__(t_eval)

def heat_sim2(tff, t_eval, ypot_final, ms=1E-8, showplot=False, deg=2):
    ''' simulates ion motion and returns heat gained
        parameters: final integration time, motion protocol, time window for heat calculation 
        all length units must be in um'''
    ypot = [InterpolatedUnivariateSpline(np.concatenate((t_anal,[tff*1.1])), np.append(yi, yi[-1]), k=deg) for yi in ypot_final.T]
     
    def fn(t, v):
        x = v[:Nion]
        dxdt = v[Nion:]
        dist = get_dist_matrix(x)
        
        eom = [] ## equation of motion
        yvals = [float(y.__call__(t)) for y in ypot]
        for i in range(Nion): ## iterate over all the ions
            eom.append(-q_ions[i]/m[i]*p_derivative(phi2, point=[t, x[i], yvals]) +
                       k0*q_ions[i]/m[i]*sum([q_ions[j]*abs(dist[i][j])/(dist[i][j]**3) for j in range(Nion) if j!=i]))
        
        return np.concatenate([dxdt, eom])
        
    outsol = solve_ivp(fn, (0, tff), np.concatenate([x0, x0dot]), method='RK45', dense_output=True, t_eval=t_eval, max_step=ms)    
#     h = get_heat(outsol.y[0]-qi.__call__(t_eval), 2*pi*get_freq_from_alpha(alpha.__call__(tff)/1e12, m[0]), m[0])
    h = 0
    
    if showplot:
        t_view = np.linspace(0, tff, int(1E9*tff))
        x = outsol.sol(t_view)[0]
        plt.plot(t_view*1e6, x, 'k--')
        plt.ylabel('ion position (um)', color='k')
        
    return outsol, h, outsol.y

def phi(t, y, a, b, m, g=0, c=0):
    ''' electric potential function [SI Units]
    y is the difference between ion position and potential minimum 
    '''
    return a*y**2 + b*y**4 + g*y + c*y**3

def phi2(t, y, ypot):
    ''' electric potential function [SI Units]
    y is the difference global position 
    '''
    return float(interp1d(y_points, ypot).__call__(y))

def p_derivative(func, ms=1e-11, var=1, point=[], order=1):
    ''' implements a partial derivative '''
    args = point[:]
    def wraps(x):
        args[var] = x
        return func(*args)
    return derivative(wraps, point[var], dx = ms, n=order)
    
def get_dist_matrix(y):
    ''' gets distance matrix given positions of N ions 
        x = Nion x 1 vector '''
    D=np.zeros((Nion, Nion))
    for i in range(Nion):
        for j in range(i+1, Nion):
            D[i][j]=y[i]-y[j]
            D[j][i]=-D[i][j]
    return D

def get_heat(yvalues, w, m_=40*ct.m_u): ## calculates <n0> (average motional state) Given by Hercul 3.2 eq 97
    maxy = np.max(yvalues)
    miny = np.min(yvalues)
    A = (maxy-miny)/2
    return A**2*m_*np.abs(w)/(2*hbar)

In [20]:
def find_ic(q0, a0, N, xguess, b0=0, disp=False):
    ''' given q, phi coefficients, and num ion, minimize the total energy of the system
    both q and coeff have length scale in um'''
    def total_potential_energy(x, *args):
        q0, a0, b0 = args
        dist = get_dist_matrix(x)
        U = sum([a0*q_ions[i]*(x[i]-q0)**2 + b0*q_ions[i]*(x[i]-q0)**4 + 0.5*k0*q_ions[i]*sum([q_ions[j]/abs(dist[i][j])
                                                                   for j in range(N) if j!=i]) for i in range(N)])
        return U/1.6E-19
    idealx = minimize(total_potential_energy, xguess, args=(q0, a0, b0), method='SLSQP',
                     options={'ftol': 1e-12, 'eps': 1E-12, 'maxiter':1000})
    if disp:
        print(idealx)
    return idealx.x

In [21]:
hbar = ct.hbar
eps0 = ct.epsilon_0
k0 = 1/(4*pi*eps0)
amu = ct.m_u
elem_charge = ct.e
um = 1e-6

Ti = 2e-6
t_anal = np.linspace(0, Ti, 1000)
a_final = np.full(len(t_anal), get_alpha_from_freq(1.2e6, m_=88*amu)/1e12)
q_final = np.array([qsta2(t, (0e-6, Ti), qspan=(-60, 60), f=Ti*1.06e6) for t in t_anal])

Nion = 2
q_ions = elem_charge*   np.full(Nion, 1)
m = amu*                np.array([88, 88])
x0dot = um*             np.full(Nion, 0)
x0guess = um*           np.array([q_final[0]+1, q_final[0]-1])

#### Transport
# x0 = find_ic(q_final[0]/1e6, a_final[0]*1e12, Nion, x0guess)
# print(x0*1e6)

#### Splitting
x0 = find_ic(q_final[0]/1e6, a_final[0]*1e12, Nion, x0guess)
print(x0*1e6)
# plt.plot(y_points, y_pot[0])

[-58.09228148 -61.90772051]


In [ ]:
heat_start = 330e-6
heat_end = 360e-6

t_eval = np.linspace(heat_start, heat_end, 4096)
outsol, n, y_eval, q_eval = heat_sim(t_eval[-1], q_final, t_eval, showplot=True, a=a_final, b=b_final, g=None, c=None, deg=2)
print('heat gained (quanta): ', n)

In [ ]:
heat_start = 203e-6
heat_end = 220e-6

t_eval = np.linspace(heat_start, heat_end, 4096)
outsol, n, y_eval = heat_sim2(t_eval[-1], t_eval, y_pot, showplot=True, deg=1)
print('heat gained (quanta): ', n)

In [ ]:
freq_modes = [1.2e6, 2.07e6, 0.55e6, 1e6]

N = 2**16
dx = t_eval[1]-t_eval[0]
ion_w = [np.fft.rfftfreq(N, dx)]*Nion
ion_fft = [np.fft.rfft((y_eval[i]-q_eval) - np.average((y_eval[i]-q_eval)), N) for i in range(Nion)]
windows = [gaussian(ion_w[0], freq_modes[i], 0.1E6) for i in range(Nion)]

ion_heat = []
for ion in range(Nion):
    ion_heat.append([get_heat(np.fft.irfft(ion_fft[ion]*windows[j], N)[:len(t_eval)], 2*pi*freq_modes[j], m_=m[ion]) for j in range(Nion)])
print(*ion_heat, sep='\n')    

plt.figure('freq domain')
plt.plot(ion_w[0], np.abs(ion_fft[0]))
plt.plot(ion_w[1], np.abs(ion_fft[1]))
print(find_peaks(np.abs(ion_fft[0]), 4e-4))
print(ion_w[0][316])


In [ ]:
#### FFT of the ion's trajectory during the entire time! ####
N = 2**16
t_fft, dx = np.linspace(0, t_eval[-1], int(N/2), retstep=1)
ion_w = [np.fft.rfftfreq(N, dx)]*Nion
ion_fft = [np.fft.rfft((outsol.sol(t_fft)[i]*np.hamming(len(t_fft))) - np.average((y_eval[i]-q_eval)), N) for i in range(Nion)]

plt.figure('freq domain')
plt.plot(ion_w[0], np.abs(ion_fft[0]))
plt.plot(ion_w[1], np.abs(ion_fft[1]))

In [ ]:
period_comparisons = [10E-6, 11E-6, 12E-6, 13E-6, 14E-6]
heat_STA2 = [3.4869745839921995e-12, 2.5265727597403887e-12, 4.119145301383639e-12, 4.369938540285962e-12, 4.469491484810881e-12]
heat_tanh = [0.9219575399291101, 0.08265132688097625, 0.0443544084065545, 0.004709410003624353, 0.007124299400353403]    

T_vec = np.linspace(1E-6, 15E-6, 113)
h_tanh = [21860379.204771508, 19527259.171407495, 17259149.16513066, 15104369.004127078, 13097740.753574891, 11261482.741053505, 9606741.316782104, 8135659.108718322, 6843513.144189095, 5720688.904020252, 4754366.1293558795, 3930004.3748323065, 3232355.3978540185, 2646293.677808416, 2157296.0853030034, 1751826.1800804606, 1417535.7614588093, 1143342.1752640265, 919477.3547679177, 737437.0990977209, 589933.3043170145, 470788.03826123715, 374814.1178856898, 297712.2719032651, 235926.47012907872, 186548.7452868783, 147193.5351413773, 115916.48709309232, 91132.48315124643, 71545.94927593949, 56107.340530300484, 43960.3306878075, 34417.81010225547, 26926.526186399187, 21046.525367393064, 16431.7599175036, 12808.729627613946, 9966.629343204646, 7738.838536683437, 5996.808511374097, 4638.719358479636, 3583.6517778733837, 2767.633738236115, 2138.08207269243, 1653.8879933963167, 1281.2537689740852, 994.0597962304524, 771.8681212955348, 598.9078726198134, 463.8152009225268, 357.71385970098856, 274.55771005817394, 209.54198806294042, 159.15432243305938, 120.5740285611604, 91.34646607834281, 69.535113405967, 53.322296693594446, 41.32415865754145, 32.34524494076195, 25.49363925854077, 20.133268539743085, 15.79421613953866, 12.233910831232825, 9.280950132692114, 6.880032132621585, 4.98537763992257, 3.5532237160085356, 2.5296564052328283, 1.830028859663429, 1.3791933109767271, 1.0965042918219388, 0.9219577269569045, 0.8061359394656257, 0.7108493145163918, 0.6135544525125097, 0.5008612869876629, 0.37801214438994285, 0.2573731093245095, 0.15541858961316757, 0.08265133807524432, 0.03927602600536682, 0.018534535571079376, 0.011012885068283563, 0.010717628891101295, 0.015589291256232674, 0.02491698593478409, 0.03623879032927943, 0.044354412905367395, 0.04440909666392209, 0.035279317689990014, 0.020771115888475158, 0.007633938797562335, 0.0007611208475235557, 0.0005725668316463508, 0.003329414824010064, 0.004709410150844816, 0.003063653153869151, 0.0005335381267536169, 0.00041805926516588483, 0.004150819031296672, 0.00968258238259697, 0.013255835335809143, 0.012037189448228538, 0.00712429999767196, 0.0018963136015318204, 1.1513064927621181e-05, 0.0021640874934902707, 0.005987833185735553, 0.007839930460671178, 0.006344592244677347, 0.0026626424042116782, 0.00014800498218162236]
h_sta2 = [0.0027457574329533027, 0.000622244713810922, 0.0007692290472509321, 8.555656806107376e-05, 2.5460732411022256e-05, 5.4963303211495336e-05, 8.866392765126439e-06, 3.5907162214263003e-06, 4.822243816901508e-06, 9.940967503224284e-07, 2.3670810362648974e-07, 8.846907546852013e-07, 4.0752140880724983e-07, 7.529221181354637e-08, 2.440920323018283e-07, 9.786007494516467e-08, 9.196726820579428e-09, 8.504924920070799e-08, 4.234162129821739e-08, 6.3874246765282796e-09, 3.7992581970298426e-08, 1.0132389599907777e-08, 3.1230146769288125e-09, 1.482711737797462e-08, 3.490294040236548e-09, 2.3577947179366326e-09, 7.078805709535544e-09, 9.037306397125792e-10, 1.7775853559486115e-09, 3.228397883198339e-09, 2.1694297294826736e-10, 1.3062290966028884e-09, 1.3340406844119502e-09, 1.1774693702483632e-10, 7.233852369493664e-10, 5.785010205332819e-10, 1.0101561777524935e-10, 4.1691232183489324e-10, 2.5036270573786693e-10, 1.0037926555593123e-10, 1.9221533969745635e-10, 1.5915026311859344e-10, 5.402821779380754e-11, 1.2003984170498364e-10, 7.412158130772998e-11, 5.1058129459067205e-11, 4.2957179912034234e-11, 7.01519064806352e-11, 2.076971766974016e-11, 3.03325547670737e-11, 4.812421728310299e-11, 1.082232936371081e-11, 2.04659862030932e-11, 4.271449450518831e-11, 1.922676366099081e-12, 2.2554223869692792e-11, 2.6688182371712017e-11, 2.381593505624296e-13, 1.9364369527091794e-11, 1.9774364556768655e-11, 9.480713158380508e-13, 1.8586232038443306e-11, 1.1410028389835638e-11, 1.97322985477496e-12, 1.4217024838670001e-11, 6.8076667469960265e-12, 2.808948036033941e-12, 1.0629489320415597e-11, 4.297910753312524e-12, 3.1679917108700623e-12, 8.231065005345076e-12, 3.3115270178914938e-12, 3.4869671849121796e-12, 6.295601460829703e-12, 2.658816699214573e-12, 3.325173171541176e-12, 5.062946965816891e-12, 2.6088188596377337e-12, 3.2665673278570265e-12, 4.135120524188767e-12, 2.5265507159352904e-12, 3.1481775292883998e-12, 3.993716837949619e-12, 3.0352849837973514e-12, 3.4557209094964513e-12, 3.945625709137979e-12, 3.222624421902003e-12, 3.4875702356798747e-12, 4.119151332789206e-12, 3.72913789044929e-12, 3.8725171705322555e-12, 4.447194263402881e-12, 3.986855119672547e-12, 4.0101996431647285e-12, 4.724797148291008e-12, 4.3096593757329285e-12, 4.370619158868956e-12, 5.052512810294186e-12, 4.381692514054926e-12, 4.462008475537296e-12, 5.123363551118903e-12, 4.4201746365215726e-12, 4.7209264807891545e-12, 5.2967259628455235e-12, 4.469433544868851e-12, 4.918950311175886e-12, 5.303699894462947e-12, 4.473240229049259e-12, 5.0916686354469566e-12, 5.230296162887539e-12, 4.4468669941297285e-12, 5.16457521441343e-12, 5.074742607144874e-12]
plt.semilogy(T_vec*1E6, h_tanh, marker='x')
plt.semilogy(T_vec*1E6, h_sta2, marker='.')

# Backwards ITVG

Methods to solve for voltages given location of potential minimum.

Make sure that the grid files are loaded at the top of this notebook

In [ ]:
def rotate_coeffs(theta, xx, xz, zz):
    """
    Find the coefficients $C'_{xx}, C'_{xz}, C'_{zz}$ of the potential
    after a rotation in the zx-plane by `theta` given the unprimed values,
    as described above.
    
    `theta` specified in radians.
    """
    sin = np.sin
    cos = np.cos
    
    xxnew = xx*cos(theta)**2 + zz*sin(theta)**2 + 0.5*xz*sin(2*theta)
    xznew = xz*cos(2*theta)  + (zz-xx)*sin(2*theta)
    zznew = zz*cos(theta)**2 + xx*sin(theta)**2 - 0.5*xz*sin(2*theta)
    
    return [xxnew, xznew, zznew]

def dcTiltAngle(vg, R, theta_targ, yy, rfEls, delta=3, debug=False):
    # expansion coefficients of the pseudopotential near the trapping point `R`
    # THIS FUNCTION ASSUMES ONE RF ELECTRODE!
    rf_coeffs = vg.printCoefficients(R, rfEls, ('zz', 'xx', 'xz', 'yy', 'xy', 'yz'), printing=False)[0]
    [rfzz, rfxx, rfxz, rfyy, rfxy, rfyz] = rf_coeffs[:,0]
    
    def tiltError(theta_tilt, debug=False):
        """
        Error in the tilt angle of the full potential, as a function of the dc tilt.
        """
        
        zz = delta*yy
        xx = -(1+delta)*yy # delta ensures Laplace (div phi = 0) is satisfied, but adjusts the principal axes of the xz ellipse
        xz = 0
        
        # rotate!
        [xxnew, xznew, zznew] = rotate_coeffs(theta_tilt, xx, xz, zz)

        if debug:
            print (np.degrees(theta_tilt))
            print (np.degrees(np.arctan((xznew+rfxz)/((rfzz+zznew) - (rfxx+xxnew))))/2)
        
        return (
            np.tan(2*theta_targ) - (xznew+rfxz)/((rfzz+zznew) - (rfxx+xxnew))
        )**2
    
    if debug:
        pass
#         print (np.degrees(theta_targ))

#         fig, ax = plt.subplots()
#         angle_space = np.linspace(0, np.pi/2, 91)
#         errors = [tiltError(theta) for theta in angle_space]
#         ax.plot(np.degrees(angle_space), errors)
#         ax.set_ylim(0,1)
    
    return sciopt.minimize_scalar(tiltError).x

def tiltWithPseudo(vg, R, theta_targ, yy, rfEls, delta=3, debug=False):
    th = dcTiltAngle(vg, R, theta_targ, yy, rfEls, delta=delta, debug=debug)
    
    if debug:
        print ("Tilting dc to {:.2f} deg".format(np.degrees(th)))
    
    zz = delta*yy
    xx = -(1+delta)*yy
    xz = 0
    return rotate_coeffs(th, xx, xz, zz)

def generate_voltages_marlo_style(vg, trap_center, sec_freq, tilt_angle):
    """
    Given specified trap parameters, produce grouped voltage set
    that gives the potential, and produce three voltage sets for
    electric field trimming (for micromotion compensation).
    
    Parameters
    ----------
    trap_center : array_like
        Length-3 array specifying desired trap center
    sec_freq : number
        Axial frequency, in Hz
    tilt_angle : number
        Principal axis tilt in the radial plane, in radians
    
    Returns
    -------
    nom_voltages : list<number>
        Grouped voltage configuration for specified trap parameters,
    shift_voltages : list<list<number>>
        Three grouped voltage configurations giving x,y,z 100V/m field shifts.
    """
    omega = 2 * np.pi * sec_freq
    convexity = 1 if sec_freq > 0 else -1  # allow anti-confining potentials
    
    R = [trap_center]
    
    yycons = 0.5 * vg.m * convexity * omega**2 * (1e-6)**2 / e
    [xxt, xzt, zzt] =  tiltWithPseudo(vg, R, tilt_angle, yycons, vg.rf_electrodes, debug=False)
    cons = [('xx', xxt), ('yy', yycons), ('zz', zzt), ('xz', xzt), ('yz', 0), ('xy', 0)]

    t = time.time()
    nom_voltages = vg.findControlVoltages(R, cons=[cons], tol=1e-12)
    
    shift_voltages = []
    for ind, direction in enumerate(['x', 'y', 'z']):
        # Add a small field in the particular direction by adding a linear constraint
        R = [trap_center]
        if direction == 'y':
            newcons = cons + [(direction, -1e-4)] #Sign convention is done to be consistent with a positive displacement
        else: # Try fitting with a larger shift and dividing down
            newcons = cons + [(direction, -1e-3)] #Sign convention is done to be consistent with a positive displacement
        
        # get the voltages needed for a shift by subtracting the nominal voltages
        shift_voltage_set = vg.findControlVoltages(R, cons=[newcons], tol=1e-6) - nom_voltages
        if not direction == 'y':
            shift_voltage_set /= 10.0
        shift_voltages.append(shift_voltage_set)

    return (nom_voltages, shift_voltages)

def save_voltage_config(vg, v_config, save_file_path):
    """
    Save a (grouped) voltage configuration to a specified file path.
    
    Parameters
    ----------
    v_config : array_like
        Voltage configuration to save, specifying voltage on each electrode _group_
        Length equal to number of electrode groups
    save_file_path : string
        Full path specification of the file to save to.
    """
    voltages_dict = {num_to_electrode_name(num): v for [num, v] in vg.ungroup_configuration(v_config)}
    np.save(save_file_path, voltages_dict)
    
def generate_voltages_and_marlo_save(vg, trap_center, sec_freq, tilt_angle_deg, path, filename_spec):
    """
    Generate and save voltage files (nominal, x/y/z-shift, and linear shim) given
    specified trap parameters. Files are saved to a specified path, with "Avon"
    naming convention:
    
        NominalVolts_<filename_spec>.npy
        XDeltaVolts_<filename_spec>.npy
        YDeltaVolts_<filename_spec>.npy
        ZDeltaVolts_<filename_spec>.npy
        Shims_<filename_spec>.npy
    
    Parameters
    ----------
    trap_center : length-3 list or array
        Location of the desired trap center.
    sec_freq : number
        Desired secular frequency, in Hz.
    tilt_angle_deg : number
        Principal axis tilt in the radial plane, in degrees.
    path : string
        Directory in which to save voltage files.
    filename_spec : string
        Files are named <Nominal/XShift/etc.>_<filename_spec>.npy
    """
    assert type(tilt_angle_deg) is int
    assert -90 <= tilt_angle_deg <= 90
    
    tilt_angle = np.radians(tilt_angle_deg)
    (nom_v, shift_vs) = generate_voltages_marlo_style(vg, trap_center, sec_freq, tilt_angle)
    
    save_voltage_config(vg, nom_v, os.path.join(path, "NominalVolts_{}.npy".format(filename_spec)))
    
def generate_wells(vg, R, f, shims=None, norm=None, fixed_v=[], tilt=None, bounds=(-10.0, 10.0), tol=1e-11, dv=0):
    ''' R, w, and shims are lists of trap centers, axial frequencies (Hz), and shim values (tuple)
    Returns a single voltage set'''
    volt_gen = vg
#     cons = list(map(lambda a: [('const', 0), ('xy', 0), ('yz', 0), ('yy', 0.5*vg.m*(2*pi*a)**2*1e-12/ct.e)], f))
    cons = list(map(lambda a: [('xy', 0), ('yz', 0), ('yy', 0.5*vg.m*(2*pi*a)**2*1e-12/ct.e)], f))

    if shims:
        shim_cons = list(map(lambda b: [('x', b[0]), ('y', b[1]), ('z', b[2])], shims))
    else:
        shim_cons = [[('x', 0), ('y', 0), ('z', 0)] for i in range(len(R))]
        
    if tilt != None:
        tilt_cons = []
        for i in range(len(R)):
            tilt_cons.append(tiltWithPseudo(volt_gen, R[i], tilt, 0.5*vg.m*(2*pi*f[i])**2*1e-12/ct.e, volt_gen.rf_electrodes, debug=False))
        tilt_cons = [[('xx', xxi), ('xz', xzi), ('zz', zzi)] for xxi, xzi, zzi in tilt_cons]
        cons = [i+j for i, j in zip(cons, tilt_cons)]
        
    cons = [i+j for i, j in zip(cons, shim_cons)]
#     print(cons) ## displays target coefficient constraints
    
    nom_v = volt_gen.findControlVoltages(R, cons=cons, tol=tol, bnds=bounds, normvec=norm, fixed_voltages=fixed_v, epss=1e-9, independent=False, dv_weight=dv)
    return nom_v

## Transport

In [ ]:
t_transport = np.linspace(0, 20, 100)*1e-6
q_ideal = np.array([qtrig(t, (0e-6, 20e-6), qspan=(120, 240), f=1.2e6, mu=1) for t in t_transport])
a_ideal = np.array([get_alpha_from_freq(1.2e6, m_=88*amu)*1e-12 for t in t_transport])

#### Call Backwards ITVG ####
R = [[0, 0.0, 50.0]]
tilt_angle = np.radians(15) ## convert from deg to rad
volt_gen = vg
fix_volt = []
bounds = (-40.0, 40.0)
a = a_ideal[-1]
norm = np.full((9, 1), 1)
# norm[1] = 1e2
# norm[6] = 1e2
volt_gen.set_electrode_grouping(electrode_grouping) ## sets previous voltages to None
for q, i in zip(q_ideal, 1+np.arange(len(t_transport))):
    trap_center = [0, q, 50.0]
    R = [trap_center]
    xx, xz, zz = tiltWithPseudo(volt_gen, R, tilt_angle, a, volt_gen.rf_electrodes, debug=False)
    print(i, 'solving at (q, a):', q, a)
    cons = [('x', 0), ('y', 0), ('z', 0), ('xx', xx), ('xz', xz), ('zz', zz), ('yy', a), ('yz', 0), ('xy', 0)]
    nom_v = volt_gen.findControlVoltages(R, cons=[cons], tol=1e-11, fixed_voltages=fix_volt, bnds=bounds, normvec=norm, epss=1e-7, independent=False)
    save_voltage_config(volt_gen, nom_v, os.path.join('/Users/lukeqi/Desktop/School/MIT/UROP/SuperUROP/IonMotion/backwards_ITVG_output/sta_trig', "NominalVolts_{}.npy".format(i)))

## Old Backwards ITVG Call
# for i, y_pos in enumerate(ypoints, 1):
#     print('Solving at position:', y_pos)
#     generate_voltages_and_marlo_save(vg,
#         [0, y_pos, 50], # Trap position
#         1200*1e3,       # Trap frequency
#         15,             # Tilt, in degrees
#         '/Users/ijulieq/Desktop/School/MIT/UROP/SuperUROP/IonMotion/backwards_ITVG_output',
#         filename_spec='{}'.format(i),
#     )

## Splitting

### STA Splitting

In [ ]:
eps0 = ct.epsilon_0
K = ct.e/(4*pi*eps0)
Ca = 40*ct.m_u
Sr = 88*ct.m_u
Ba = 133*ct.m_u
mass = Ca

_a10, _a11 = -780, 127
a0 = get_alpha_from_freq(600e3, m_=mass)
l0 = 2*a0/mass*np.array([1, 3]) ## (-, +)
lf = 2*a0/mass*np.array([1, 1.000016])
gn, gp = (l0/lf)**0.25
rho_p_t = lambda s, a10, a11: 1-(126*(1-gp) + a10 + 5*a11)*s**5 + (420*(1-gp) + 5*a10 + 24*a11)*s**6 - (540*(1-gp) + 10*a10 + 45*a11)*s**7 + (315*(1-gp) + 10*a10 + 40*a11)*s**8 - (70*(1-gp) + 5*a10 + 15*a11)*s**9 + a10*s**10 + a11*s**11

t_transport, dx = np.linspace(0, 50, 100, retstep=True) ## us
s_view = t_transport/t_transport[-1]
rho_p = rho_p_t(s_view, _a10, _a11)
d2rho_pdt2 = np.array([p_derivative(rho_p_t, ms=1e-5, var=0, point=[s, _a10, _a11], order=2) for s in s_view])
lp_t = l0[1]*rho_p**(-4) - d2rho_pdt2/rho_p
ln_t = np.full(len(s_view), l0[0])
d_ideal = (4*K/(mass*(lp_t-ln_t)))**(1/3) ## SI units [m]
a_ideal = (mass/8*(3*lp_t - 5*ln_t)) ## SI units [V/m^2]
b_ideal = (2*K*(d_ideal)**(-5) - 2*a_ideal*(d_ideal)**(-2)) ## SI units [V/m^4]

a_ideal *= 1e-12
b_ideal *= 1e-24
b_ideal *= 2e-10/np.max(b_ideal)

# plt.plot(t_transport, a_ideal/a_ideal[0])
# plt.plot(t_transport, b_ideal/np.max(b_ideal))
# plt.plot(t_transport, np.abs(a_ideal)/(625*(b_ideal+1e-15)))
# plt.plot(t_transport, np.abs(a_ideal)/(625*1e5*(b_ideal+1e-15)))


#### Call Backwards ITVG ####
volt_gen = vg
trap_center = [0, 0.0, 50.0]
R = [trap_center]
fix_volt = [(18, 0.0), (19, 0.0)]
bounds = (-10.0, 10.0)
tilt_angle_deg = 0 #in degrees
tilt_angle = np.radians(tilt_angle_deg)
for a, b, i in zip(a_ideal, b_ideal, 1+np.arange(len(t_transport))):
    print(i, 'solving at (a, b):', a, b)
    cons = [('yy', a), ('yz', 0), ('xy', 0), ('yyyy', b)]
    nom_v = volt_gen.findControlVoltages(R, cons=[cons], tol=1e-12, fixed_voltages=fix_volt, bnds=bounds, norm=1e6)
    save_voltage_config(volt_gen, nom_v, os.path.join('/Users/lukeqi/Desktop/School/MIT/UROP/SuperUROP/IonMotion/backwards_ITVG_output/splitting_STA_LL', "NominalVolts_{}.npy".format(i)))
    

### Kaufmann Splitting

In [ ]:
bmax = 2e-10
eps0 = ct.epsilon_0
K = ct.e/(4*pi*eps0)
Ca = 40*ct.m_u
Sr = 88*ct.m_u
Ba = 133*ct.m_u
d_CP = 0.5*(2*K/(bmax*1e24))**0.2
mass = Ca

t_transport, dx = np.linspace(0, 50, 100, retstep=True) ## us
d_ideal = np.array([d_t(t, (t_transport[0], t_transport[-1]), qspan=(2.1542814, 80)) for t in t_transport]) ## um (half the separation)
t_CP = interp1d(d_ideal, t_transport).__call__(d_CP*1e6)
print('d_CP: ', d_CP*1e6, 't_CP: ', t_CP)

b_ideal = np.array([bmax*qtanhN(t, (0, t_CP), qspan=(0, 1),N=2) if t<t_CP else bmax*(1-0*qtanhN(t-t_CP, (t_CP*0.5, t_CP*1.5), qspan=(0, 1))) for t in t_transport]) ## V/um^4
a_ideal = 1e-12*np.array([K*d**(-3) - 0.5*b*d**2 for b, d in zip(b_ideal*1e24, 2*d_ideal*1e-6)]) ## V/um^2

# plt.plot(t_transport, a_ideal/a_ideal[0])
# plt.plot(t_transport, b_ideal/np.max(b_ideal))
# plt.plot(t_transport, np.abs(a_ideal)/(625*5e2*(b_ideal+1e-15)))
# plt.ylim(0, 1)

#### Call Backwards ITVG ####
volt_gen = vg
trap_center = [0, 0.0, 50.0]
R = [trap_center]
fix_volt = []
bounds = (-10.0, 10.0)
for a, b, i in zip(a_ideal, b_ideal, 1+np.arange(len(t_transport))):
    print(i, 'solving at (a, b):', a, b)
    cons = [('yy', a), ('yz', 0), ('xy', 0), ('yyy', 0), ('yyyy', b)]
    nom_v = volt_gen.findControlVoltages(R, cons=[cons], tol=1e-12, fixed_voltages=fix_volt, bnds=bounds, norm=1e6) ## 5e2, 1e3
    save_voltage_config(volt_gen, nom_v, os.path.join('/Users/lukeqi/Desktop/School/MIT/UROP/SuperUROP/IonMotion/backwards_ITVG_output/LL_loading_quartic', "NominalVolts_{}.npy".format(i)))
    

## NDSP Testing

In [ ]:
norm_vec = np.vstack(np.full(18, 1)) ## number of constraints
norm_vec[6] = 1e1 ## pick out the ones you want to modify
norm_vec[15] = 1e1

z_null = 50
for D in [0, 10, 20, 30, 40, 50]:
    positions = [[0, -120, z_null], [0, 120+D, z_null]]
    frequencies = [0.8e6, 0.8e6]
    shims = [(1e-4, 2e-4, 3e-4), (3e-4, 2e-4, 1e-4)]
    fixed_volts = [(0, 3), (8, 3), (11, 3), (19, 3)]
#     fixed_volts = []
    volt_set = generate_wells(vg, positions[:], frequencies[:], fixed_v=fixed_volts, tilt=np.radians(15), norm=norm_vec, bounds=(-40, 40))
    el_config = vg.ungroup_configuration(volt_set)
    for el in vg.rf_electrodes:
        el_config.append([el, 1])

    plt.figure('potential')
    ymin, ymax = -420, 420
    yrange = ymax-ymin
    yeval = np.linspace(ymin, ymax, yrange*2+1)
    ypoints = np.array([np.zeros(yrange*2+1), yeval, np.zeros(yrange*2+1)]).T + [[0.0, 0, z_null]]
    phi = vg.compute_potential(ypoints, el_config)
    plt.plot(yeval, phi)
    print('final voltages: ', volt_set, '\n')

    ## supposed minimum
    min1 = np.where(yeval==positions[0][1])[0][0]
    min2 = np.where(yeval==positions[1][1])[0][0]
    ddphi = np.gradient(np.gradient(phi))
    print('achieved yy: ', ddphi[min1], ddphi[min2])
    
    ## actual minimum
    minima = find_peaks(-phi)
    print(minima)
    
    plt.plot([yeval[min1], yeval[min2]], [phi[min1], phi[min2]], marker='x', ls='', color='k')
    plt.plot([yeval[m] for m in minima[0]], [phi[m] for m in minima[0]], marker='s', ls='', color='r')



## Stringer Sequence

### Split

In [ ]:
bmax = 4e-10
eps0 = ct.epsilon_0
K = ct.e/(4*pi*eps0)
Sr = 88*ct.m_u
d_CP = 0.5*(2*K/(bmax*1e24))**0.2
mass = Sr

volt_gen = vg
trap_center = [0, 0.0, 50.0]
R = [trap_center]
fixed_v = []
bounds = (-10.0, 10.0)

#### Split to 120um ####
volt_gen.set_electrode_grouping(electrode_grouping) ## sets previous voltages to None
t_transport, dx = np.linspace(0, 250, 500, retstep=True) ## [us]
# d0 = (2*K*ct.e/mass/(2*np.pi*1.2e6)**2)**(1/3)
d_ideal = np.array([d_t(t, (t_transport[0], t_transport[-1]), qspan=(2, 110)) for t in t_transport]) ## [um] (half the separation)
t_CP = interp1d(d_ideal, t_transport).__call__(d_CP*1e6)
b_ideal = np.array([bmax*qtanhN(t, (0, 0.6*t_CP), qspan=(0, 1), N=3) if t<t_CP else bmax*(1-0*qtanhN(t-t_CP, (t_CP*0.5, t_CP*1.5), qspan=(0, 1), N=2)) for t in t_transport]) ## [V/um^4]
a_ideal = 1e-12*np.array([K*d**(-3) - 0.5*b*d**2 for b, d in zip(b_ideal*1e24, 2*d_ideal*1e-6)]) ## [V/um^2]
a0 = a_ideal[0]

norm = np.full((10, 1), 1)
norm[3] = 1e-2
norm[5] = 1e-2
norm[6] = 1e3
norm[8] = 1e-2
norm[9] = 1e3
for a, b, d, i in zip(a_ideal, b_ideal, d_ideal, 1+np.arange(len(t_transport))):
    print(i, 'splitting (a, b):', a, b)
    [xxt, xzt, zzt] =  tiltWithPseudo(volt_gen, R[0], np.radians(15), a0, volt_gen.rf_electrodes, debug=False)
    cons = [[('yy', a), ('yyyy', b), ('yz', 0), ('xy', 0), ('x', 0), ('z', 0), ('y', 0), ('xx', xxt), ('zz', zzt), ('xz', xzt)]]    
    nom_v = volt_gen.findControlVoltages(R, cons=cons, tol=1e-11, bnds=bounds, normvec=norm, fixed_voltages=fixed_v, epss=1e-8, independent=False)
    save_voltage_config(volt_gen, nom_v, os.path.join('/Users/lukeqi/Desktop/School/MIT/UROP/SuperUROP/IonMotion/backwards_ITVG_output/stringer_10V', "NominalVolts_{}.npy".format(i)))


In [ ]:
#### Split from 120um to 180um ####
# t_transport, dx = np.linspace(0, 100, 100, retstep=True) ## [us]
# d_ideal = np.array([d_t(t, (t_transport[0], t_transport[-1]), qspan=(80, 130)) for t in t_transport]) ## [um] (half the separation)
# b_ideal = np.array([bmax*(1-0*qtanhN(t, (t_transport[-1]*0.2, t_transport[-1]), qspan=(0, 1), N=3)) for t in t_transport]) ## [V/um^4]
# a_ideal = 1e-12*np.array([K*d**(-3) - 0.5*b*d**2 for b, d in zip(b_ideal*1e24, 2*d_ideal*1e-6)]) ## [V/um^2]

# norm = np.full((10, 1), 1)
# norm[3] = 1e-2
# norm[5] = 1e-2
# norm[6] = 1e3
# norm[8] = 1e-2
# norm[9] = 1e3
# for a, b, d, i in zip(a_ideal, b_ideal, d_ideal, 500+np.arange(len(t_transport))):
#     print(i, 'splitting (a, b):', a, b)
#     [xxt, xzt, zzt] =  tiltWithPseudo(volt_gen, R[0], np.radians(15), a0, volt_gen.rf_electrodes, debug=False)
#     cons = [[('yy', a), ('yyyy', b), ('yz', 0), ('xy', 0), ('x', 0), ('z', 0), ('y', 0), ('xx', xxt), ('zz', zzt), ('xz', xzt)]]    
#     nom_v = volt_gen.findControlVoltages(R, cons=cons, tol=1e-12, bnds=bounds, normvec=norm, fixed_voltages=fixed_v, epss=1e-8, independent=False)
#     save_voltage_config(volt_gen, nom_v, os.path.join('/Users/lukeqi/Desktop/School/MIT/UROP/SuperUROP/IonMotion/backwards_ITVG_output/stringer_full', "NominalVolts_{}.npy".format(i)))

#### Smoothly transform expansion point ####
# af = a_ideal[-1]
# bf = b_ideal[-1]
# norm = np.full((8, 1), 1)
# # norm[4] = 1e3
# norm[7] = 1e-3
# # norm[1] = 1e-3
# norm[6] = 1e-3
# for i, y0 in enumerate(np.linspace(0, d_ideal[-1], 10)):
#     ind = 1000+i
#     print(ind, 'expansion point:', y0)
#     R = [[0, y0, 50.0]]
#     cons = [[('yy', af + 6*bf*y0**2), ('yyyy', bf), ('yz', 0), ('xy', 0), ('x', 0), ('z', 0), ('y', -2*af*y0-4*bf*y0**3), ('yyy', -4*bf*y0)]]    
#     nom_v = volt_gen.findControlVoltages(R, cons=cons, tol=1e-12, bnds=bounds, normvec=norm, fixed_voltages=fixed_v, epss=1e-8, independent=False)
#     save_voltage_config(volt_gen, nom_v, os.path.join('/Users/lukeqi/Desktop/School/MIT/UROP/SuperUROP/IonMotion/backwards_ITVG_output/stringer_full', "NominalVolts_{}.npy".format(ind)))

#### Double well ####
# R = [[0, -120, 50], [0, 120, 50]]
# for i in range(1500, 1550):
#     dv = 10e-4*(1-qtanhN(i, (1500, 1550), qspan=(0, 0.9), N=4))
#     norm = np.full((18, 1), qtanhN(i, (1500, 1650), qspan=(0, 10), N=4))
#     print(i, 'dv:', dv, 'norm:', norm[0][0])
#     [xxt1, xzt1, zzt1] =  tiltWithPseudo(volt_gen, R[0], np.radians(15), a0, volt_gen.rf_electrodes, debug=False)
#     [xxt2, xzt2, zzt2] =  tiltWithPseudo(volt_gen, R[1], np.radians(15), a0, volt_gen.rf_electrodes, debug=False)
#     cons = [[('yy', a0), ('yz', 0), ('xy', 0), ('x', 0), ('z', 0), ('y', 0), ('xx', xxt1), ('zz', zzt1), ('xz', xzt1)],
#            [('yy', a0), ('yz', 0), ('xy', 0), ('x', 0), ('z', 0), ('y', 0), ('xx', xxt2), ('zz', zzt2), ('xz', xzt2)]]
#     nom_v = volt_gen.findControlVoltages(R, cons=cons, tol=1e-18, bnds=bounds, normvec=norm, fixed_voltages=fixed_v, epss=1e-8, independent=False, dv_weight=dv)
#     save_voltage_config(volt_gen, nom_v, os.path.join('/Users/lukeqi/Desktop/School/MIT/UROP/SuperUROP/IonMotion/backwards_ITVG_output/stringer_full', "NominalVolts_{}.npy".format(i)))   
    

### Double Well Transport

In [ ]:
z_null = 50
t_transport = np.linspace(0, 50, 100)*1e-6
f0 = get_freq_from_alpha(a0, m_=Sr)
q_ideal = np.array([qsta2(t, (0e-6, 50e-6), qspan=(0, 120), f=50e-6*f0) for t in t_transport])
norm = np.full((18, 1), 1)
norm[6] = 1e2
norm[15] = 1e2
t_ = 1e-13
dv_ = 5e-6
q1 = [120+q_ideal, 240+q_ideal, [360]*len(q_ideal)*4, 360-q_ideal, 240-q_ideal, 120-q_ideal, q_ideal, [120]*len(q_ideal)*2]
q2 = [[-120]*len(q_ideal)*2, -120+q_ideal, -q_ideal, -120-q_ideal, -240-q_ideal, [-360]*len(q_ideal)*4, -360+q_ideal, -240+q_ideal]

for q1i, q2i, i in zip(np.concatenate(q1), np.concatenate(q2), 5000+np.arange(len(np.concatenate(q1)))):
    print(i, 'solving at position:', q)
    positions = [[0, q2i, z_null], [0, q1i, z_null]]
    frequencies = [f0]*2
    fixed_volts = []
    volt_set = generate_wells(vg, positions[:], frequencies[:], fixed_v=fixed_volts, tilt=np.radians(15), norm=norm, bounds=(-40, 40), tol=t_, dv=dv_)
    save_voltage_config(vg, volt_set, os.path.join('/Users/lukeqi/Desktop/School/MIT/UROP/SuperUROP/IonMotion/backwards_ITVG_output/stringer_full', "NominalVolts_{}.npy".format(i)))

    

In [ ]:
#     plt.figure('potential')
#     ymin, ymax = -420, 420
#     yrange = ymax-ymin
#     yeval = np.linspace(ymin, ymax, yrange*2+1)
#     ypoints = np.array([np.zeros(yrange*2+1), yeval, np.zeros(yrange*2+1)]).T + [[0.0, 0, z_null]]
#     phi = vg.compute_potential(ypoints, el_config)
#     plt.plot(yeval, phi)
# #     print('final voltages: ', volt_set, '\n')

#     ## target minimum
#     min1 = np.where(yeval==int(positions[0][1]))[0][0]
#     min2 = np.where(yeval==int(positions[1][1]))[0][0]
#     ddphi = np.gradient(np.gradient(phi))
    
#     ## actual minimum
#     minima = find_peaks(-phi)    
#     print('achieved yy: ', [ddphi[m] for m in minima[0]], '\n')
    
#     ## plots target and realized minimum
#     plt.plot([yeval[min1], yeval[min2]], [phi[min1], phi[min2]], marker='x', ls='', color='k')
#     plt.plot([yeval[m] for m in minima[0]], [phi[m] for m in minima[0]], marker='s', ls='', color='r')


### Merge

In [ ]:
#### Merge from 120um #### (assumes that everything used in split is already defined)
volt_gen.set_electrode_grouping(electrode_grouping) ## sets previous voltages to None
t_transport, dx = np.linspace(0, 250, 500, retstep=True) ## [us]
d_ideal = np.array([112 - d_t(t, (t_transport[0], t_transport[-1]), qspan=(2, 110)) for t in t_transport]) ## [um] (half the separation)
t_CP = interp1d(d_ideal, t_transport).__call__(d_CP*1e6)
b_ideal = np.array([bmax if t<t_CP else bmax*(1-qtanhN(t, (t_CP + 0.6*(t_transport[-1]-t_CP), t_transport[-1]), qspan=(0, 1), N=3)) for t in t_transport]) ## [V/um^4]
a_ideal = 1e-12*np.array([K*d**(-3) - 0.5*b*d**2 for b, d in zip(b_ideal*1e24, 2*d_ideal*1e-6)]) ## [V/um^2]

norm = np.full((10, 1), 1)
norm[3] = 1e-2
norm[5] = 1e-2
norm[6] = 1e3
norm[8] = 1e-2
norm[9] = 1e3
for a, b, d, i in zip(a_ideal, b_ideal, d_ideal, 10000+np.arange(len(t_transport))):
    print(i, 'splitting (a, b):', a, b)
    [xxt, xzt, zzt] =  tiltWithPseudo(volt_gen, R[0], np.radians(15), a0, volt_gen.rf_electrodes, debug=False)
    cons = [[('yy', a), ('yyyy', b), ('yz', 0), ('xy', 0), ('x', 0), ('z', 0), ('y', 0), ('xx', xxt), ('zz', zzt), ('xz', xzt)]]    
    nom_v = volt_gen.findControlVoltages(R, cons=cons, tol=1e-11, bnds=bounds, normvec=norm, fixed_voltages=fixed_v, epss=1e-8, independent=False)
    save_voltage_config(volt_gen, nom_v, os.path.join('/Users/lukeqi/Desktop/School/MIT/UROP/SuperUROP/IonMotion/backwards_ITVG_output/stringer_full', "NominalVolts_{}.npy".format(i)))


# Protocol Exploration

## STA Transport

In [29]:
t_anal = np.linspace(0, 20E-6, 10000)
a_final = np.full(len(t_anal), get_alpha_from_freq(1.2e6, m_=88*amu)*1e-12) ## V/um^2
q_final = np.full(len(t_anal), 0)
tstart = 1e-6

hbar = ct.hbar
eps0 = ct.epsilon_0
k0 = 1/(4*pi*eps0)
amu = ct.m_u
elem_charge = ct.e
um = 1E-6

Nion = 1
q_ions = elem_charge*   np.full(Nion, 1)
m = amu*                np.array([88])
x0dot = um*             np.full(Nion, 0)
x0guess = um*           np.array([q_final[0]])
x0 = find_ic(q_final[0]/1e6, a_final[0]*1e12, Nion, x0guess) ## must be in SI units
print(x0*1e6)

tanh = []
for T in [n/10*1E-6 for n in range(10, 150)]:
    t_anal = np.linspace(0, T + tstart, 10000)
    heat_start = t_anal[-1]
    heat_end = t_anal[-1]+25e-6
    t_eval = np.linspace(heat_start, heat_end, 4096)
#     q_final = np.array([qtrig(t, (tstart, tstart+T), f=1.2e6, mu=1) for t in t_anal])
#     q_final = np.array([qsta2(t, (tstart, tstart+T), f=1.2e6*T) for t in t_anal])
    q_final = np.array([120*(min(max(tstart, t), tstart+T)-tstart)/T for t in t_anal])
    outsol, n, y_eval, q_eval = heat_sim(t_eval[-1], q_final, t_eval, showplot=False, ms=1e-8, a=a_final, deg=1)  

    if Nion > 1:
        freq_modes = [1060266.7621143984, 1.91e6]
        N = 2**16
        dx = t_eval[1]-t_eval[0]
        ion_w = [np.fft.rfftfreq(N, dx)]*Nion
        ion_fft = [np.fft.rfft((y_eval[i]-q_eval) - np.average((y_eval[i]-q_eval)), N) for i in range(Nion)]
        windows = [gaussian(ion_w[0], freq_modes[i], 0.01e6) for i in range(Nion)]

        ion_heat = []
        for ion in range(Nion):
            ion_heat.append([get_heat(np.fft.irfft(ion_fft[ion]*windows[j], N)[:len(t_eval)], 2*pi*freq_modes[j], m_=m[ion]) for j in range(Nion)])

#         plt.figure('freq domain')
#         plt.plot(ion_w[0], np.abs(ion_fft[0]))
#         plt.plot(ion_w[1], np.abs(ion_fft[1]))
        print(T*1e6, *ion_heat, sep=': ')
        tanh.append(ion_heat)
    else:
        print(T*1e6, n, sep=': ')
        tanh.append(n)
    

[-5.85422894e-08]
1.0: 1828476.5249640301
1.1: 3118093.9932664176
1.2: 3546327.2856468116
1.3: 3021603.4672222733
1.4: 1925003.7653413955
1.5: 812664.3209688087
1.5999999999999999: 127861.59292361983
1.6999999999999997: 28766.44339805597
1.7999999999999998: 379153.93602281314
1.9: 870405.2846419503
2.0: 1196795.8610823527
2.0999999999999996: 1195345.3127510282
2.2: 895240.9609673193
2.2999999999999994: 468838.8271214142
2.4: 124533.84731534956
2.5: 0.0003044000884173032
2.6: 106096.62045367541
2.7: 340206.90284649946
2.8: 552658.3428094677
2.9: 626839.6345650798
3.0: 531938.2810241345
3.1: 326965.3725204113
3.1999999999999997: 119956.65336734774
3.3: 7634.0840532246275
3.3999999999999995: 28313.825940492854
3.5: 149270.2626571394
3.5999999999999996: 291125.82761045796
3.7: 373035.5813215758
3.8: 353627.5949098298
3.9: 248082.15437178573
4.0: 114280.24970020459
4.1: 19473.144333940185
4.199999999999999: 4712.809473627739
4.3: 66432.65670028816
4.4: 162296.3495473096
4.5: 236404.11598537

In [30]:
# print([s[0][0] for s in tanh])
# print('\n')
# print([s[0][1] for s in tanh])
# print('\n')
# print([s[1][0] for s in tanh])
# print('\n')
# print([s[1][1] for s in tanh])

print(tanh)

# print('')

[1828476.5249640301, 3118093.9932664176, 3546327.2856468116, 3021603.4672222733, 1925003.7653413955, 812664.3209688087, 127861.59292361983, 28766.44339805597, 379153.93602281314, 870405.2846419503, 1196795.8610823527, 1195345.3127510282, 895240.9609673193, 468838.8271214142, 124533.84731534956, 0.0003044000884173032, 106096.62045367541, 340206.90284649946, 552658.3428094677, 626839.6345650798, 531938.2810241345, 326965.3725204113, 119956.65336734774, 7634.0840532246275, 28313.825940492854, 149270.2626571394, 291125.82761045796, 373035.5813215758, 353627.5949098298, 248082.15437178573, 114280.24970020459, 19473.144333940185, 4712.809473627739, 66432.65670028816, 162296.3495473096, 236404.11598537775, 249118.25686760322, 196144.8973433139, 107648.35582308413, 29870.78488286253, 9.342158652817093e-05, 27575.66485092612, 91716.39624793919, 154264.0686422705, 180771.04673336982, 158243.54084110158, 100197.41789826524, 37808.20944401516, 2471.29918624975, 9402.872061480068, 50790.07770368713

In [ ]:
#### Diabatic transport of Sr-Ba Plot ####
from matplotlib.lines import Line2D

Tvec = np.array([n/10 for n in range(10, 150)])
Sr_STA_C = [1376.8037438139565, 878.3142269253716, 509.96908611619614, 291.43028411286593, 165.9923630000904, 81.73819351222448, 32.602946955072426, 11.581897471657669, 6.314771041172429, 4.267032574061701, 3.233683032628027, 2.058771657122317, 0.9675676923244312, 0.27712252322793657, 0.26809867730769116, 0.14264538987387082, 0.09464571427919234, 0.09796169522380635, 0.09918852990711088, 0.08603192312629927, 0.023527203118050753, 0.005120244484239915, 0.014763516108940184, 0.033022231954350625, 0.047647350621114117, 0.02248009924172133, 0.002487364250054413, 0.0020412477406059172, 0.010508959842003084, 0.02011539725397691, 0.009526317023469155, 0.001636719303671646, 0.0008194907366178937, 0.003983259307199997, 0.010539711436807845, 0.007502558706221017, 0.0022356417361263947, 4.521123119794693e-05, 0.0011457492809523195, 0.004569859985411941, 0.0044523920442831495, 0.0016496126352709061, 7.389450956879639e-06, 0.0005206925982659072, 0.0022673785020978993, 0.0029859365649185328, 0.0013247633658962034, 9.552967779492313e-05, 0.00019595200706140766, 0.0009692087685050238, 0.0017646456516771474, 0.0009006098176081844, 0.00016847383051456788, 5.9897480104205285e-05, 0.00043360319219145087, 0.0010767395203479606, 0.0007179795253896873, 0.00020498384006896385, 9.048605674806648e-06, 0.00016842644004011215, 0.0006141691783909801, 0.0005660536894763368, 0.00021582651563044024, 6.117109038942173e-07, 7.009072648927526e-05, 0.0003426364822863773, 0.0004219860070220782, 0.00018750453290498353, 9.306573588743641e-06, 3.555882934411608e-05, 0.0001826493004256991, 0.00030625707401956326, 0.00015823758638687169, 2.467880760161297e-05, 1.3042535710718495e-05, 9.016011593334428e-05, 0.00020604277008006816, 0.00012587930543919061, 3.534172393487674e-05, 2.9323702711637183e-06, 4.4369328855861e-05, 0.0001381196035001585, 0.00011475618996502275, 4.122096127936037e-05, 4.7991318931265124e-08, 1.8448474062388578e-05, 8.417998643759565e-05, 9.63611191074809e-05, 4.2714492144916316e-05, 1.5208703076553937e-06, 8.870986777009422e-06, 5.001850608986165e-05, 7.581169188368788e-05, 3.7687733402364345e-05, 4.213331961689087e-06, 4.5922909967540135e-06, 2.774628237099102e-05, 5.8175910240539136e-05, 3.311841886658777e-05, 8.446584551438655e-06, 1.1603920067748752e-06, 1.390440957050335e-05, 3.978842079367274e-05, 3.0269273142903208e-05, 1.027648375890666e-05, 1.5775675891133196e-07, 7.420346508216799e-06, 2.8541276839491974e-05, 2.9362282421360784e-05, 1.2395047406539112e-05, 1.9093965730879286e-07, 3.290428046020601e-06, 1.685436913672591e-05, 2.361272916307427e-05, 1.1752713878415758e-05, 1.06073359574778e-06, 1.6448051352397213e-06, 1.1160003550486819e-05, 2.023359637087348e-05, 1.084721263532933e-05, 2.078370397291552e-06, 7.423011467010597e-07, 5.574347398049861e-06, 1.4626024019599391e-05, 1.0305489149793682e-05, 3.5075893602336642e-06, 9.070490876229313e-08, 3.2936491345784996e-06, 1.093999526766576e-05, 1.0084194579129665e-05, 3.713906581067515e-06, 2.9351032499929668e-08, 1.6034800100777473e-06, 7.429647189575799e-06, 9.439526642394737e-06, 4.679761169237863e-06, 3.3895002598021653e-07, 6.23127605184923e-07, 4.60206366152061e-06, 7.61231535004286e-06]
Sr_STA_S = [10926.773362767879, 12524.611255097385, 13197.703535075472, 8698.097086514941, 2589.8996473857187, 612.7755818997748, 1247.668242605627, 1785.9969063810224, 1387.6418749846723, 556.9788839121646, 50.2997093873695, 58.86972073144911, 248.93722639829647, 298.6225391294821, 173.89361352874224, 34.94888716638271, 2.9141919868223938, 50.923955763867966, 86.32793877856203, 66.06436623959559, 21.27106175746913, 0.08098244552233838, 11.527916702886726, 28.591792739806095, 27.77003872456427, 12.354048799479392, 0.8148812693532521, 2.5550213580893195, 10.280778799814222, 12.558511843440447, 7.199775463861585, 1.1410455453607378, 0.4558029470734433, 3.8600254827958818, 5.976998876236951, 4.25755059894652, 1.1065848474215294, 0.032906392636356106, 1.456187791481876, 2.937073417754693, 2.554667607320779, 0.9352801120355543, 0.0074505000225902175, 0.5283339522690893, 1.46907111640323, 1.5485245627495587, 0.7395198195200167, 0.05231024393938834, 0.1725940327428967, 0.7359775022510535, 0.9439235903565417, 0.5631812116757161, 0.0896263496040837, 0.04414417841549774, 0.3635052324260114, 0.5759340236220752, 0.41875271292818395, 0.10759531399763053, 0.005431395461491424, 0.17358794872731279, 0.3494022376215636, 0.3056964382447238, 0.11003646469483541, 0.0002767399885822575, 0.077917011619548, 0.20963178562185628, 0.21959425348553413, 0.102767863011789, 0.005700851999347748, 0.0312776795769647, 0.12316522389337033, 0.15527601983452244, 0.09051150294186983, 0.012668506220847354, 0.010138119874995556, 0.0702873990020463, 0.10799070577247892, 0.07650327723802265, 0.01803562084460373, 0.0019444586303613218, 0.03837173772151741, 0.0736748396356708, 0.06255381372286092, 0.02109425439831629, 5.9260818267061365e-06, 0.01960994825448035, 0.04914196876089492, 0.04974991709614854, 0.02206227098037209, 0.0007798452391568084, 0.009039379511726117, 0.03187506944396366, 0.038585004117943376, 0.021445766249139398, 0.002441654111751261, 0.0034796218118366097, 0.019957767480800897, 0.02918781509560784, 0.019765581567465353, 0.00410126470283367, 0.0009147410462887939, 0.011920913306069678, 0.021524049048712267, 0.017485119730579313, 0.005377777444949156, 5.729602212487767e-05, 0.006681175760490014, 0.015438530907327691, 0.014932056017976997, 0.006159195319842939, 0.00010201847452897247, 0.0034096786763216113, 0.010729215156762617, 0.012362662443281328, 0.0064623864401097495, 0.0005545784349325572, 0.0014958689900361583, 0.007184366322795941, 0.009937187549672679, 0.006369470037735979, 0.0011231562299005252, 0.0004917121659765, 0.004590901306569088, 0.007760092567955972, 0.0059819044500718606, 0.0016456629919714455, 7.257418478143335e-05, 0.002760573279918557, 0.005876495994498844, 0.0053987647157948985, 0.0020439998462036977, 7.143139375295075e-06, 0.001525374332342534, 0.004304618883486587, 0.004700807338100584, 0.002291413713947296, 0.00013423817548829356, 0.0007407695390060813, 0.003036276411931122, 0.003963701830612299]
Ba_STA_C = [3699.364831064117, 2080.427926037481, 1122.4186371809576, 637.9965652302865, 370.0550751159602, 183.47440194189994, 73.04642638692641, 25.90745417023512, 11.779150286340066, 8.949799999896252, 7.267960647926467, 4.627742229295187, 2.2890846826093205, 0.9974129380915667, 0.21418008589260715, 0.09525407277995038, 0.21269634996187392, 0.2235155554857601, 0.20878123379881744, 0.13265138940745091, 0.048497171533359, 0.011505802644574762, 0.0168003265981944, 0.03580664204264295, 0.033246767646766706, 0.009720314662707076, 0.0011937349177788563, 0.003953979145078387, 0.011852173894235731, 0.016931855444464988, 0.008367719370352179, 0.001785173788818006, 0.00029231686615187774, 0.0026925359241717437, 0.005854190100752653, 0.0036554022950556746, 0.0009546403603280363, 9.224081686775174e-05, 0.00101766690887803, 0.0030276713030048374, 0.002717884972771361, 0.000991052934653505, 1.5331751893771108e-05, 0.00022979355741646018, 0.0011580623721308579, 0.0014652627248633093, 0.0006373291942392149, 4.754645015918661e-05, 9.221946478422778e-05, 0.0005586915217589018, 0.0009437120992123305, 0.00046519840263428843, 8.558002869178113e-05, 2.4605487259461948e-05, 0.0002185533265200554, 0.0005237594449357851, 0.0003483358540433595, 0.00010525110278783769, 3.0115727737792795e-06, 9.692201852371006e-05, 0.0003034588959036704, 0.0002718307923444381, 9.719424558126584e-05, 1.0902743643063295e-07, 3.153116397108934e-05, 0.0001641740180184988, 0.0002047385705783534, 9.22267957706104e-05, 6.051778946233916e-06, 1.2831111397749479e-05, 8.45564345166867e-05, 0.0001405885510035876, 6.959582323843729e-05, 1.0723626699331214e-05, 5.301070556163895e-06, 4.343232294230666e-05, 9.855508007576337e-05, 6.17739907573637e-05, 1.6543452180236635e-05, 1.128022500314112e-06, 1.8119031419197275e-05, 6.025215828805788e-05, 5.2424766562188595e-05, 1.9290963447365134e-05, 8.18898615756827e-08, 8.542182671921879e-06, 3.901892380897745e-05, 4.3917026953235806e-05, 1.8619314170082208e-05, 5.32226697581497e-07, 4.166956748795263e-06, 2.2415659401359065e-05, 3.526155122022291e-05, 1.8202722081899847e-05, 2.5448460418094092e-06, 1.3804254004028925e-06, 1.2435332751198676e-05, 2.5093826346236343e-05, 1.3741125775968455e-05, 3.253410838946779e-06, 6.328090541889852e-07, 6.8219701419723075e-06, 1.9464311028644174e-05, 1.510786210747428e-05, 5.260340166828402e-06, 1.2521116268362092e-07, 2.495767321270021e-06, 1.1436358549416303e-05, 1.202777186948376e-05, 5.003021420569724e-06, 7.503388148834715e-08, 1.5539131199699456e-06, 8.617891294273637e-06, 1.1848850370580995e-05, 5.614062279515505e-06, 4.918589071270805e-07, 7.647240410544946e-07, 4.092606086379535e-06, 8.310173986979987e-06, 4.850216814107752e-06, 1.0992674570786513e-06, 2.0589636057760938e-07, 2.9481393881040008e-06, 7.140941791129888e-06, 4.705867063268877e-06, 1.2654819636042455e-06, 1.1210357327115787e-07, 1.105516715880775e-06, 4.7033139218740545e-06, 4.555301639535501e-06, 2.1420435133943004e-06, 7.073108776975261e-08, 6.655619720233111e-07, 3.2228090018383803e-06, 3.936217869676326e-06, 1.6341338978444842e-06, 6.250752904828092e-08, 4.901210209969244e-07, 2.165541677102134e-06, 3.819049769946451e-06]
Ba_STA_S = [5250.745422888203, 6011.820124182948, 6116.423900210286, 3941.893185506469, 1158.3275362965715, 273.0259980372377, 556.5683496489112, 797.5241136022098, 619.1698385267397, 248.13374406979744, 22.387021971878326, 26.20170236951759, 110.83664773302989, 132.97097500080827, 77.41326538024107, 15.554317637399281, 1.2969090336671292, 22.664851064688722, 38.42476324773589, 29.404272380305663, 9.46663325433316, 0.03603959087248482, 5.130376179978403, 12.7248825040491, 12.35913868447922, 5.498046023013798, 0.36264685611024894, 1.137065658757179, 4.575339962669339, 5.589042454260496, 3.204156923816164, 0.5078001548659596, 0.20284598268861978, 1.7178381873788306, 2.659971590211521, 1.8947513559520313, 0.4924640582354222, 0.014644320479668594, 0.6480484739467549, 1.3070919092445596, 1.1369082325645934, 0.4162280832934485, 0.003315693204656689, 0.23512448982342452, 0.653781973765869, 0.6891413052956294, 0.32910868061575127, 0.023279609884171295, 0.07680948071218573, 0.32753223961518396, 0.42007470107923206, 0.25063258420590184, 0.03988638643768292, 0.01964546828732778, 0.16177070450268818, 0.2563079755113201, 0.1863575058604227, 0.04788311080024896, 0.002417131817297489, 0.07725180265194324, 0.15549444555227818, 0.1360440464653327, 0.04896949558119173, 0.00012315749119164946, 0.034675382669220774, 0.09329238961126583, 0.09772598643267076, 0.045734751386841126, 0.002537048014206085, 0.013919494464945604, 0.05481218487271734, 0.06910244774163365, 0.04028030605812549, 0.005637860567053174, 0.004511763674296558, 0.0312799781591154, 0.04805907169407624, 0.03404622884247812, 0.008026385648823948, 0.0008653417100869716, 0.017076560840110746, 0.03278748998257867, 0.027838302848697122, 0.00938756824939039, 2.637282288420977e-06, 0.008727008009992534, 0.02186963342152709, 0.022140188487214787, 0.009818364326062114, 0.0003470542253629891, 0.004022791647447654, 0.014185350687556974, 0.017171470714028185, 0.009544001442332879, 0.0010866084160259303, 0.001548534823296202, 0.008881797879536265, 0.01298944279091099, 0.008796269457911907, 0.0018251843033194244, 0.0004070868537651451, 0.0053051595670102055, 0.00957883968248933, 0.007781396314917876, 0.0023932702664474464, 2.549842682846906e-05, 0.0029733211161395486, 0.0068706036660173084, 0.006645207311612308, 0.0027410243676145364, 4.5401243157088134e-05, 0.0015174080955223563, 0.004774818575132556, 0.005501751017866939, 0.0028759533999011227, 0.00024680382953827794, 0.0006657060334635832, 0.003197255826042901, 0.004422342815497736, 0.002834602836353343, 0.0004998377797894223, 0.00021882647812583876, 0.0020430871009084507, 0.0034534710544015914, 0.0026621246756184046, 0.0007323688222971442, 3.229766668993822e-05, 0.001228536914483463, 0.0026152147925837567, 0.0024026102281872867, 0.0009096405080098322, 3.178908706147302e-06, 0.0006788367744107374, 0.0019156829402108926, 0.002091998582083531, 0.0010197470064863062, 5.9739963508614024e-05, 0.0003296643904008599, 0.0013512329793863777, 0.0017639647607476758]
Sr_STA2_C = [1400.8913910442234, 2302.9555937460723, 1281.5508623751648, 850.1247416034955, 539.1950916470553, 359.34854481197203, 242.7839779520983, 162.22345002712729, 105.3293473185646, 65.92314239210891, 40.59573003619105, 25.533514864933828, 16.920455069214782, 11.832092814229503, 8.48733110565753, 6.017342742846517, 4.099617377015711, 2.6435926657682276, 1.6069795846746389, 0.924756638094322, 0.508840384349943, 0.2711357976408971, 0.1411104025442544, 0.07150243368945487, 0.034532688780172925, 0.01525662202717562, 0.005794658678376772, 0.0018659731404769779, 0.0005558171454676056, 0.00019472491402557673, 0.00030698983061882423, 0.00039163319091476025, 0.00040520615090950253, 0.00034194812819191635, 0.00021869064333227957, 0.0001347533976192385, 7.357820086384595e-05, 3.446722239631137e-05, 3.2280744474666216e-05, 4.010683766637996e-05, 2.032062909744537e-05, 1.9895256476586974e-06, 3.0391668413845156e-06, 8.820772364902668e-06, 8.520133943856205e-06, 8.035658223324912e-06, 3.317695193801965e-06, 7.29926030164083e-07, 1.338119958223494e-06, 3.846193550118376e-06, 4.21502846714585e-06, 1.2237410687355386e-06, 1.1240848291179633e-07, 6.562866532847501e-07, 1.3308930025357914e-06, 2.1984571849863634e-06, 1.1047795625642118e-06, 1.930449143169041e-07, 6.89014028939068e-08, 3.108137114553302e-07, 7.687062538847828e-07, 4.762099341649856e-07, 1.1019775328002008e-07, 4.581675302189559e-08, 2.1536655008804644e-07, 5.537337174591043e-07, 4.779659640118636e-07, 1.371392964918328e-07, 2.0828493630503316e-08, 8.886732821717263e-08, 1.2961016629388944e-07, 1.4116725810082362e-07, 1.0223620630195135e-07, 3.355972582067146e-08, 4.09233864639391e-08, 1.2855920009893587e-07, 1.7012918550305595e-07, 6.772595278943348e-08, 2.7472664701236597e-08, 4.325000460305201e-08, 6.359577761923931e-08, 8.187511264544856e-08, 7.808431976054119e-08, 3.944498123818288e-08, 3.0150402243655616e-08, 2.9606076584766333e-08, 4.766166487363825e-08, 2.9227704310847444e-08, 2.958291024385835e-08, 3.0020635797857504e-08, 4.128066252093187e-08, 4.8696506332029296e-08, 5.7146859816007355e-08, 4.388808640341183e-08, 3.03166695152178e-08, 3.0134029962158174e-08, 3.0048620199791684e-08, 3.023902977620714e-08, 3.029558706303631e-08, 3.239833676477917e-08, 3.129732307902981e-08, 3.923458641558225e-08, 4.948564525298798e-08, 3.8676650029107953e-08, 3.05617302856859e-08, 3.170575339651173e-08, 3.0394383406453234e-08, 3.0435227345576926e-08, 3.048542500875863e-08, 2.8631139750373278e-08, 3.142011843962243e-08, 3.243160316582763e-08, 3.97752218492533e-08, 3.4395814175243076e-08, 3.0435006525157955e-08, 3.053584390574254e-08, 3.060770444590249e-08, 3.0568371966481377e-08, 3.0613267864093885e-08, 3.064755621775234e-08, 3.1592801093105607e-08, 3.064808468330281e-08, 3.1698636374378797e-08, 3.0688242851955275e-08, 3.061364942115613e-08, 3.061857152459234e-08, 3.103300253189302e-08, 3.22320775782498e-08, 3.064748628801806e-08, 3.065522647924251e-08, 3.2078254100234345e-08, 3.0657241953914213e-08, 3.064060806457879e-08, 3.0741227715367214e-08, 3.059889594068087e-08, 3.064935485528936e-08, 3.090403970980981e-08, 3.0908385765400076e-08, 3.0702545479454924e-08, 3.08934695824638e-08]
Sr_STA2_S = [8375.598557378593, 8996.197000481412, 9602.522576220597, 11961.89017891672, 13215.456956030035, 12550.460742589281, 8809.075360523037, 4507.494263970576, 1724.308724778797, 489.2181779103263, 116.6933852030461, 56.15657321889344, 48.60431811158309, 30.996873230643914, 12.686174436548441, 4.885468113080435, 4.931708932900357, 5.579309325841096, 3.8466396429534573, 1.3001911598156914, 0.05177215802773943, 0.30340734987381457, 0.903014040021097, 0.9560854294585448, 0.5133676180795426, 0.09814264501783983, 0.007316956913873685, 0.12561916896671452, 0.2047807206020589, 0.15127852590217775, 0.04838591440288837, 0.0004541608733128952, 0.020299623577924034, 0.051192648066154416, 0.04930091983161376, 0.022242530218608857, 0.0019314951003865544, 0.002989617940823462, 0.013709617362682162, 0.017095288575181417, 0.010084545559575463, 0.0019053552088562607, 0.00028944456520547094, 0.00379848809401699, 0.006228718264164139, 0.004625652106256929, 0.001374000776340015, 1.140306112700061e-06, 0.0010462018459546432, 0.0023521769436436963, 0.002160295705236031, 0.0008829465230748284, 3.412616509837248e-05, 0.00027126286999538783, 0.0009091654836083315, 0.0010278819874914452, 0.0005401791970226908, 6.488710752324526e-05, 5.988812614192119e-05, 0.00035495076799688005, 0.0004973787369233741, 0.00032356137829586137, 6.918398752229276e-05, 8.648282143386492e-06, 0.00013796955223691206, 0.00024405971786591157, 0.00019234542852060798, 5.9912128783803884e-05, 3.4707923056504617e-07, 5.2431389095865404e-05, 0.00012109701700678294, 0.00011431178823263803, 4.708538573113839e-05, 1.582164385841116e-06, 1.899327750766867e-05, 6.0564302620791075e-05, 6.822693351802202e-05, 3.5192009261161944e-05, 3.625870791132449e-06, 6.314788531365883e-06, 3.0433930977487814e-05, 4.100255602457391e-05, 2.5633107013274808e-05, 4.767506782923319e-06, 1.8366132931942788e-06, 1.530825303540052e-05, 2.4861346337671644e-05, 1.845782307265302e-05, 5.058359238925317e-06, 5.063446405868954e-07, 7.684610520606095e-06, 1.5198626395863111e-05, 1.3259028294232887e-05, 4.842197633544855e-06, 3.098059533276282e-07, 3.844209868832185e-06, 9.433253876512418e-06, 9.55806498897389e-06, 4.392140598728884e-06, 4.650839909439027e-07, 1.927939487621947e-06, 5.9118640695535785e-06, 6.930739228857366e-06, 3.8724222060268965e-06, 6.851704404926013e-07, 9.971569631073138e-07, 3.75090147205482e-06, 5.085647606353277e-06, 3.3652386976992876e-06, 8.763789192713308e-07, 5.748686291408598e-07, 2.415221936413975e-06, 3.764578105775082e-06, 2.9048547108251195e-06, 1.0173117785398161e-06, 4.1280206894237543e-07, 1.5867636439521679e-06, 2.8154558104452572e-06, 2.503567577322955e-06, 1.108892681595841e-06, 3.834027303155124e-07, 1.075402963242606e-06, 2.127649029327903e-06, 2.1504976346349046e-06, 1.1592924925486145e-06, 4.186455673721547e-07, 7.659513909800873e-07, 1.6270537989396066e-06, 1.848558253706819e-06, 1.176497803409091e-06, 4.820045518997482e-07, 5.870358911111823e-07, 1.260058794213779e-06, 1.5890854000634444e-06, 1.1675845358831112e-06, 5.523340120173586e-07, 4.932366174516128e-07, 9.940958433318036e-07, 1.3665839858285492e-06, 1.1375370723387131e-06]
Ba_STA2_C = [3540.834075749202, 6643.583474252463, 3543.328754786391, 2013.931395882556, 1218.2366823388504, 785.604611337314, 530.1715291163115, 359.9258082137586, 235.68368165189597, 148.0312363187909, 91.23648917289113, 57.39037388684134, 38.029385197680135, 26.593069348456016, 19.075030650364134, 13.522957415847973, 9.213203809308979, 5.940996406958105, 3.61137268871655, 2.078112210080081, 1.143392892253951, 0.6092725118167123, 0.31709938454544867, 0.16068000303398033, 0.07760151750238048, 0.03428317764277007, 0.013020804429676066, 0.0038824279399937676, 0.0008867588226238122, 0.0006218283492877904, 0.0006964692401959189, 0.0008800162677436037, 0.0008561744622938592, 0.0006914515052361458, 0.0004885673330258854, 0.0003030209817286646, 0.0001653344046369933, 7.57765341446972e-05, 2.634039554550027e-05, 6.055018782659771e-06, 7.863944730600432e-06, 4.6319363020312405e-06, 5.2409534416164954e-06, 6.755067559534086e-06, 9.177271722368878e-06, 7.640335888449492e-06, 3.3241683884654556e-06, 1.6401748798084527e-06, 9.361129081041586e-07, 1.0648477205961285e-06, 1.9837356681242173e-06, 1.0061105577859825e-06, 2.755719572642855e-07, 3.187593945494227e-07, 4.391978358923451e-07, 7.659466655894819e-07, 2.901193701266377e-07, 4.894096324655503e-08, 7.864211638255898e-08, 2.4085875851909837e-07, 5.689363219156244e-07, 4.2048641743299954e-07, 1.303029921760899e-07, 9.248235550440488e-08, 8.30587236166087e-08, 7.947829598846292e-08, 8.561076472689166e-08, 5.1855031759416974e-08, 4.680244989535385e-08, 7.506637911104739e-08, 1.6861093573938065e-07, 2.8939081945700094e-07, 8.422684590943906e-08, 7.54100598508658e-08, 7.613401754954445e-08, 7.734300313809562e-08, 7.035325992185815e-08, 7.268532847851466e-08, 6.143386974937752e-08, 6.431450957949236e-08, 6.892475185022984e-08, 7.08317329725137e-08, 6.988013606980601e-08, 6.908807818301364e-08, 6.78082408117076e-08, 7.231117418617338e-08, 7.06182178516048e-08, 7.407179035285452e-08, 6.809799024592251e-08, 6.745395693386108e-08, 6.924598468886215e-08, 6.692679695646029e-08, 6.914427165179012e-08, 6.873654574065775e-08, 6.812272657494216e-08, 6.772011128207488e-08, 6.791254204196283e-08, 7.188519883011123e-08, 6.931713635636872e-08, 7.28005726396254e-08, 7.008470651545988e-08, 6.93455770989967e-08, 6.878915054022627e-08, 6.87407920568979e-08, 6.853204989653465e-08, 7.124415245397952e-08, 6.856075647579977e-08, 7.065726759936566e-08, 6.853641550459601e-08, 6.433537304461762e-08, 7.060236120277883e-08, 6.892644405180616e-08, 6.886381181659592e-08, 6.875256377236302e-08, 6.838834006749706e-08, 6.861498298318239e-08, 6.858502314057852e-08, 6.882166945188704e-08, 6.879083921551191e-08, 6.88663132037787e-08, 7.099034033972401e-08, 6.886744621693991e-08, 6.881704129530445e-08, 6.723932927901845e-08, 6.876148453275095e-08, 6.877093100929112e-08, 6.886382616735525e-08, 7.195863145515288e-08, 6.886135090761158e-08, 6.88830015561776e-08, 7.208137901874094e-08, 6.888808868851709e-08, 6.885170523377519e-08, 6.907678325772489e-08, 6.875690534841901e-08, 6.883296587059657e-08, 6.943787095668895e-08, 6.887393668785369e-08, 6.889190881212143e-08, 6.895262485720712e-08]
Ba_STA2_S = [4769.405402010024, 4782.139688364276, 5057.718247081537, 5665.966331931857, 6219.359155356218, 5774.272399863785, 3993.335180980711, 2023.6129744448372, 769.9035513196719, 217.92241107211353, 51.94443470611953, 24.994350532070516, 21.632564457923653, 13.795444442043513, 5.645940640562527, 2.1918931804428543, 2.1947876135812723, 2.4829916954344458, 1.7118849467636086, 0.5786273979827499, 0.025969491070096638, 0.13502702239241554, 0.4018692916943457, 0.42548717644188033, 0.22846403834758516, 0.04367638719941037, 0.0032562632363603086, 0.05590427039209226, 0.09113352157323726, 0.06732344576060974, 0.021533166781591735, 0.00020260120511143511, 0.009033937136581542, 0.022782242065811115, 0.021940371412719963, 0.009898589447571646, 0.0008595725108777033, 0.0013304685921759441, 0.006101185879560136, 0.007607909759031571, 0.004487921501212853, 0.00084793952590326, 0.00012881139734983343, 0.001690439819140606, 0.002771964128188313, 0.0020585522389137564, 0.0006114710764185198, 5.07469751430513e-07, 0.00046559081190288617, 0.0010467884427976402, 0.0009613956069125231, 0.0003929373495875387, 1.5187155248657946e-05, 0.00012072000971502677, 0.00040460556777946943, 0.00045743792751758595, 0.00024039574693205212, 2.88766845231789e-05, 2.6651989498704154e-05, 0.0001579636026347525, 0.0002213482716955619, 0.00014399439571101616, 3.0788923258196997e-05, 3.848742856011816e-06, 6.140053664339929e-05, 0.00010861379494546061, 8.559941188128183e-05, 2.6662671893816805e-05, 1.5446053440342218e-07, 2.3333519040348284e-05, 5.389175450295944e-05, 5.0872135193355124e-05, 2.0954391482440598e-05, 7.041093544467634e-07, 8.452570766621886e-06, 2.6952908200463313e-05, 3.0363005940471013e-05, 1.5661483909752365e-05, 1.6136202286221887e-06, 2.8102666352365265e-06, 1.3543999942619728e-05, 1.824735119490894e-05, 1.1407491954060857e-05, 2.121683222938272e-06, 8.173472441870764e-07, 6.812625806546619e-06, 1.1064035243537114e-05, 8.214277271891338e-06, 2.2511196425486514e-06, 2.2533832806666065e-07, 3.419879258087067e-06, 6.763838797473604e-06, 5.9006602804809724e-06, 2.1549213012360274e-06, 1.3787275523061326e-07, 1.7107871390527302e-06, 4.19807743153767e-06, 4.253621712554788e-06, 1.954632233588925e-06, 2.0697611259211472e-07, 8.579902083055436e-07, 2.630954639288437e-06, 3.0843840690720874e-06, 1.7233426286060696e-06, 3.049210720782064e-07, 4.4376439447790584e-07, 1.6692623025040523e-06, 2.2632646417873908e-06, 1.4976307726575422e-06, 3.9001456235235127e-07, 2.558335760586153e-07, 1.074845419599854e-06, 1.6753478248279191e-06, 1.2927463389068537e-06, 4.527338571001303e-07, 1.8370925555134957e-07, 7.061567408022278e-07, 1.252961242384061e-06, 1.1141630639053073e-06, 4.934900849038111e-07, 1.706255779609582e-07, 4.785861707431095e-07, 9.468669234196075e-07, 9.570342588453155e-07, 5.159193941663703e-07, 1.863096926601271e-07, 3.4087109071001387e-07, 7.240875519820726e-07, 8.226638002861696e-07, 5.235769464875379e-07, 2.145059974516669e-07, 2.6124838914784904e-07, 5.60763419887752e-07, 7.071900537013048e-07, 5.196096366609851e-07, 2.458050298548553e-07, 2.1950482706556402e-07, 4.4240209205311385e-07, 6.081703161746011e-07, 5.062377098843223e-07]
Sr_tanh_C = [17206811.284428254, 30773602.90036665, 42146208.94262819, 21790093.163839377, 6585780.879139952, 13945167.766886275, 24979953.865233622, 12618842.733012823, 18011033.03158824, 16171427.73219839, 13947452.364641264, 12372439.371169763, 9674841.935184008, 9794384.035136374, 8365655.607678908, 7441356.5197086735, 7650316.739187256, 6102305.632725132, 8732522.971025696, 5511142.449563541, 3885445.8641240424, 3186518.619154943, 1970966.4318439625, 2238801.5316160326, 2016949.0590376866, 1456237.3651428076, 1516167.5226873825, 1365873.5237098231, 1114476.9752954994, 1249488.2111629369, 1091836.7304474113, 788597.6255203483, 411580.8058212698, 405183.4645941267, 437015.0242147153, 383015.341073917, 333656.0376564938, 283804.9714324911, 238983.44505111992, 200473.6447709472, 167694.7818689236, 139998.33970618196, 116859.09099222148, 97601.2171244019, 81560.85092406829, 68195.89480191171, 57113.21982088677, 47890.51124924664, 40221.87817080638, 33810.10611568859, 28445.78028024111, 23939.839719211504, 20139.6331688406, 16925.02351996107, 14198.45985875121, 11882.912198342663, 9918.151646509223, 8253.641600293624, 6848.90454091069, 5669.247774988664, 4683.974876317471, 3866.473457155202, 3192.9712525591817, 2641.3574359409504, 2191.9960596990372, 1827.0398915824555, 1530.7410782246986, 1289.3718885761346, 1091.1950913988715, 926.58553183555, 787.8656631460923, 669.0872021832984, 565.9987263775314, 475.7317658355038, 396.3803427195213, 326.86527604675143, 266.50448322436705, 214.85525550800213, 171.4670941828016, 135.81460047462673, 107.21064940774207, 84.8581691478753, 67.84457680138021, 55.24111358110735, 46.128842429032346, 39.643918282146934, 35.01572679304774, 31.57994073484429, 28.796279798637332, 26.255744102713365, 23.68598290058919, 20.951860869076853, 18.03609155290314, 15.018154638640143, 12.034928817546373, 9.24130740991673, 6.772134711092346, 4.717864203690361, 3.1143449187007253, 1.9467912274856494, 1.1660890360408998, 0.7067452090269841, 0.5017624172535659, 0.49121262287507594, 0.6216787074684212, 0.8425234557564113, 1.1016623743412306, 1.3454691222282391, 1.5237901467017527, 1.5989916355957592, 1.5543428041080438, 1.3987152941733572, 1.1641126376486, 0.8961933005864251, 0.6414157141931331, 0.4343999721889718, 0.29056520390020035, 0.20555422552802813, 0.16192794011770725, 0.1389730359560983, 0.12215964305137521, 0.10793428926335462, 0.10288720216225654, 0.11775136602616586, 0.15964442203371826, 0.22615809986567809, 0.30405029509650333, 0.37314982380985334, 0.41377927249355556, 0.4144741707932465, 0.3765561103050677, 0.3137887389647369, 0.24705173916323392, 0.1961489351727137, 0.17341510930756449, 0.17807868140338992, 0.19932717759491375, 0.22088893996234577, 0.22862347387534218, 0.2163628213258229]
Sr_tanh_S = [442404.69729067106, 1278889.9416605784, 765519.9597138779, 923511.4425135824, 486148.74689215986, 1707083.1944048477, 374010.6313119212, 657754.3885960086, 1144639.64548546, 774348.3887241808, 641036.8681221608, 360797.00561426504, 387650.2902073594, 427670.7066932694, 58433.04996039677, 32856.49553342316, 214594.8041770054, 50767.71892494828, 297701.9446329342, 65363.75235998749, 56409.661894425066, 155722.46754466358, 38349.11333415992, 28405.416710194306, 29471.954342753892, 28167.601292607447, 34213.65529079097, 22631.01277561438, 12699.457477210115, 39529.68044854858, 24382.25913664342, 23035.401788164287, 25875.87825536532, 7839.040762057225, 15629.819468450774, 6709.890404377424, 12304.643002818977, 8109.309000210174, 5955.058556685252, 5417.922959309789, 5170.210382302632, 4930.231353893789, 3722.7748443753967, 2562.972152667806, 2037.0695971541802, 1606.9031612063548, 1257.9636436650298, 980.0149841188514, 760.5834920044978, 585.0711971127357, 445.35351213649227, 335.92399548739303, 254.17821255336204, 196.31884174104303, 177.1122550076266, 178.34741092180423, 164.88627703995311, 134.68757949222052, 95.89515639878465, 61.87983335688093, 40.528196893874345, 30.83969058204641, 27.337869583645297, 25.252360434453326, 22.118822996166777, 17.199093806779743, 11.610766630068222, 6.436398327678512, 2.8945213041388564, 1.9885250307120952, 1.3532074520459623, 0.8823581139152984, 1.3296852513993431, 1.8462695062095131, 2.1250942923147367, 2.1385874049050893, 1.913158003541419, 1.5029819534231161, 1.024061863434342, 0.6294832992942979, 0.43052683042764667, 0.4070418018490973, 0.4091556689608865, 0.33912090268913786, 0.30484581393220195, 0.3236020666445071, 0.2749079163107613, 0.2105228653957368, 0.1371751871627731, 0.0856678719407419, 0.06266141273736395, 0.05535245897676042, 0.08624213936938888, 0.12467471014003197, 0.11065809469567722, 0.07353472011611964, 0.03232493523623665, 0.012341364169942349, 0.015903501053021803, 0.017185615068968797, 0.01713871762074045, 0.016654946246347446, 0.014890025428495964, 0.013781885435133324, 0.013466318770532854, 0.01261113173396393, 0.01701846197192088, 0.019911848979088924, 0.015637408610103348, 0.018317730972408113, 0.02069115596765923, 0.01570551215464335, 0.008983982555149992, 0.009686429753051005, 0.01236267825185128, 0.011701814613526368, 0.010025544808837552, 0.00939311416316468, 0.008793166603157432, 0.01011206363706117, 0.010721522164478126, 0.009662097929535962, 0.0077009876853038365, 0.007701083108830601, 0.009131214575683229, 0.009975338740939244, 0.009059005744436858, 0.007776129120520838, 0.007855396785361053, 0.009309485687848939, 0.01129538817554128, 0.011285025920475167, 0.008162491898022475, 0.006593384886370162, 0.007434695123499288, 0.008470901674270125, 0.008335912820513519, 0.006979578825070531, 0.006010491719015485, 0.006644705903912927]
Ba_tanh_C = [73324817.73475066, 67955445.37123683, 70303477.95102252, 26912510.240897764, 29412235.104539517, 40738979.63864655, 46868727.03807209, 26733013.11270891, 36763342.206957206, 27857094.86185328, 22336822.461980723, 23400178.5077153, 18565819.436323393, 17399647.753362108, 16334625.151046518, 14834712.087636055, 12116301.996285295, 10691815.047016883, 11785890.802371891, 9492637.098824544, 7357713.672392952, 6904316.490411608, 4626879.731939823, 4346224.803508275, 4056877.9196955995, 3230746.9642815753, 2981806.72723893, 2673033.125898873, 2206091.9177174587, 1609806.799476601, 1842885.5388429395, 1442742.4968132586, 581958.0662770706, 516053.8377041925, 838594.5743487058, 713767.2089234035, 615559.502292108, 526925.1828762281, 450669.973558128, 385405.5133030105, 328435.18669742264, 279291.0923326302, 237275.84956782748, 201317.90919253518, 170511.52570115053, 144271.35895434205, 121979.19450377564, 103149.99117285451, 87208.56266316962, 73749.42174006533, 62345.77689001679, 52682.683266853564, 44470.97971367829, 37478.64366967806, 31516.587150251482, 26431.75288549927, 22098.457009638423, 18417.60703259544, 15301.352071208927, 12678.039814363814, 10483.737711156096, 8660.198945353703, 7155.760150549168, 5922.297022552397, 4916.642180382545, 4099.322140753688, 3435.3829735063096, 2894.274120080596, 2449.821711546985, 2080.5404113906625, 1769.222292988818, 1502.6394010126469, 1271.2349443860742, 1068.5508354841752, 890.4062108631504, 734.2733193685758, 598.7292422264111, 482.7017161536413, 385.24946399305975, 305.1511792052756, 240.88992187622492, 190.6683879426696, 152.44020305828965, 124.12248484356475, 103.65005490549744, 89.0785711995959, 78.68010860569328, 70.95982596354561, 64.70476158219675, 58.99693492305545, 53.2228225845505, 47.079240812292625, 40.527421203610196, 33.74614149208184, 27.042593967754016, 20.76550800555998, 15.217305788536432, 10.60132954065033, 6.9980931627887255, 4.37455798999009, 2.620259794388792, 1.5880937812202427, 1.1274780250607013, 1.1037765331959015, 1.3969455308223582, 1.8932055138160406, 2.475504416695178, 3.023358451362436, 3.424066247763163, 3.5930467378145945, 3.492710969517933, 3.1430025217263995, 2.6158164387360343, 2.013788920680262, 1.441289140082082, 0.9761223370503528, 0.6529158180475464, 0.46189228055243853, 0.3638622369871053, 0.3122796476400011, 0.27449974533973154, 0.2425340657743476, 0.23119454637948925, 0.2645941193986244, 0.3587315352241284, 0.5081898953266405, 0.6832168476042345, 0.8384853175964233, 0.9297855286819617, 0.931347318108911, 0.8461408215324728, 0.7051055789880928, 0.5551375144108351, 0.44075644588937507, 0.3896730690314114, 0.40015065065651556, 0.44789884370368394, 0.49634872024085636, 0.5137283225669628, 0.48617780174450675]
Ba_tanh_S = [826152.5764953049, 580687.3201972947, 316123.31672577746, 405432.82105799543, 196423.6423137813, 744128.6632115353, 268904.81515574636, 286767.56926549965, 498128.1876704644, 484791.94929239526, 587947.2115413675, 323801.28083051805, 176423.57700594314, 194818.17354967762, 81329.85412931904, 135186.89839803887, 85212.38301586441, 135818.23605760635, 135784.7143404602, 96261.41974531004, 55020.50725781752, 115848.82210806743, 13401.922512464222, 36202.55419495967, 24036.41903793109, 18428.69846318021, 12224.12541763421, 19474.366270355556, 23632.904280737956, 28365.54192392861, 20046.313484000762, 15383.008025675821, 13003.690880757924, 9960.812375226498, 10267.63495145251, 11539.783867626888, 12038.83301245811, 11143.795526317548, 9058.12951974785, 6520.224711430207, 4291.605018895523, 2801.3119565518978, 2048.7404747311953, 1773.2844192446546, 1666.4951586719608, 1515.1628185046145, 1232.571577496935, 864.0569578120192, 515.2126759323224, 331.7360210864197, 243.3890301829658, 210.75123663042737, 200.40294662458885, 190.79149810452748, 169.5955176063759, 135.7496749204808, 95.4905981716918, 72.32112126654191, 55.55095076120082, 37.901197113316854, 23.74469153847064, 17.151799604392483, 17.620319746299906, 23.151993835691467, 28.509093605120505, 30.571984976197086, 28.28717938500147, 22.422068321338664, 15.068901246893304, 8.827280509622083, 5.30587787334853, 4.989676252640338, 5.872121624375513, 6.583222491324622, 6.487778756650987, 5.506169824933515, 3.9187838039899963, 2.289849441302437, 1.144357395508361, 0.6442170538971707, 0.5144880854504638, 0.5582587237873825, 0.6622552229359076, 0.7462694670147824, 0.7161271356545067, 0.5658769515083601, 0.41244377436890467, 0.3306574390145829, 0.25416406671384967, 0.18530733827312665, 0.1621399125647447, 0.20816708883846743, 0.25671639197045715, 0.22926779934457875, 0.1574898934362577, 0.1167698936358581, 0.07608175998945707, 0.041505498372735046, 0.02741859292193847, 0.030000120032559148, 0.0228606986497751, 0.009251890283226982, 0.004991136921238741, 0.012896905181750679, 0.019117702132896496, 0.023672879998477604, 0.020948911518083865, 0.010652650441511953, 0.006318638476520473, 0.00795286132667186, 0.014001744398836218, 0.024531229716924677, 0.027264504861456474, 0.02089227339544673, 0.010911206793854092, 0.006836563372128804, 0.005899111558926996, 0.0035657519730587796, 0.003785746418827401, 0.004505594465374144, 0.00472733730084352, 0.004329384707957753, 0.004850844844630746, 0.004206802812943135, 0.0056768160275140825, 0.008982860352461182, 0.009939024674508797, 0.0070471988891636, 0.004913244005699889, 0.004890963678036032, 0.004139458335411271, 0.0039045977759198137, 0.004970010945194929, 0.004057630849226389, 0.004540174948055675, 0.007466881904017837, 0.007318927670868569, 0.005031016235624147, 0.004500514195791644, 0.005756309342845694]

fig, ax = plt.subplots()
plt.semilogy(Tvec, Sr_STA_C, color='k', marker='1', markersize=10, lw=2)
plt.semilogy(Tvec, Sr_STA_S, color='k', ls=':', marker='1', markersize=10, lw=2)
plt.semilogy(Tvec, Ba_STA_C, color='tab:green', marker='1', markersize=10, lw=2)
plt.semilogy(Tvec, Ba_STA_S, color='tab:green', ls=':',  marker='1', markersize=10, lw=2)
plt.semilogy(Tvec, Sr_STA2_C, color='k', marker='.', lw=2)
plt.semilogy(Tvec, Sr_STA2_S, color='k', ls=':', marker='.', lw=2)
plt.semilogy(Tvec, Ba_STA2_C, color='tab:green', marker='.', lw=2)
plt.semilogy(Tvec, Ba_STA2_S, color='tab:green', ls=':',  marker='.', lw=2)
plt.semilogy(Tvec, Sr_tanh_C, color='k', marker='x', lw=2)
plt.semilogy(Tvec, Sr_tanh_S, color='k', ls=':', marker='x', lw=2)
plt.semilogy(Tvec, Ba_tanh_C, color='tab:green', marker='x', lw=2)
plt.semilogy(Tvec, Ba_tanh_S, color='tab:green',ls=':', marker='x', lw=2)
plt.xlabel('Transport Time (us)')
plt.ylabel('phonon')
# plt.legend(loc='best',shadow=1)
plt.title('Diabatic transport of 88Sr-133Ba at 1.2MHz, 120μm')

custom_lines = [Line2D([0], [0], color='k'), Line2D([0], [0], color='tab:green'), Line2D([0], [0], color='tab:grey'), Line2D([0], [0], color='tab:grey', ls=':'), Line2D([0], [0], ls='', marker='x', color='tab:grey'), Line2D([0], [0], ls='', marker='1', markersize=10, color='tab:grey'), Line2D([0], [0], ls='', marker='.', color='tab:grey')]

ax.legend(custom_lines, ['88Sr', '133Ba', 'ω COM', 'ω stretch', 'tanh', 'STA-5', 'STA-9'])


In [ ]:
#### Diabatic transport of Sr vs Axial Frequency Plot ####
Tvec = np.array([n/10 for n in range(10, 150)])
h_10 = [2306943.9868815304, 1879277.4833926156, 1494784.2020272433, 1159100.969027378, 874544.0891649076, 640491.5656890749, 453929.9475691508, 310089.82856855425, 203106.2425461201, 126635.39750742234, 74384.07226626073, 40519.891333876716, 19949.099785199007, 8466.734816351687, 2792.793377713066, 519.9768068461844, 0.6935909172427128, 201.12433226979635, 546.6848875041148, 777.6635641370026, 827.3459998129813, 728.6429250347114, 549.8843944367918, 356.58349076794485, 193.62654462064134, 81.57455374894401, 21.097378778702076, 0.7034749570380098, 4.4016806165566775, 17.42301402957792, 29.371731960067184, 35.040767446211206, 33.6001186432738, 26.980750697531626, 18.173342778804688, 9.90725809258, 3.912541386308635, 0.7446145703390311, 0.013117526119132075, 0.8063191059812695, 2.123239538460648, 3.1882250297126227, 3.5973856949027407, 3.3100351745269396, 2.537436155099581, 1.5936149771363863, 0.763452377177528, 0.22121639858683703, 0.008028037664526136, 0.05706539873267896, 0.2446496970211041, 0.4441837093180035, 0.5657041400133216, 0.5730645189845847, 0.479728402483817, 0.3303310506978316, 0.17769775937801396, 0.06328799482819109, 0.006769097712331231, 0.005090514600592805, 0.0391010196192637, 0.08366972688711029, 0.1171905379051695, 0.12760999851666968, 0.11392231910545769, 0.08375212734222694, 0.04873262363783612, 0.019690244430964742, 0.003226262738647721, 0.00047912982395481704, 0.007957983847832513, 0.01969089151017779, 0.02983283084177991, 0.03437619368078197, 0.032238277661770294, 0.024893837375589416, 0.015338239444580197, 0.0067560906722584375, 0.0014035316207441262, 3.515851308002465e-05, 0.0019503829998278978, 0.005517534533887121, 0.008906995618630599, 0.01074293195172078, 0.010476072568735128, 0.0084055452680148, 0.0054106526976916295, 0.002538775421653318, 0.0006149532017504596, 3.787141675523362e-07, 0.0005536671267464465, 0.0017716692514520654, 0.0030187961230813956, 0.003780943259850049, 0.0038059730738584457, 0.0031492753039751974, 0.0020985523011511, 0.0010336357675590614, 0.0002790104006777496, 1.1277652444713626e-06, 0.00017711698764771408, 0.0006342127687146265, 0.0011342917553264075, 0.0014664249261326086, 0.0015155475841765564, 0.0012870669918705836, 0.0008814953278097977, 0.00045119239217901994, 0.00013205620577863482, 2.0856441324386004e-06, 6.254345191143923e-05, 0.00024903638500097495, 0.0004642186675180437, 0.0006166217994478354, 0.0006516290518963243, 0.0005648366437174837, 0.0003962236019739448, 0.00020925361235833056, 6.522012449065951e-05, 1.9763602125170797e-06, 2.4001236732616084e-05, 0.00010558808383888018, 0.00020415821073147984, 0.0002780041096760075, 0.00029897452189195584, 0.0002638705082308633, 0.00018876245491093152, 0.00010243962010934239, 3.353617380652148e-05, 1.494217822864347e-06, 9.889067358085299e-06, 4.779673588419767e-05, 9.547002370645205e-05, 0.00013248613804729307, 0.00014505797890192277, 0.00013001781174735893, 9.45845182734148e-05, 5.2400394273718904e-05, 1.7898846931982984e-05, 1.0380029557291104e-06]
ht_10 = [2091051.7539212105, 1879039.0817298614, 1670481.809884668, 1468963.7601112172, 1277508.0029909785, 1098524.322890021, 933790.6367947545, 784461.3194309648, 651106.2047400732, 533767.7904026681, 432035.3297042252, 345129.5109618553, 271986.63408271637, 211349.0508804471, 161844.53232112847, 122058.23437571066, 90593.46372469529, 66120.0191236235, 47409.82735569265, 33360.4703488267, 23007.78455263404, 15529.235100095153, 10239.633686066152, 6581.7996235540895, 4113.2488049031, 2490.983059745826, 1455.6768818449816, 816.2962208846467, 435.9318626598305, 219.3164409724021, 102.25671510717403, 43.017294396863655, 15.543394674636108, 4.327292166077193, 0.6590704470643342, 1.2754354318384633e-11, 0.22186879171472693, 0.48560391412450227, 0.5695183193690793, 0.49732047199418367, 0.3538298747167958, 0.20938925412682752, 0.10062601566002201, 0.03576351814682744, 0.006739833287007213, 1.2641406121289695e-11, 0.0032420054233607875, 0.008261892785823125, 0.01112882556959477, 0.011032159794296788, 0.00882189496705441, 0.005816819440434504, 0.0030910188791910664, 0.001206619860980647, 0.00024827506910076857, 6.865109575748551e-10, 0.00014017353980402914, 0.00038649202582743025, 0.0005551553746006084, 0.000587641584622742, 0.0005004025908635175, 0.0003500462443155778, 0.00019686104255469387, 8.11347702224411e-05, 1.7580923450509824e-05, 1.2629073752992148e-11, 1.093371553784703e-05, 3.1472049214052944e-05, 4.736525195193002e-05, 5.23081997066624e-05, 4.636884959022175e-05, 3.188134048956099e-05, 1.970683210419831e-05, 8.486085136306626e-06, 1.8884826687593348e-06, 1.2150645617869223e-11, 1.2577594170329876e-06, 3.718749595713311e-06, 5.797617467518824e-06, 6.597022075296352e-06, 6.021112535430368e-06, 4.506461020916467e-06, 2.704580114439317e-06, 1.1873183849371077e-06, 2.734318469541877e-07, 1.2830716903096154e-11, 1.9108748685153896e-07, 5.802449691685947e-07, 9.235997064869067e-07, 1.0743872815557834e-06, 1.0035522776126786e-06, 8.786725145503574e-07, 4.705536971664895e-07, 2.1078383249471885e-07, 4.959784567858693e-08, 1.2438615957609677e-11, 3.610145470608686e-08, 1.0780859514888384e-07, 1.8073860003171232e-07, 2.1804200022986678e-07, 2.0259812036232172e-07, 1.5766485805654172e-07, 9.887402040716994e-08, 4.511203002254898e-08, 1.0769350975000406e-08, 5.447185233541757e-10, 8.077759724132297e-09, 2.535693983807238e-08, 4.1756618955710694e-08, 5.6273596167670246e-08, 4.8438240459026296e-08, 3.8423706701670095e-08, 2.4212111349960752e-08, 1.1199760658940544e-08, 2.7981750788991255e-09, 1.2821151511687028e-11, 2.099702682231802e-09, 6.667520029934376e-09, 1.1125045055407265e-08, 1.3569316826824894e-08, 1.3251463157101096e-08, 1.0571872427201625e-08, 6.805216652820113e-09, 6.6089854396321556e-09, 7.915861725941219e-10, 1.2782474846309161e-11, 6.339590405628451e-10, 1.6475348427359764e-09, 3.372579358565302e-09, 4.133206375319027e-09, 3.980041714124188e-09, 3.369307423265966e-09, 2.172780208478588e-09, 1.0493963054745489e-09, 2.666110747356654e-10, 1.2783437134528778e-11, 1.8846125857288263e-10, 6.811485606744679e-10, 1.1484743177720935e-09, 1.4241303854841503e-09]
# h_11 = [694147.060672849, 509924.7609895815, 359879.9469408499, 242925.9098741219, 155896.96579925145, 94318.4242755941, 53144.29614347138, 27368.087914023694, 12478.832192454205, 4739.779823282071, 1296.9007465113684, 147.10027353501584, 16.40605817653888, 191.86952929044452, 346.31937061201705, 384.1908338971573, 323.66736507845496, 218.83564151163102, 117.64267866052417, 46.27576683770836, 9.797126523565957, 0.015624545088080075, 4.267520447067662, 11.686489304654092, 16.02095672228082, 15.60324346350103, 11.747336787881697, 6.81633453224137, 2.7757476181930185, 0.5388761344282554, 0.0060873509311629255, 0.49096839378604923, 1.2124325283236748, 1.6356336285836122, 1.5861249163187463, 1.1783561887917005, 0.6552600535361238, 0.23537444737723875, 0.0261092040336056, 0.014789761805179302, 0.11360918409398173, 0.22136293638999488, 0.27053342212046033, 0.24461051772549755, 0.16817135793136218, 0.08253136847085417, 0.021900048156812433, 0.00015448589577700874, 0.010589912431520819, 0.034854532468461354, 0.05442843289749683, 0.058822396173490096, 0.04793846710453854, 0.029059677870286328, 0.011440600806564798, 0.0014997659127357235, 0.0006159913452326565, 0.0058250542342078455, 0.012210201796110182, 0.01579531535735256, 0.014977022363045234, 0.010709155503149017, 0.005405253805815228, 0.0014285971995181847, 2.8373001518621293e-06, 0.0009050703712531335, 0.0029083250526692494, 0.004591254363456249, 0.005032722747327985, 0.004147990636331674, 0.0025154680811685813, 0.0009563600343799508, 9.763841394862463e-05, 0.00010113788334209217, 0.0006983575536696623, 0.001402241916953643, 0.0017877255272550742, 0.001679265185414571, 0.0011819629055980945, 0.0005724348285645445, 0.0001300227797552998, 1.7419090909536133e-06, 0.00015733538051109435, 0.00043854554345197996, 0.0006602657819608371, 0.0007046958812540252, 0.0005651359436429803, 0.0003277076596400843, 0.00011176262163726402, 5.566783864419172e-06, 2.891805376267444e-05, 0.0001360542765242764, 0.0002495405922562427, 0.00030381973233648566, 0.0002748950747869967, 0.00018461903558408515, 8.183507459773243e-05, 1.3697266777757496e-05, 2.7696506703312694e-06, 3.9972996080789624e-05, 9.487950238452796e-05, 0.00013334806303855686, 0.00013566027095660992, 0.00010350695111651913, 5.561618899457834e-05, 1.5785803517083537e-05, 5.4078657643547606e-08, 1.0408181919696463e-05, 3.560415373596242e-05, 5.9027088451041944e-05, 6.763383275222388e-05, 5.799722604095992e-05, 3.634296539618898e-05, 1.4089396184049326e-05, 1.3213929535766506e-06, 1.9858930260638737e-06, 1.2822385560659483e-05, 2.6049272657247584e-05, 3.390440114404564e-05, 3.250535932097622e-05, 2.3219499376356685e-05, 1.1236065107359053e-05, 2.358870160798958e-06, 1.248652503779054e-07, 4.2428634850534275e-06, 1.1319891359957454e-05, 1.698391683081785e-05, 1.8194575776165626e-05, 1.462303448418033e-05, 8.377580410901273e-06, 2.6800826791020746e-06, 5.983152616683136e-08, 1.1828083378502663e-06, 4.756293048856768e-06, 8.423873672574528e-06, 1.0161549880261894e-05, 9.10511739575115e-06, 6.008122960052051e-06, 2.5215515934248884e-06, 3.1999045539579817e-07]
# ht_11 = [495593.931148388, 435173.868258562, 377102.1589498217, 322408.15011932707, 271882.44495596556, 226071.6181366219, 185287.2782669902, 149625.13851576476, 118994.33095269024, 93151.01294707232, 71735.988734802, 54311.22671108167, 40394.35839073486, 29488.92888538562, 21109.462760377857, 14800.630751231012, 10150.55271898646, 6798.663337891077, 4438.828322058347, 2818.6507926038203, 1735.9344699901428, 1033.3172759894164, 591.8419029914036, 324.2643962449641, 168.5914738225791, 82.23298542094108, 36.983123919240754, 14.904919341107858, 5.10596012124279, 1.3201373717250042, 0.17383335075103706, 0.0009722091512050661, 0.07495394625809354, 0.14008115043359232, 0.14633302125017947, 0.1132412894347746, 0.06967848126317765, 0.03391609644841641, 0.011993466690376866, 0.00221801083011071, 8.405536803576756e-10, 0.0009918015693654299, 0.002392212755196359, 0.0030048505806067826, 0.0027242435972512236, 0.0019362149030736867, 0.0010813107703928878, 0.0004407042105030148, 9.849638777008624e-05, 5.200848578956309e-07, 3.689465056426747e-05, 0.00010731536298869697, 0.00015116692178267305, 0.00015086483172758764, 0.00011715015716391367, 7.178856003877479e-05, 3.202005072973603e-05, 9.04749575912908e-06, 1.5745327883305037e-07, 2.415019673377698e-06, 8.230658637771548e-06, 1.2781838976264694e-05, 1.3700445151393883e-05, 1.1357458526735717e-05, 7.391876773128722e-06, 3.5696751622456423e-06, 1.0207313892222691e-06, 4.933630114278653e-08, 2.2587531125129328e-07, 9.215937015828942e-07, 1.5145097229171859e-06, 2.231651805200483e-06, 1.5255479991440214e-06, 1.0108613727875827e-06, 5.396658491980399e-07, 1.6936462072361533e-07, 1.0244512573755244e-08, 2.749403598089814e-08, 1.3318332902660526e-07, 2.3892342773170996e-07, 2.8580328040979837e-07, 2.6129445639630517e-07, 1.8845790640570613e-07, 1.0115623034602608e-07, 3.4537665711829334e-08, 2.86749798352692e-09, 4.074923815713627e-09, 2.3916602259603062e-08, 4.570012007973936e-08, 5.7425203743186716e-08, 5.0834484136743655e-08, 3.454454221279404e-08, 2.279316822786973e-08, 8.309480850978049e-09, 8.76443737621966e-10, 7.258344297486685e-10, 5.120370535352954e-09, 1.0594876291513336e-08, 1.3502056200334737e-08, 1.214295363753662e-08, 8.849863603142445e-09, 6.090629641177687e-09, 2.2653053847469414e-09, 2.8564996721887636e-10, 1.5971096566681549e-10, 2.5827443762939837e-09, 2.7659085189968004e-09, 3.72076489067994e-09, 3.757507208596889e-09, 1.6298684604668062e-09, 1.2713383161887139e-09, 7.048340479962647e-10, 1.0379931961737157e-10, 4.733777285258484e-11, 3.7044120336482854e-10, 8.638719267676589e-10, 1.1899072606070745e-09, 1.2267976601789773e-09, 9.846700801491742e-10, 6.066829887287971e-10, 1.0768730208430268e-10, 6.556441786297262e-11, 2.494023573686541e-11, 1.3327001347773408e-10, 3.189309259093391e-10, 4.3910904412007787e-10, 4.51469927051884e-10, 1.0469242190138929e-09, 2.3688468652437403e-10, 1.031773932880429e-10, 6.743989786606905e-11, 1.7804999355235166e-11, 7.087882754668171e-11, 1.529510005471549e-10, 1.8494584843102717e-10, 2.0020363203639517e-10, 1.9869423970941645e-10, 1.0813134295276301e-10, 5.207838536396838e-11, 1.766099164528855e-11]
h_12 = [2.80567627933004e-07, 7.258199604426839e-11, 5.365928176277793e-11, 1.8476963787734096e-11, 2.2689206984559558e-11, 6.482258047186766e-08, 7.263096629750859e-12, 2.032481468922375e-11, 1.1206677594723897e-11, 2.032319405743324e-11, 1.259177239397298e-10, 8.229916864613956e-11, 1.833557457829625e-11, 1.822479442728762e-11, 1.8770846575186463e-11, 1.815491969047042e-11, 1.7023230038225845e-11, 2.125941501426394e-11, 1.4585383128726702e-11, 1.6969307645115912e-11, 6.246938439389739e-10, 1.7563061480464658e-11, 1.8466211852178304e-11, 1.1927952324235654e-10, 1.7459730825537957e-11, 1.837028215617862e-11, 1.7600502645242274e-11, 1.7850213232176232e-11, 2.0283025707334612e-11, 1.8238349589900212e-11, 3.232684070500453e-10, 1.7877963889076794e-11, 1.7868402113859282e-11, 2.3839917124926147e-11, 1.876056099747906e-11, 1.751895218931347e-11, 2.0840702738168942e-11, 1.665623129391535e-11, 1.9793906002990487e-11, 1.8307994761589175e-11, 1.293047979524847e-10, 1.6849377571821102e-11, 1.671313002455733e-11, 2.1403486527616977e-11, 2.2136764158753793e-11, 1.7839885211436097e-11, 1.798594169207595e-11, 1.0351950739957743e-12, 1.8095502932706204e-11, 5.0463988895825806e-11, 1.8902708787332597e-10, 1.8152040020722214e-11, 2.2456476381300255e-11, 1.57046740711241e-11, 1.7938329509254878e-11, 2.0448747587826062e-11, 1.8531348080155623e-11, 3.2147287260334735e-08, 1.3954109694391118e-11, 1.8714946397322957e-11, 1.8172687358185608e-10, 1.957851223543613e-11, 2.374298125435722e-11, 1.7993386665999472e-11, 1.735865164134833e-11, 1.7341552184619125e-11, 7.0232470710849605e-12, 2.2003448288935505e-09, 2.4686776794551104e-11, 1.800577898265688e-11, 1.0805230074531554e-10, 2.431563721452455e-10, 1.767813615372862e-11, 2.670579492202648e-10, 1.649361899017499e-11, 1.9943748022258655e-11, 2.4374836420774288e-11, 1.7989817705568062e-11, 1.493905345807659e-11, 1.7972505586908212e-11, 2.5622894590370252e-11, 1.791258406171002e-11, 2.393009247990694e-11, 1.869221073663066e-11, 2.484121323148112e-11, 1.8684787679945182e-11, 1.7740680342402105e-11, 1.79112201683932e-11, 1.7923061654277758e-11, 1.8790360422873714e-11, 2.6563123112228466e-11, 7.167802216499611e-12, 2.4649258096206894e-11, 1.8782179248677327e-11, 1.929295706962586e-11, 1.7775832291402498e-11, 1.7827678334198935e-11, 1.7447761109731475e-11, 1.787378935649081e-11, 7.253953319593088e-10, 2.2291112199977392e-11, 1.7620541567860433e-11, 1.7317433073612773e-11, 1.8094740422794385e-11, 1.8978118710843238e-11, 1.4164010946626751e-10, 1.797421649040866e-11, 1.741609493700708e-11, 2.5106886851274962e-11, 2.2050924460193705e-11, 1.437969972200411e-11, 1.7991687314731846e-11, 1.790581899763946e-11, 2.1749560257125242e-11, 2.4781371270647193e-11, 1.6619161979728428e-11, 2.4533069350658902e-11, 1.7255493642754982e-11, 1.7880743039633735e-11, 1.7917409924824074e-11, 1.7870642213747943e-11, 3.9784977592678975e-11, 1.7374469301920043e-11, 4.101935899909238e-11, 2.4890360457808337e-11, 1.7922244731460223e-11, 1.774663424258964e-11, 2.118087127956463e-11, 1.788946293893029e-11, 1.7923447314665858e-11, 1.811688891748064e-11, 1.7926051667615134e-11, 2.4059526804637508e-11, 1.6357861685261256e-11, 4.792544840646286e-11, 1.7952512327076293e-11, 8.776834790000646e-12, 1.7589248855970598e-11, 1.677599477462534e-11, 1.692292454667327e-11]
ht_12 = [2.3682847765014527e-07, 8.428723930154928e-11, 5.237622014168806e-11, 4.439147719735107e-11, 2.8608440905489526e-11, 1.2086921551513066e-07, 3.5258756726772646e-12, 4.520587711780528e-12, 2.3527159260953852e-12, 1.0700081215924835e-11, 1.199959933954395e-10, 1.593273799204405e-10, 1.987927365662314e-11, 2.3655979752877693e-11, 2.685492467505198e-11, 4.211694692394087e-11, 2.337533764739837e-11, 1.4397259329275028e-11, 1.5689558274659642e-11, 1.781475625652686e-11, 2.9244547500206125e-09, 1.631389310391483e-11, 1.7731736928741952e-11, 1.3553309858030725e-10, 1.7635647142508674e-11, 1.3395599496951831e-11, 1.7186424097664105e-11, 1.9875827999259892e-11, 1.962137002457263e-11, 1.8686354317599422e-11, 2.4194409762106605e-10, 1.783122703956339e-11, 1.776150891486096e-11, 2.721364602553207e-11, 2.095791306189476e-11, 2.0121519365627866e-11, 2.089596445811017e-11, 1.674922956395785e-11, 2.0768279981533974e-11, 1.8094048761033018e-11, 2.873047624485816e-10, 1.719442064729003e-11, 1.6539079749076203e-11, 2.2568743158308495e-11, 2.275150038347787e-11, 1.8055307665691013e-11, 1.788455799291595e-11, 9.541854209040272e-13, 1.8378448721263393e-11, 1.3986896772582504e-10, 3.1277781952303426e-10, 1.7594618506550438e-11, 2.241310135488746e-11, 1.3999243545795261e-11, 1.7852096266358568e-11, 1.3521746396081843e-10, 1.846236060843975e-11, 9.7843309303405e-08, 1.4318647939903694e-11, 1.9009388621665036e-11, 5.548221641729324e-10, 1.9745375467082414e-11, 2.4246171317674103e-11, 1.8268928883627476e-11, 1.7052429727695088e-11, 1.6691024782526682e-11, 1.3271683484330783e-11, 5.416868606750309e-09, 2.4917807697874676e-11, 1.856235338740302e-11, 2.4512664712591255e-10, 2.4402202354644734e-10, 1.7721752506704058e-11, 3.2278217919653015e-10, 1.5800812672088432e-11, 2.102014197341037e-11, 2.4583450480230495e-11, 1.914766349243033e-11, 1.0676633189110346e-10, 1.790234991084161e-11, 1.7780590134968482e-11, 1.790236648871286e-11, 3.079786982447005e-10, 1.9654893553568435e-11, 2.5049632624214505e-11, 1.901175038482826e-11, 1.597934986813546e-11, 1.786483315052191e-11, 1.763308865098356e-11, 1.8942385173543295e-11, 8.547170251597398e-11, 6.305257191794723e-11, 2.3499936172624624e-11, 1.9186468878064102e-11, 2.025705066848623e-11, 1.778636483147487e-11, 1.791732700072575e-11, 1.9325044959820466e-11, 1.777312737778974e-11, 8.86549620409622e-10, 3.155458622427424e-11, 1.950581387649232e-11, 1.6963558228430515e-11, 1.7115171003231354e-11, 2.0235705392494885e-11, 4.4159385552825815e-10, 1.798409315105654e-11, 3.208041885958177e-12, 2.577627859900608e-11, 5.964396009720464e-11, 4.001666172032974e-12, 1.827761728589136e-11, 1.7902490822991883e-11, 2.274544565355947e-11, 3.941137605796334e-11, 1.5398670637302322e-11, 2.4955422493776378e-11, 2.5081327262254885e-11, 1.7847208860219705e-11, 1.7978751609167525e-11, 8.516306256020076e-13, 4.6416640793262975e-11, 1.7249077579146007e-11, 1.9254289081714887e-11, 2.560979163157732e-11, 1.794257795735317e-11, 1.736644323505514e-11, 7.746799207154015e-10, 1.788106611436763e-11, 1.7973439929455684e-11, 2.555898803703077e-11, 1.796031568093831e-11, 2.392967560745231e-11, 1.6323049334793657e-11, 6.262740792589495e-10, 1.8000558895352488e-11, 7.98288096842377e-12, 1.864326187758173e-11, 5.346932091678085e-12, 1.670772445831465e-11]
# h_13 = [420410.14915604884, 308835.9255172347, 217961.2996882, 147128.09060135373, 94419.01070198097, 57123.849774780094, 32186.847223616674, 16575.48300044556, 7557.802622405513, 2870.644966659094, 785.4675242384541, 89.09039093708954, 9.936575718754144, 116.20651835888033, 209.7490465811148, 232.68543482458026, 196.02904543789896, 132.53740088207954, 71.24980142874008, 28.02652831783702, 5.9335630353310425, 0.009455871521941888, 2.584716310363436, 7.077956418822622, 9.703105904179987, 9.450026429762755, 7.114640072717606, 4.128172292372198, 1.6810293229276967, 0.32632655641908365, 0.0036923488741799817, 0.2973781918362559, 0.7343220846001943, 0.9906043882374271, 0.9605945332003026, 0.7136176312266805, 0.3968098134103074, 0.1425252151233138, 0.015804261668036595, 0.008962218893046036, 0.0688020586385668, 0.134066495050752, 0.16383484686334387, 0.14812573239394325, 0.10182948929470424, 0.049967466619169475, 0.013255303493200978, 9.307915379528957e-05, 0.00641648384878124, 0.021110909540041686, 0.032953024034492975, 0.03561631737582296, 0.029022155847266827, 0.01758972546183579, 0.006922634418294767, 0.0009063939028819035, 0.00037388901963270705, 0.0035015624013923466, 0.007394252731445732, 0.00956228533532145, 0.009061847598625062, 0.006479755285987792, 0.003269075825630789, 0.0008631719310955444, 1.6528881551485382e-06, 0.0005487188846583038, 0.0017615631385596922, 0.0027752550755637206, 0.003044877627081718, 0.002508554170935271, 0.0015195652174905044, 0.0005791177126887196, 5.8692724852558516e-05, 6.191373792662354e-05, 0.0004230193011593521, 0.000848542003221925, 0.0010810749004690242, 0.0010148359683783107, 0.0007138163313244494, 0.00034527203734577283, 7.818073114498283e-05, 1.099070135556574e-06, 9.542448428106054e-05, 0.0002653706716207447, 0.0003990393313555717, 0.00042547678068991846, 0.0003408588280898574, 0.00019737395604356057, 6.712062858332768e-05, 3.27932822663498e-06, 1.7653223395460393e-05, 8.235351212135325e-05, 0.00015071875844762736, 0.00018322756060394572, 0.00016554237238746434, 0.00011098136999682346, 4.9048041329152295e-05, 8.13125420762512e-06, 1.7214422539567201e-06, 2.440377909695522e-05, 5.731010629077566e-05, 8.031254599118694e-05, 8.153787275156254e-05, 6.206913772853212e-05, 3.323955121762528e-05, 9.289198725594832e-06, 2.5705265454020482e-08, 6.339961680314134e-06, 2.1493588121424674e-05, 3.5477598417273375e-05, 4.056604797640515e-05, 3.467366066817401e-05, 2.1645089963020442e-05, 8.328663924976847e-06, 7.547611995281982e-07, 1.2297385473642356e-06, 7.752125830109566e-06, 1.564388462752367e-05, 2.0274972494523376e-05, 1.936253808460324e-05, 1.3768279399193891e-05, 6.595286001976319e-06, 1.3607180687583503e-06, 8.681155254070739e-08, 2.5778549678229308e-06, 6.7924733496181235e-06, 1.0125359280234043e-05, 1.0803000784377734e-05, 8.624644526701738e-06, 4.903050856729148e-06, 1.5422652526649107e-06, 2.8789825523772917e-08, 7.290459978714399e-07, 2.8573435779417043e-06, 5.033229921314316e-06, 5.996927354063531e-06, 5.346300919190566e-06, 3.49118960631711e-06, 1.4437740568181743e-06, 1.7247747890397106e-07]
# h_14 = [1352262.8646055926, 993380.753802631, 701079.5661897511, 473242.2541494006, 303701.7928476733, 183740.79460122177, 103530.03232466204, 53315.57704327166, 24309.91838899644, 9233.524582505755, 2526.4820418380355, 286.5628376566414, 31.961120027652935, 373.7814364600232, 674.6640201870331, 748.4400224376234, 630.5338439052665, 426.3111636355035, 229.17768168966114, 90.14854734435154, 19.085550852600182, 0.030420824976150892, 8.313749601409873, 22.766448496884433, 31.210343382629414, 30.3963782426259, 22.884577634867277, 13.278512388033908, 5.407167022324023, 1.0496749595334531, 0.011872121089941907, 0.9565080243814268, 2.3619608863507215, 3.1863247732555373, 3.0898170321790097, 2.295417254731104, 1.276390369306588, 0.45846042883081045, 0.05084194298378682, 0.02882342577831279, 0.22130825583980224, 0.4312307944776006, 0.526991066633919, 0.47646931980738494, 0.3275566480300322, 0.1607361024215171, 0.042642882624199484, 0.000299780035924847, 0.02063666428518893, 0.06790291376002018, 0.10600371580237455, 0.11456861320203086, 0.09336012730225339, 0.05658613783080943, 0.022271970843149035, 0.0029169966734734053, 0.0012019745583535728, 0.011283941000080804, 0.02378453821672544, 0.030760718959755195, 0.029154915929822143, 0.02084733914468169, 0.010518781343529966, 0.0027780666254927254, 5.3681786638272e-06, 0.001764514330512939, 0.005666015952920968, 0.008931048122516765, 0.009796500509879005, 0.008071786933077791, 0.004890872654168018, 0.001862832272424436, 0.00018914025299186473, 0.00019861744557262928, 0.0013606080210625475, 0.00272994355979352, 0.0034786413144798686, 0.003266021409733323, 0.002297644596686369, 0.0011117166782830558, 0.0002519244364766001, 3.4989132921853173e-06, 0.00030682366383922416, 0.0008537609264463731, 0.0012842033924726937, 0.001369617394294692, 0.001097515548315833, 0.0006357408385394872, 0.00021634983049327754, 1.0620844153156026e-05, 5.666951873512079e-05, 0.00026493095751620735, 0.00048512411146997314, 0.0005899817815937361, 0.0005332299575969974, 0.00035764097005277405, 0.00015817623358805656, 2.6284842253918446e-05, 5.500889353415973e-06, 7.833995144301512e-05, 0.00018446272313904385, 0.0002586874122408233, 0.0002627684762038273, 0.00020014250953798537, 0.00010727083865721664, 3.0095085237327063e-05, 8.738522583118707e-08, 2.0362761311765937e-05, 6.918943648333539e-05, 0.00011433233108999342, 0.00013079863706314657, 0.00011188931240760398, 6.991412918753823e-05, 2.695238964207981e-05, 2.4631381413125233e-06, 3.9328981465023455e-06, 2.4944657475135137e-05, 5.042521402858382e-05, 6.542206199042897e-05, 6.253862237349885e-05, 4.452094545175396e-05, 2.13808875389646e-05, 4.430259460940797e-06, 2.6914964124179224e-07, 8.284398861592018e-06, 2.189754851769992e-05, 3.2696967176506506e-05, 3.4920542076138044e-05, 2.792602204750229e-05, 1.5906652898640222e-05, 5.024492600593701e-06, 9.762304044676508e-08, 2.33386164342688e-06, 9.208528199105906e-06, 1.6243193183199215e-05, 1.941447107422237e-05, 1.7330164604584972e-05, 1.1346749203492503e-05, 4.709686469683703e-06, 5.709179392537528e-07]
# h_115 = [151652.9686948214, 111405.2155496557, 78624.34783078442, 53072.95103638232, 34059.404474956056, 20606.125880629785, 11610.63703655827, 5979.210827251989, 2726.2986132630267, 1035.5183241691554, 283.3389446602564, 32.137719846225274, 3.584249979234083, 41.918337761058375, 75.66158912317628, 83.93559755407625, 70.7128651648568, 47.809933198970775, 25.701933911522953, 10.110114049850962, 2.140427721048541, 0.00341483769361779, 0.9323248742888007, 2.553185907064404, 3.5001573200264584, 3.4089143556263535, 2.566509821210678, 1.489216010654873, 0.6064465957160978, 0.11773820676367473, 0.0013289299973916818, 0.10725957391653967, 0.2648825584285603, 0.35734621898203417, 0.3465343411436357, 0.25744966426346416, 0.14316588585842202, 0.05142843092919977, 0.005705757041317295, 0.0032303126740687634, 0.02482161474931173, 0.048362361937372596, 0.05910694045284831, 0.05344506669547149, 0.03674526677147479, 0.018034120548853023, 0.004786109662412402, 3.383857875190103e-05, 0.002313132485128812, 0.007614576119158142, 0.011893267142845686, 0.012852857177610187, 0.010475419169319614, 0.006350626589209137, 0.0025006172954645392, 0.0003280087301964512, 0.00013443170568336216, 0.0012774208429193123, 0.002667763509912049, 0.0034516091702661276, 0.0032737044388353813, 0.002340793925849928, 0.001181737297705997, 0.00031248162006903375, 6.320783192465551e-07, 0.00019763499693714167, 0.0006353684333380933, 0.0010040478673678244, 0.0011000924176584501, 0.0009068890724735641, 0.0005502712117280234, 0.00020895794184263647, 2.141116718620266e-05, 2.1978686985974298e-05, 0.00015256222764109846, 0.0003064837635380935, 0.00039087020836379316, 0.0003672735376388713, 0.0002585952248350516, 0.0001253184170802613, 2.850894744374136e-05, 3.7306480157237926e-07, 3.435006211717028e-05, 9.585290813743445e-05, 0.00014440393034222164, 0.00015419573715538473, 0.00012372248048607881, 7.179422599218585e-05, 2.4519777039603265e-05, 1.2332371275532421e-06, 6.293263860003466e-06, 2.9732891463998916e-05, 5.4592905704740754e-05, 6.651734828515377e-05, 6.022901036906813e-05, 4.048471706299924e-05, 1.7971892060969853e-05, 3.022425637155768e-06, 5.976193166637303e-07, 8.698348767405908e-06, 2.0756356921594977e-05, 2.9214126041350888e-05, 2.9751304535467838e-05, 2.2725475636672094e-05, 1.2231019265344577e-05, 3.498145515898256e-06, 1.3503326915975169e-08, 2.2678135568818195e-06, 7.791349852268221e-06, 1.2945003811802352e-05, 1.484814590587655e-05, 1.2752712547887306e-05, 8.00624244702365e-06, 3.1153160928664325e-06, 2.973239748201363e-07, 4.2942512215510665e-07, 2.804323990406111e-06, 5.71537348816855e-06, 7.454480647544717e-06, 7.160463301460047e-06, 5.12626880434733e-06, 2.4929858917499315e-06, 5.279872902383465e-07, 2.5773570155804956e-08, 9.259415796913233e-07, 2.485018546757353e-06, 3.7400671796247783e-06, 4.014515010002112e-06, 3.236860931447824e-06, 1.8614056598515624e-06, 6.005172502189961e-07, 1.483215304167187e-08, 2.565777439893733e-07, 1.043730189153769e-06, 1.8531992106586223e-06, 2.2488171216819383e-06, 2.0198795812960277e-06, 1.3395857514832267e-06, 5.66326169280967e-07, 7.41629178920654e-08]
# h_1175 = [35532.21630569327, 26102.18683372272, 18421.644579015927, 12434.964785523936, 7980.100320857971, 4828.013761464184, 2720.366092792833, 1400.9260610744798, 638.7705793422468, 242.6216522570234, 66.38620794430607, 7.529926751654175, 0.8397671836924486, 9.821383333687201, 17.727411797596165, 19.66604766030852, 16.567996921166742, 11.20187764946014, 6.021991346362877, 2.3688275543780635, 0.5015067667418851, 0.0008006968938861383, 0.21843544530621198, 0.5982072988788223, 0.820083973965597, 0.7987135064227143, 0.6013433102313486, 0.34893443685062286, 0.14209875712157574, 0.027589695938374095, 0.00031090044436045086, 0.02512890905463483, 0.06206081161819055, 0.08372748449683136, 0.08119636773275911, 0.060324865679587024, 0.03354778174373931, 0.012052129179783383, 0.001337596762118462, 0.0007564559466635831, 0.005816152177635584, 0.011331443188649233, 0.013849884015098204, 0.012524061322870756, 0.008611379712383498, 0.004226875208139642, 0.0011220971156937227, 7.969452833721336e-06, 0.0005417378244871914, 0.0017839849337945497, 0.002787561610395279, 0.0030122167204886673, 0.002455379179985744, 0.0014888151739389595, 0.0005864306965103901, 7.701612854934574e-05, 3.142985226547077e-05, 0.00030155578067696, 0.0006251278651260282, 0.0008090602674999354, 0.0007677858664076833, 0.000548973989021811, 0.0002772702505786147, 7.338819096347414e-05, 1.5401225607799243e-07, 4.62592833534415e-05, 0.0001488550282621441, 0.0002357073028278219, 0.00025802047343503236, 0.00021279458881987264, 0.00012926024781116446, 4.896732579876382e-05, 5.054117222498822e-06, 5.094812322976397e-06, 3.5740272469062205e-05, 7.187025478687251e-05, 9.17209700245313e-05, 8.623881184424737e-05, 6.076113608615205e-05, 2.9482455372011227e-05, 6.727767170294197e-06, 8.408610237541595e-08, 8.037140860123176e-06, 2.2478077737929152e-05, 3.3905636191873394e-05, 3.6239781228081686e-05, 2.9107908440989626e-05, 1.691471071101374e-05, 5.793220816829851e-06, 2.971177244269031e-07, 1.4632058502218365e-06, 6.9704320363492266e-06, 1.2826211237605599e-05, 1.5651291452776975e-05, 1.41922811187694e-05, 9.556221134509888e-06, 4.254618413123927e-06, 7.224154946735924e-07, 1.367095728437657e-07, 2.021861530203833e-06, 4.8765262234360515e-06, 6.883324486147556e-06, 7.024049398954655e-06, 5.377242162089785e-06, 2.903580072139479e-06, 8.430071005343621e-07, 4.152667474583631e-09, 5.286815772876114e-07, 1.8318466097989305e-06, 3.0564815073822874e-06, 3.5128798117680603e-06, 3.0265152956656475e-06, 1.9071211729987241e-06, 7.475987113950659e-07, 7.392546219139481e-08, 9.872683521533647e-08, 6.586367165731248e-07, 1.3506627480163363e-06, 1.7688957283209682e-06, 1.7054809656977443e-06, 1.2263271136547774e-06, 6.0229605371127e-07, 1.298280346449702e-07, 5.569487264914567e-09, 2.1668132550642093e-07, 5.879882932222983e-07, 8.902738459939786e-07, 9.592475208656677e-07, 7.783357517661398e-07, 4.5094838141025787e-07, 1.4792680836323533e-07, 4.497577781065697e-09, 5.946574228088464e-08, 2.468978638564143e-07, 4.4045889388172707e-07, 5.40574315752882e-07, 4.877944921067097e-07, 3.2671633229522157e-07, 1.4011355649940177e-07, 1.9565077267141924e-08] 
# h_1225 = [31356.369342117865, 23034.58587599327, 16256.686303437546, 10973.578247814252, 7042.2614571655795, 4260.581658481753, 2400.662631404101, 1236.28563287123, 563.6999198599035, 214.10730422341283, 58.58419942995428, 6.6447245218567526, 0.7411489288784033, 8.66736779111302, 15.644253102935442, 17.354916793516434, 14.620853304559319, 9.885282133573815, 5.314125703002182, 2.0903204530278283, 0.4425484941595605, 0.000704471972733076, 0.19279189184384085, 0.5279145468586541, 0.7237095129899995, 0.7048233239833375, 0.5306317807582601, 0.30788481107430143, 0.1253684820479841, 0.024334242036930402, 0.00027601413473146317, 0.0221825976724143, 0.054770928906574125, 0.07388250675607143, 0.07164143582408257, 0.05321930229010001, 0.02959072870136141, 0.010626997750439368, 0.001177784293439417, 0.0006689848007158911, 0.005131027780833186, 0.00999915554812353, 0.012218121747082337, 0.011045460229099932, 0.007592349529471061, 0.003724862341069558, 0.0009877051852627179, 6.888478226048473e-06, 0.0004788765526975529, 0.0015747021136032506, 0.002456517759428573, 0.002655393161097488, 0.0021633086459699902, 0.001310787114918851, 0.0005156157929341199, 6.738735056732511e-05, 2.7977565281383114e-05, 0.0002582375291176663, 0.0005514062023869929, 0.0007127419949535767, 0.0006748770823323105, 0.0004825970572073831, 0.00024330976829724295, 6.415043786292632e-05, 1.1639880031894381e-07, 4.099018402260319e-05, 0.00013140171408335737, 0.00020638657303612178, 0.0002267467905642338, 0.0001866909504505321, 0.00011289925122391589, 4.318224816551909e-05, 4.328858858717135e-06, 4.692142638731041e-06, 3.1559074913629235e-05, 6.32076823498743e-05, 8.044691884172766e-05, 7.544524135123572e-05, 5.301284876740691e-05, 2.559371597579686e-05, 5.768227784320801e-06, 8.732855905530171e-08, 7.132926841768543e-06, 1.9767134130548074e-05, 2.9667588649340513e-05, 3.1586937479950805e-05, 2.5265742144729422e-05, 1.4598556304614014e-05, 4.943083600546833e-06, 2.3472586579739408e-07, 1.3326332953077376e-06, 6.137807040456354e-06, 1.119526594311063e-05, 1.3579120283542764e-05, 1.2241593263589935e-05, 8.184990242970457e-06, 3.601098283636161e-06, 5.885608904815693e-07, 1.3373373874280194e-07, 1.841941353117472e-06, 4.257944358414542e-06, 5.940096932690017e-06, 6.0121026479143445e-06, 4.560669664660971e-06, 2.4299782900739628e-06, 6.630913404243548e-07, 1.5656714395470153e-09, 4.773465496688194e-07, 1.5958084043463749e-06, 2.616170157509259e-06, 2.981644231029491e-06, 2.536155444242736e-06, 1.5739686116591016e-06, 5.988214607909723e-07, 5.167436355884841e-08, 9.5159053719414e-08, 5.771110544300432e-07, 1.1524400862786022e-06, 1.483646354994208e-06, 1.4083744257157672e-06, 9.944549104999943e-07, 4.690039825695306e-07, 9.433636942716075e-08, 8.246818549588441e-09, 1.936516705664857e-07, 4.999530461590781e-07, 7.376396460193729e-07, 7.819808996600669e-07, 6.178527402096803e-07, 3.4704774359445184e-07, 1.0642410943891085e-07, 1.7507402362970757e-09, 5.6252155933064356e-08, 2.109406980454496e-07, 3.682278695284434e-07, 4.3012624213007517e-07, 3.803685519541255e-07, 2.443360981488985e-07, 9.879984920426602e-08, 1.0913312654396631e-08] 
# h_125 = [118086.0844019247, 86746.77733552453, 61221.63701965479, 41325.78937858012, 26520.702022281137, 16045.106864946149, 9040.740568806938, 4655.772379791605, 2122.8583939466525, 806.3151513157343, 220.62445977201915, 25.023856552130177, 2.791050284181697, 32.64054525718581, 58.91505375881144, 65.35743229614904, 55.06122387681167, 37.22745323777594, 20.012793661412637, 7.872122280662924, 1.6666279763758296, 0.0026549950762827647, 0.7260159766348495, 1.9880828069777818, 2.7254399131109848, 2.6543413694278346, 1.9983630028871604, 1.1595139331250597, 0.47215833170039406, 0.09165346808219113, 0.0010378953991707896, 0.08353174269589854, 0.20626035671337187, 0.2782415884818724, 0.2698088403812809, 0.20043556462255183, 0.11145036752440834, 0.04002881104493588, 0.004437919940771539, 0.002518007911941026, 0.01932459592437692, 0.03765673106918203, 0.04601651035172749, 0.04160283871139065, 0.028598887229250834, 0.014032541562542387, 0.0037220052227906297, 2.607651561404082e-05, 0.001802660340613282, 0.0059298735492771095, 0.009254327405563461, 0.010002694008245336, 0.00815018229128178, 0.00493921771196644, 0.0019435593162955183, 0.0002543192417550139, 0.00010513319114938449, 0.0009798496239009098, 0.0020768003916025275, 0.002685303016129894, 0.0025440611132470058, 0.0018191789896970438, 0.0009175817017363871, 0.00024216233839538095, 4.554138288945064e-07, 0.00015420597719322646, 0.0004948122987676795, 0.0007787608905476476, 0.0008548077141622835, 0.0007040956471849128, 0.00042627040493769397, 0.00016265010018452048, 1.6424077660867047e-05, 1.748349713716676e-05, 0.00011882770747884086, 0.000238239427302138, 0.0003034230672344335, 0.00028474075807625397, 0.00020021351816632796, 9.67820172638341e-05, 2.188035189726861e-05, 3.152043158630467e-07, 2.682278117198438e-05, 7.450529437400066e-05, 0.00011196432089345664, 0.00011932420378579558, 9.554366722245142e-05, 5.528492445829988e-05, 1.8773674508240326e-05, 9.085123932225585e-07, 4.978239227911432e-06, 2.3125825193374254e-05, 4.2276200151557146e-05, 5.135630722260715e-05, 4.63655818842537e-05, 3.105640175836593e-05, 1.3704851272447613e-05, 2.261266170111485e-06, 4.900089845054274e-07, 6.881884221094655e-06, 1.6076244536748188e-05, 2.2495568013508184e-05, 2.2815337994493217e-05, 1.734766251226063e-05, 9.274484320616758e-06, 2.5715599847502876e-06, 6.5608428111924075e-09, 1.7861917430242804e-06, 6.027899621409186e-06, 9.927040373594896e-06, 1.1338991179219315e-05, 9.676316748531274e-06, 6.028787056610056e-06, 2.311072980978546e-06, 2.0596682321431157e-07, 3.4950796825717164e-07, 2.175807050858242e-06, 4.37595800433707e-06, 5.658755565068236e-06, 5.393518902543727e-06, 3.8263339707799415e-06, 1.8234544131713602e-06, 3.7301122078058465e-07, 2.6344783012156626e-08, 7.255783909810543e-07, 1.8992764862812982e-06, 2.8218099512954265e-06, 3.004324841197752e-06, 2.390391176803295e-06, 1.3535606184948285e-06, 4.221808150879645e-07, 7.3706321229607145e-09, 2.0689965062648347e-07, 7.996363607585726e-07, 1.4044657728136984e-06, 1.6626682715744524e-06, 1.4784324840354975e-06, 9.602564433761013e-07, 3.9415721322274884e-07, 4.5782500461548564e-08] 

plt.semilogy(Tvec, h_10, marker='.', label='1.0 MHz poly')
plt.semilogy(Tvec, ht_10, marker='x', label='1.0 MHz trig')
# plt.semilogy(Tvec, h_11, marker='.', label='1.1 MHz poly')
# plt.semilogy(Tvec, ht_11, marker='x', label='1.1 MHz trig')
plt.semilogy(Tvec, h_12, marker='.', label='1.2 MHz poly')
plt.semilogy(Tvec, ht_12, marker='x', label='1.2 MHz trig')
# plt.semilogy(Tvec, h_13, marker='.', label='1.3 MHz')
# plt.semilogy(Tvec, h_14, marker='1', label='1.4 MHz', markersize=10)
# plt.semilogy(Tvec, h_115, marker='1', label='1.15 MHz', markersize=10)
# plt.semilogy(Tvec, h_1175, marker='.', label='1.175 MHz')
# plt.semilogy(Tvec, h_1225, marker='.', label='1.225 MHz')
# plt.semilogy(Tvec, h_125, marker='1', label='1.25 MHz', markersize=10)
plt.xlabel('Transport Time (us)')
plt.ylabel('phonon')
plt.legend(loc='best',shadow=1)
plt.title('Diabatic transport of 88Sr at 1.2MHz, 120μm, STA 9th-order')

In [4]:
#### Diabatic transport of Sr Plot ####
Tvec = np.array([n/10 for n in range(10, 150)])
h_sin = [2172878.6577557228, 606026.9280873075, 49636.34054928855, 34618.853926838754, 203978.9423138365, 344181.15297503385, 373486.00131616025, 302426.1204290242, 185159.2695199228, 78008.42620020043, 14786.681639453238, 498.0507094652864, 18875.2999380585, 46023.53369966517, 62802.983830119585, 61402.18900667101, 45180.16669083053, 23789.627068004993, 6993.167578083872, 131.76245323201974, 2778.9637835607873, 10345.085703267549, 17181.144179153445, 19431.47707911123, 16405.301686379036, 10174.886401483327, 3981.4467775122644, 435.7245000617582, 391.11537483809985, 2879.0517346964994, 5924.3504090491915, 7684.892953899265, 7314.693180645252, 5189.426921333428, 2502.4119343987813, 537.127134313376, 20.294929721311878, 855.6903575750134, 2304.150764001324, 3449.5363064912117, 3678.298617558079, 2936.1480147834714, 1669.119399891248, 527.3950905020291, 10.638401624409722, 239.32940878114684, 947.1404254974965, 1667.0658252487437, 1992.4335461701628, 1772.9231909622138, 1156.1031302521458, 474.545934125292, 54.36233980489918, 50.97727417794833, 391.29528648002605, 838.1568925845274, 1129.9197267865313, 1116.0082570362342, 820.4070274635799, 409.3745365783291, 90.81090663667278, 3.54180206804697, 153.9702438442163, 427.0190805687275, 657.7659002277978, 720.9620049965329, 591.0174994838125, 344.7377097421392, 111.67524369934527, 2.3076816870074315, 53.14288292119556, 215.1323079237077, 387.0675582340089, 472.5837208707141, 429.31493042146064, 285.63725816227605, 119.55877122831531, 13.958809488569127, 13.333596857379563, 104.20270357350243, 227.1422068149048, 311.4655420140958, 312.77446511261303, 233.6734699683971, 118.45108855918505, 26.682294133831537, 1.0563561441940954, 46.59811990507555, 131.0900272153434, 204.75626820514555, 227.499154829594, 188.98716068683242, 111.67411814985802, 36.63743391484329, 0.7665260943983851, 17.86711560937742, 73.19142064146601, 133.22244190950025, 164.5115596374366, 151.1175110702123, 101.64247507809525, 42.99962469914302, 5.072947287183883, 4.895466514121707, 38.643202602787625, 85.0637594825146, 117.77135466668139, 119.38662093786971, 90.02212926231496, 46.04870443881192, 10.465660210621378, 0.4179650985773603, 18.596113309531585, 52.756579010225025, 83.08640569300594, 93.0665100386103, 77.92983099930622, 46.41095506049814, 15.343712835046203, 0.3234552147550776, 7.595547510159235, 31.34233257270325, 57.45946318800777, 71.45668820661044, 66.09511958298638, 44.75985374407643, 19.06291531976816, 2.263859642610627, 2.1988662598079083, 17.468236786062732, 38.694855817495615, 53.90508117082819, 54.977964449560034, 41.70463279699415, 21.459327551485792, 4.9055355157862595, 0.19703612031573725, 8.81604291950403, 25.149984331623184, 39.826003021442844]
h_tanh = [48060163.260307945, 43924981.453549005, 39879085.74315495, 35979432.778307684, 32269768.90954234, 28781647.461039957, 25535369.55843643, 22541911.66895183, 19804432.725548703, 17319984.99240721, 15080744.063989881, 13076256.47024448, 11292601.1088128, 9715085.002979007, 8327923.827615642, 7114708.4661493255, 6059964.72922772, 5147039.655135054, 4360800.178522423, 3686570.86588387, 3110591.553033941, 2620182.044941461, 2203703.7046465, 1850769.7590749126, 1552132.8979414918, 1299755.751559593, 1086663.6707557274, 906908.1993123235, 755434.5598978144, 627971.0713075469, 520926.03596825554, 431210.0492050703, 356268.4604861205, 293866.9922972331, 242091.0029465923, 199281.53603534208, 163995.26280308558, 134983.02064881055, 111164.09157888094, 91614.87586115279, 75562.65991217982, 62332.693545592425, 51415.30469338077, 42371.2409260946, 34856.88228206022, 28601.550032408737, 23392.292379187504, 19061.625452673055, 15475.1972054812, 12522.218476048234, 10108.691293868747, 8152.763961024731, 6580.865234203771, 5326.7504260914975, 4331.291137295536, 3542.3804611538626, 2915.1004752926465, 2411.8621507694693, 2003.233857297845, 1665.2362338189118, 1380.8435055110642, 1138.5278061392723, 930.089779275742, 750.9077623956596, 598.1565450825901, 469.8568206552251, 364.88168343055526, 280.9800236883878, 216.05249011461453, 167.43195053107567, 132.2208226713095, 107.31154168719695, 90.05799110500143, 77.9929301113498, 69.04977621136476, 61.74398743037355, 54.979547849947636, 48.12575499073426, 40.948614617411415, 33.541081147401464, 26.21021223923804, 19.42050188387164, 13.524988091651599, 8.805741088366172, 5.35593984303487, 3.0968307113996105, 1.8323962763744779, 1.3170055001804122, 1.3146265722802621, 1.6264671950587257, 2.101544713460274, 2.594138766527498, 3.025031508722179, 3.255243467480553, 3.229694215062474, 2.9336240385488974, 2.4199181166121484, 1.792808000613389, 1.179935737461439, 0.6858711459653475, 0.3649979771979563, 0.21166432575205996, 0.17547814471931172, 0.19444693427374776, 0.224698280165916, 0.2542096420071644, 0.2995244336664485, 0.38105337721446053, 0.502667566468675, 0.6407964617253198, 0.7532998208589203, 0.792929335860104, 0.7463246496449617, 0.6269776270312767, 0.4803902695053795, 0.35889803271220677, 0.3002989499126183, 0.30714205717749643, 0.35321719299163984, 0.3976562061637555, 0.4090402854011379, 0.3763670982731851, 0.3222992091108475, 0.28105514375847374, 0.2757205196638184, 0.31732580343055933, 0.3842098199292406, 0.44178247888099414, 0.45926950463184874, 0.42605942943010805, 0.3582364612725389, 0.28610573136960826, 0.24463575982340105, 0.24839772657763828, 0.2902618339673787, 0.3408804308623914, 0.3698605661458274, 0.3595008670812928, 0.31455175683481956, 0.2489047346743001]
h_tanh3 = [23887523.556994848, 19109602.12351228, 15084424.400823966, 11777231.439205347, 9116970.784500094, 7013673.518298666, 5372772.41850232, 4104396.8652828354, 3129052.3733366933, 2380090.295795433, 1804367.3975735216, 1360835.2125770603, 1018620.1262028379, 754728.4697747976, 552203.0697784451, 398277.4634558278, 283033.26528411696, 198393.31537317138, 137634.1160105015, 95071.08490044813, 65958.97674628465, 46445.17957091844, 33541.053673939285, 25028.04354952841, 19327.370250879783, 15352.563179625346, 12381.733357937474, 9960.728893860305, 7836.836408527933, 5909.480433683703, 4184.456344600467, 2722.154177484271, 1585.2699599972327, 798.0954610614076, 329.39557347144097, 101.83957741205158, 19.963255239534075, 1.9943419352365834, 0.6076183090870001, 4.682424909226471, 24.544484713919566, 70.88066216984917, 139.34692020134784, 208.35468617540442, 249.50999562794632, 243.37512972147795, 190.73989005124355, 113.08380610605315, 42.01311627471204, 3.615971750545649, 6.278346047550107, 38.11455792480784, 74.82172175015003, 93.25978915307036, 83.30060667376429, 51.89610339762553, 17.80598861059133, 0.41154934590824926, 8.977086372159969, 38.06214735763789, 71.15968976550947, 90.11988852303566, 85.0121338308938, 59.05822437746721, 26.33385114140816, 3.7700230178491476, 2.0282468195883023, 20.17656264974091, 46.79607781390577, 66.60332038789424, 68.80858067302519, 52.61501638731779, 27.167974347558474, 6.16034379645913, 0.28881148893343994, 11.665242615182297, 33.078816172148834, 52.27066835004706, 58.711814045260304, 49.16202999541034, 29.072006684592125, 9.309924185386148, 0.11086412008917826, 5.637052980159618, 22.01004358964401, 39.721423383081, 48.946439200842946, 44.814209682793376, 29.80568780013096, 12.131429274342727, 1.1166679373993693, 2.16328592982634, 14.015329942596582, 29.68701908706958, 40.41746559354735, 40.427237554194505, 29.898758258186543, 14.649199368624567, 2.8237020174252114, 0.44011327857137394, 8.25031691081816, 21.525318339137392, 32.78103506819992, 35.8838588645876, 29.27933700091824, 16.67535339382917, 4.8677967374060644, 0.002663800369277139, 4.3030671132635465, 15.012296196923982, 26.03357285230313, 31.306139326674003, 28.051380019037108, 18.15106507100411, 6.990933082201614, 0.4662428426387485, 1.8044691061755216, 9.921289277189631, 20.134413334614226, 26.785935820535585, 26.30256480446878, 19.044716925358486, 8.997661737864858, 1.5260187541376016, 0.45681699372004564, 6.0735657962618275, 15.069441566471594, 22.421854649802114, 24.137606153711097, 19.363577638511227, 10.749309994610597, 2.9360025374061047, 0.004873797194707061, 3.307359224291723, 10.815620421218197, 18.3031650363377, 21.67026124660545, 19.1414520712466, 12.152972948967752, 4.497527920064258]
# h_sta = [0.0007984020717201005, 0.00027825356594446404, 9.783870916897551e-08, 1.1075656674358416e-05, 1.0184259589628027e-05, 2.2461970262678205e-07, 0.00027530819915482234, 1.2705102496947733e-07, 2.293525009759526e-05, 7.237769973523518e-05, 1.6970963105790228e-08, 6.836880313648988e-06, 5.603857808107958e-07, 4.4271158079679325e-09, 2.4240024944510607e-05, 1.3739471151449422e-05, 1.4293800334719747e-06, 2.2083282176959823e-05, 2.0237915338164855e-07, 9.655276439969996e-07, 1.903925503644611e-06, 3.232562453968339e-06, 3.1425422934864692e-06, 3.037441531716761e-06, 1.9947206982385063e-06, 9.406152027798864e-10, 4.88756872132541e-07, 9.3232808057077e-07, 3.164302375279376e-07, 7.578533905961713e-07, 5.890749829108413e-07, 8.184088630285304e-07, 3.189486121369502e-09, 3.0135270811239197e-07, 5.786034880625951e-07, 1.0778964145031753e-09, 3.6000554481688666e-07, 9.349476693960186e-08, 4.73680625365775e-10, 1.9761004362687086e-08, 1.0999450040754514e-07, 1.0917407260079336e-08, 2.6065405053481375e-08, 3.6173866379259356e-09, 6.915078469500287e-08, 4.0217058777971936e-08, 3.253197048115991e-10, 1.0955263022783135e-08, 1.594836937443622e-09, 3.322325977438231e-09, 1.1517260357930339e-08, 7.949491286505768e-09, 4.514181915757331e-09, 4.804903185748429e-09, 3.7382910395380945e-09, 3.73503113021973e-09, 8.83821311501343e-09, 5.77851853509516e-09, 7.525830804863002e-10, 4.911122677435117e-09, 8.146234595134721e-10, 4.750286430128846e-09, 4.619624003926034e-09, 4.038888222180196e-09, 5.541631869978016e-09, 5.557337844099515e-09, 4.43131217988456e-09, 2.425020995085909e-09, 1.885337855517978e-09, 3.0044256023172902e-09, 1.2571369444556686e-10, 7.160501349452597e-10, 1.1208913055422286e-09, 1.6110984334194006e-09, 2.6936719338327393e-09, 3.389600323287775e-09, 2.5453409046431642e-09, 4.56437839496728e-10, 1.240364500885896e-09, 8.655638878667447e-11, 3.851418494819331e-10, 1.538715691184514e-11, 1.8783311855080604e-09, 1.2538803507056419e-09, 1.934818543102611e-09, 8.776170267220685e-10, 7.990858726218056e-10, 1.4000654580212478e-09, 5.071130744118142e-10, 1.6780792632603434e-09, 1.7543637861950362e-11, 1.3610627273683658e-09, 8.099482671862791e-10, 1.7130426934095275e-10, 3.4598606117332874e-11, 9.145925473484085e-10, 2.7375059134027235e-11, 1.3101002045283394e-09, 1.3996160774635322e-09, 4.543947969441655e-11, 7.184874532329076e-10, 7.597648908491487e-10, 6.148459977622211e-10, 1.5507575000136195e-12, 6.723874146543681e-10, 2.6912868515327556e-11, 1.1476573102673652e-09, 9.627289311919425e-10, 4.0214384556760243e-10, 1.289128916789332e-10, 1.1391904199687999e-10, 2.533383687573952e-10, 5.303518301163294e-10, 2.846830728077417e-10, 3.779633663644215e-10, 7.606990852521797e-10, 4.729648717972541e-10, 1.6845328167210193e-10, 4.3948356050692094e-10, 2.706824215517181e-10, 1.3643923250643995e-10, 1.404179117751503e-10, 2.060333858064842e-10, 2.5827714286175565e-10, 6.698469580115269e-10, 5.443520172024762e-10, 6.594645746213307e-11, 1.1217606092521354e-10, 2.359794750789788e-10, 1.8311645442230972e-11, 4.288014524861609e-10, 2.54287541657957e-10, 2.0428915138994965e-10, 4.957199889060241e-10, 2.4315415740146526e-10, 2.2781112884308924e-11, 1.7651569901085028e-11, 1.058467451251741e-11, 5.688871467710504e-11, 8.067882736829343e-11]
h_sta = [4.185502536906726e-08, 3.4597734393050696e-07, 8.831642338593252e-08, 2.0364407134684833e-08, 1.9117969522469555e-07, 2.7879766455400897e-07, 2.385334153739628e-07, 3.4604655425823213e-07, 1.1824705167756305e-08, 5.797854546268451e-09, 1.388029820146283e-08, 2.505794146823077e-07, 3.344665804244243e-08, 2.7310165826730937e-09, 3.850442042663309e-09, 5.436039766858491e-09, 1.3485953055044783e-08, 1.4096187650602013e-08, 7.499101093539558e-09, 4.0833463570834143e-10, 4.1360901882184366e-10, 1.1380264656370558e-09, 3.5853043766486555e-09, 1.0723412554959306e-08, 1.1069013187873e-08, 1.0570534704454063e-09, 1.312286823161189e-09, 2.115311685228674e-09, 2.115311685228674e-09, 8.529885528039229e-10, 1.8662647531232954e-09, 2.785268051671418e-09, 1.3215158202655099e-09, 2.0906182684473906e-09, 9.75373195661687e-10, 4.95602162935901e-10, 1.6546260674845963e-10, 4.335368874702576e-10, 2.0983081448660575e-10, 1.0780729101682085e-09, 1.6558540687647449e-09, 1.7345934162832015e-09, 1.026575441901978e-09, 4.890595967141558e-10, 4.81875740000611e-10, 4.1897777584641194e-10, 6.822224246698595e-11, 1.2656753658786458e-09, 1.3857629532061724e-09, 1.1494137109340247e-09, 1.328996296850725e-09, 8.86741712916149e-10, 3.325181232217458e-10, 1.7598015447220574e-10, 3.6424327606748205e-10, 9.204585172072644e-10, 1.1503812505961707e-09, 1.0739109135129161e-09, 7.922337317707986e-10, 8.251961677552461e-10, 4.669706858927536e-10, 9.060809752317684e-11, 1.7206903059394543e-10, 6.634262125632945e-10, 7.557709023933669e-10, 9.429361564972575e-10, 1.1968739457823798e-09, 7.302330493172759e-10, 5.950923959844918e-10, 1.5795233040070609e-10, 5.008433600977803e-11, 4.768264071315932e-10, 6.862720029554438e-10, 9.681595835141963e-10, 1.1461253831761595e-09, 8.225952361925669e-10, 6.523260788597242e-10, 3.07656969720221e-10, 1.9910121202051865e-10, 9.341686421545836e-11, 4.125038301864372e-10, 4.1515442796660466e-10, 9.117397209949089e-10, 9.187238193402854e-10, 5.962206756192804e-10, 3.7074386280299377e-10, 2.2705132050939573e-10, 2.873493762867448e-10, 4.815291452475182e-10, 6.323453617632113e-10, 4.915416493587807e-10, 8.645062161232263e-10, 8.013969033158534e-10, 4.0080331906090046e-10, 2.349111591399095e-10, 2.7691773079580713e-10, 2.0425482417868914e-10, 7.38970646362153e-10, 7.665514147477514e-10, 8.354566812390482e-10, 7.594911151391687e-10, 5.006314204172759e-10, 3.971319928853687e-10, 1.7919019080703064e-10, 3.704873116151437e-10, 4.403422411398328e-10, 8.156690828089588e-10, 7.647373972716409e-10, 8.237677169305503e-10, 5.197547298467144e-10, 4.155599950677841e-10, 2.7280732640823155e-10, 3.970601232864018e-10, 4.799835873042366e-10, 6.51326882707652e-10, 8.507659423003178e-10, 8.053451250743598e-10, 5.737805415876181e-10, 4.969330379373358e-10, 3.539319968637789e-10, 3.209750392501185e-10, 4.0291863751426604e-10, 5.716330240097883e-10, 6.948663075915433e-10, 8.993268198127533e-10, 7.192662500478882e-10, 4.503005825167443e-10, 3.4315637222240806e-10, 3.568735773213765e-10, 2.8981872668468954e-10, 6.264424701421536e-10, 6.402455839055758e-10, 7.556513185998472e-10, 7.942219913917313e-10, 5.467989016339993e-10, 3.4270035567415716e-10, 2.172390561573595e-10, 2.9448866453787506e-10, 4.271622129183402e-10, 5.475228885981058e-10]
# h_sta2 = [2.80567627933004e-07, 7.258199604426839e-11, 5.365928176277793e-11, 1.8476963787734096e-11, 2.2689206984559558e-11, 6.482258047186766e-08, 7.263096629750859e-12, 2.032481468922375e-11, 1.1206677594723897e-11, 2.032319405743324e-11, 1.259177239397298e-10, 8.229916864613956e-11, 1.833557457829625e-11, 1.822479442728762e-11, 1.8770846575186463e-11, 1.815491969047042e-11, 1.7023230038225845e-11, 2.125941501426394e-11, 1.4585383128726702e-11, 1.6969307645115912e-11, 6.246938439389739e-10, 1.7563061480464658e-11, 1.8466211852178304e-11, 1.1927952324235654e-10, 1.7459730825537957e-11, 1.837028215617862e-11, 1.7600502645242274e-11, 1.7850213232176232e-11, 2.0283025707334612e-11, 1.8238349589900212e-11, 3.232684070500453e-10, 1.7877963889076794e-11, 1.7868402113859282e-11, 2.3839917124926147e-11, 1.876056099747906e-11, 1.751895218931347e-11, 2.0840702738168942e-11, 1.665623129391535e-11, 1.9793906002990487e-11, 1.8307994761589175e-11, 1.293047979524847e-10, 1.6849377571821102e-11, 1.671313002455733e-11, 2.1403486527616977e-11, 2.2136764158753793e-11, 1.7839885211436097e-11, 1.798594169207595e-11, 1.0351950739957743e-12, 1.8095502932706204e-11, 5.0463988895825806e-11, 1.8902708787332597e-10, 1.8152040020722214e-11, 2.2456476381300255e-11, 1.57046740711241e-11, 1.7938329509254878e-11, 2.0448747587826062e-11, 1.8531348080155623e-11, 3.2147287260334735e-08, 1.3954109694391118e-11, 1.8714946397322957e-11, 1.8172687358185608e-10, 1.957851223543613e-11, 2.374298125435722e-11, 1.7993386665999472e-11, 1.735865164134833e-11, 1.7341552184619125e-11, 7.0232470710849605e-12, 2.2003448288935505e-09, 2.4686776794551104e-11, 1.800577898265688e-11, 1.0805230074531554e-10, 2.431563721452455e-10, 1.767813615372862e-11, 2.670579492202648e-10, 1.649361899017499e-11, 1.9943748022258655e-11, 2.4374836420774288e-11, 1.7989817705568062e-11, 1.493905345807659e-11, 1.7972505586908212e-11, 2.5622894590370252e-11, 1.791258406171002e-11, 2.393009247990694e-11, 1.869221073663066e-11, 2.484121323148112e-11, 1.8684787679945182e-11, 1.7740680342402105e-11, 1.79112201683932e-11, 1.7923061654277758e-11, 1.8790360422873714e-11, 2.6563123112228466e-11, 7.167802216499611e-12, 2.4649258096206894e-11, 1.8782179248677327e-11, 1.929295706962586e-11, 1.7775832291402498e-11, 1.7827678334198935e-11, 1.7447761109731475e-11, 1.787378935649081e-11, 7.253953319593088e-10, 2.2291112199977392e-11, 1.7620541567860433e-11, 1.7317433073612773e-11, 1.8094740422794385e-11, 1.8978118710843238e-11, 1.4164010946626751e-10, 1.797421649040866e-11, 1.741609493700708e-11, 2.5106886851274962e-11, 2.2050924460193705e-11, 1.437969972200411e-11, 1.7991687314731846e-11, 1.790581899763946e-11, 2.1749560257125242e-11, 2.4781371270647193e-11, 1.6619161979728428e-11, 2.4533069350658902e-11, 1.7255493642754982e-11, 1.7880743039633735e-11, 1.7917409924824074e-11, 1.7870642213747943e-11, 3.9784977592678975e-11, 1.7374469301920043e-11, 4.101935899909238e-11, 2.4890360457808337e-11, 1.7922244731460223e-11, 1.774663424258964e-11, 2.118087127956463e-11, 1.788946293893029e-11, 1.7923447314665858e-11, 1.811688891748064e-11, 1.7926051667615134e-11, 2.4059526804637508e-11, 1.6357861685261256e-11, 4.792544840646286e-11, 1.7952512327076293e-11, 8.776834790000646e-12, 1.7589248855970598e-11, 1.677599477462534e-11, 1.692292454667327e-11]
h_sta2 = [2.2474108453679474e-08, 4.706871159809965e-10, 5.062665112589084e-10, 4.862618073640078e-10, 4.869046986860683e-10, 3.311549867680295e-08, 4.919458367480999e-10, 5.004314735480421e-10, 4.892641921470926e-10, 4.913806983637859e-10, 3.0377391607497347e-10, 4.966012160086156e-10, 4.93890612532096e-10, 4.95121408974977e-10, 4.933639111483537e-10, 4.873143489570042e-10, 4.89301415611054e-10, 4.92845688774768e-10, 4.917115320538407e-10, 4.93559394643549e-10, 1.5500619309478578e-10, 4.935882069273801e-10, 4.929943956279558e-10, 4.917098595805044e-10, 4.929510512880466e-10, 4.96031532998558e-10, 4.961274392611864e-10, 4.946460078036433e-10, 4.938221964223764e-10, 4.95649134631227e-10, 4.6333712845069424e-10, 4.940015511729441e-10, 4.941320114453635e-10, 4.925476684559854e-10, 4.9278817364076e-10, 4.923608402361384e-10, 4.937800999051156e-10, 4.927227909469092e-10, 4.927737573298402e-10, 4.929940476477449e-10, 5.378423611172343e-10, 4.937009615692164e-10, 4.918664327902435e-10, 4.989949998965659e-10, 4.931216341237957e-10, 4.922687107813664e-10, 4.933194410137848e-10, 5.274543985553624e-10, 4.88492979929621e-10, 4.924161133523593e-10, 9.521856400122719e-10, 4.909122187352858e-10, 4.924520436283752e-10, 4.988210414004221e-10, 4.933418716383864e-10, 5.048178738312477e-10, 4.931184366476832e-10, 4.941585758943614e-10, 4.934202849285078e-10, 4.917111193653506e-10, 1.0856980327673441e-09, 4.918955866589446e-10, 4.955039744233115e-10, 4.899138992877057e-10, 4.932392083856714e-10, 4.931687056519853e-10, 4.926826763432022e-10, 4.88109885030091e-10, 4.907118790204881e-10, 4.929811289693291e-10, 9.845171204224347e-10, 4.930841568748523e-10, 4.935680774109487e-10, 4.797250093905725e-10, 4.928848096814982e-10, 4.931835192917096e-10, 4.934331005674795e-10, 5.035082786086468e-10, 4.966283051502018e-10, 4.935456198835407e-10, 7.52806452434146e-10, 4.933410231379723e-10, 4.934973335777598e-10, 4.931921987522677e-10, 4.93512848478387e-10, 4.924887360312811e-10, 4.90875867234792e-10, 4.930859621884946e-10, 4.932976200516546e-10, 4.933047123553005e-10, 5.689896923627882e-10, 4.813587098915127e-10, 4.946346577964713e-10, 4.908690094309845e-10, 4.933705905442338e-10, 4.935774131032055e-10, 4.930892465627528e-10, 4.927380327501829e-10, 4.934657822881126e-10, 1.185943702926627e-09, 4.788021988644012e-10, 4.933799678831498e-10, 4.928010028127198e-10, 4.931159787309161e-10, 4.958534241519652e-10, 4.402804973152156e-10, 4.929565969954271e-10, 4.943398428793404e-10, 4.938319916294285e-10, 4.792193842865322e-10, 4.695612097046302e-10, 4.936239405502859e-10, 4.933079974580084e-10, 4.930184718069867e-10, 4.854546359453492e-10, 4.92121417193834e-10, 4.953189407062226e-10, 4.930315867209456e-10, 4.951590943428904e-10, 4.930868974727654e-10, 4.922686021176019e-10, 4.386572287383242e-10, 4.959194068597682e-10, 4.723074320978351e-10, 4.92580494758444e-10, 4.929203438572651e-10, 4.926306058923309e-10, 4.924436532590746e-10, 4.931695540042017e-10, 4.933126096736466e-10, 5.018919776957517e-10, 4.932489978091169e-10, 4.932718619298303e-10, 4.947931557669325e-10, 4.93262855447948e-10, 4.934100804046131e-10, 4.933042119763778e-10, 4.92389878340602e-10, 4.93644571757085e-10, 4.932539360662506e-10]
h_trig = [2.3682847765014527e-07, 8.428723930154928e-11, 5.237622014168806e-11, 4.439147719735107e-11, 2.8608440905489526e-11, 1.2086921551513066e-07, 3.5258756726772646e-12, 4.520587711780528e-12, 2.3527159260953852e-12, 1.0700081215924835e-11, 1.199959933954395e-10, 1.593273799204405e-10, 1.987927365662314e-11, 2.3655979752877693e-11, 2.685492467505198e-11, 4.211694692394087e-11, 2.337533764739837e-11, 1.4397259329275028e-11, 1.5689558274659642e-11, 1.781475625652686e-11, 2.9244547500206125e-09, 1.631389310391483e-11, 1.7731736928741952e-11, 1.3553309858030725e-10, 1.7635647142508674e-11, 1.3395599496951831e-11, 1.7186424097664105e-11, 1.9875827999259892e-11, 1.962137002457263e-11, 1.8686354317599422e-11, 2.4194409762106605e-10, 1.783122703956339e-11, 1.776150891486096e-11, 2.721364602553207e-11, 2.095791306189476e-11, 2.0121519365627866e-11, 2.089596445811017e-11, 1.674922956395785e-11, 2.0768279981533974e-11, 1.8094048761033018e-11, 2.873047624485816e-10, 1.719442064729003e-11, 1.6539079749076203e-11, 2.2568743158308495e-11, 2.275150038347787e-11, 1.8055307665691013e-11, 1.788455799291595e-11, 9.541854209040272e-13, 1.8378448721263393e-11, 1.3986896772582504e-10, 3.1277781952303426e-10, 1.7594618506550438e-11, 2.241310135488746e-11, 1.3999243545795261e-11, 1.7852096266358568e-11, 1.3521746396081843e-10, 1.846236060843975e-11, 9.7843309303405e-08, 1.4318647939903694e-11, 1.9009388621665036e-11, 5.548221641729324e-10, 1.9745375467082414e-11, 2.4246171317674103e-11, 1.8268928883627476e-11, 1.7052429727695088e-11, 1.6691024782526682e-11, 1.3271683484330783e-11, 5.416868606750309e-09, 2.4917807697874676e-11, 1.856235338740302e-11, 2.4512664712591255e-10, 2.4402202354644734e-10, 1.7721752506704058e-11, 3.2278217919653015e-10, 1.5800812672088432e-11, 2.102014197341037e-11, 2.4583450480230495e-11, 1.914766349243033e-11, 1.0676633189110346e-10, 1.790234991084161e-11, 1.7780590134968482e-11, 1.790236648871286e-11, 3.079786982447005e-10, 1.9654893553568435e-11, 2.5049632624214505e-11, 1.901175038482826e-11, 1.597934986813546e-11, 1.786483315052191e-11, 1.763308865098356e-11, 1.8942385173543295e-11, 8.547170251597398e-11, 6.305257191794723e-11, 2.3499936172624624e-11, 1.9186468878064102e-11, 2.025705066848623e-11, 1.778636483147487e-11, 1.791732700072575e-11, 1.9325044959820466e-11, 1.777312737778974e-11, 8.86549620409622e-10, 3.155458622427424e-11, 1.950581387649232e-11, 1.6963558228430515e-11, 1.7115171003231354e-11, 2.0235705392494885e-11, 4.4159385552825815e-10, 1.798409315105654e-11, 3.208041885958177e-12, 2.577627859900608e-11, 5.964396009720464e-11, 4.001666172032974e-12, 1.827761728589136e-11, 1.7902490822991883e-11, 2.274544565355947e-11, 3.941137605796334e-11, 1.5398670637302322e-11, 2.4955422493776378e-11, 2.5081327262254885e-11, 1.7847208860219705e-11, 1.7978751609167525e-11, 8.516306256020076e-13, 4.6416640793262975e-11, 1.7249077579146007e-11, 1.9254289081714887e-11, 2.560979163157732e-11, 1.794257795735317e-11, 1.736644323505514e-11, 7.746799207154015e-10, 1.788106611436763e-11, 1.7973439929455684e-11, 2.555898803703077e-11, 1.796031568093831e-11, 2.392967560745231e-11, 1.6323049334793657e-11, 6.262740792589495e-10, 1.8000558895352488e-11, 7.98288096842377e-12, 1.864326187758173e-11, 5.346932091678085e-12, 1.670772445831465e-11]
h_lin = [1828476.5249640301, 3118093.9932664176, 3546327.2856468116, 3021603.4672222733, 1925003.7653413955, 812664.3209688087, 127861.59292361983, 28766.44339805597, 379153.93602281314, 870405.2846419503, 1196795.8610823527, 1195345.3127510282, 895240.9609673193, 468838.8271214142, 124533.84731534956, 0.0003044000884173032, 106096.62045367541, 340206.90284649946, 552658.3428094677, 626839.6345650798, 531938.2810241345, 326965.3725204113, 119956.65336734774, 7634.0840532246275, 28313.825940492854, 149270.2626571394, 291125.82761045796, 373035.5813215758, 353627.5949098298, 248082.15437178573, 114280.24970020459, 19473.144333940185, 4712.809473627739, 66432.65670028816, 162296.3495473096, 236404.11598537775, 249118.25686760322, 196144.8973433139, 107648.35582308413, 29870.78488286253, 9.342158652817093e-05, 27575.66485092612, 91716.39624793919, 154264.0686422705, 180771.04673336982, 158243.54084110158, 100197.41789826524, 37808.20944401516, 2471.29918624975, 9402.872061480068, 50790.07770368713, 101407.42438998261, 132841.30941423128, 128656.04962345441, 92114.13379566387, 43278.898949810246, 7513.979312947822, 1851.9079289391777, 26562.77871234766, 66002.77964303784, 97694.21305425448, 104571.09558780045, 83589.8604585499, 46543.658560634525, 13097.199276369707, 2.3055827356280924e-05, 12417.401384246752, 41831.0238235061, 71216.71237053542, 84461.77624391411, 74799.7364670211, 47890.81847390764, 18266.805745237034, 1206.7778693690036, 4639.166119178535, 25307.14214865234, 51013.3247887981, 67463.89096097267, 65939.86695143465, 47633.63573689382, 22574.238765094455, 3952.750063702253, 982.1758872929315, 14201.744422137821, 35560.022440197434, 53040.03419322112, 57202.950317365, 46056.86886799785, 25822.563422861615, 7317.455606159908, 1.0151467444013481e-10, 7031.533569187871, 23836.7267178274, 40843.278129504724, 48736.30158252003, 43419.98644934726, 27964.68500436423, 10729.180788619307, 712.7205043898927, 2755.002041775197, 15110.958790590083, 30621.763273948993, 40707.80898161978, 39992.33189150778, 29030.474737169985, 13826.771190540085, 2432.780903263909, 607.3064775115813, 8821.673071845695, 22191.658351035952, 33246.92706532665, 36003.2632110405, 29111.839880921805, 16392.666612291803, 4664.911981050773, 1.4612363431163858e-06, 4517.452544275898, 15378.266097111375, 26446.172003568397, 31678.070389448483, 28325.620118448453, 18310.194081044512, 7050.04714864557, 470.0323522438599, 1822.9724918432587, 10032.666747007825, 20400.48311739016, 27208.451250813938, 26814.931248232744, 19528.24805565572, 9329.169898636093, 1646.4896131392063, 412.34305210986946, 6007.1247029945, 15152.881895261145, 22767.523876182015, 24730.39127583069, 20054.30238322084, 11323.256898968937, 3230.3885557925114]
plt.semilogy(Tvec, h_lin, marker='x', label='linear')
plt.semilogy(Tvec, h_sin, marker='.', label='sin')
plt.semilogy(Tvec, h_tanh, marker='x', label='tanh N=5')
# plt.semilogy(Tvec, h_tanh3, marker='x', label='tanh N=3')
# plt.semilogy(Tvec, h_sta, marker='.', label='STA 5th-order')
plt.semilogy(Tvec, h_sta2, marker='.', label='STA 9th-order')
# plt.semilogy(Tvec, h_trig, marker='s', label='STA trig')
plt.xlabel('Transport Time (us)')
plt.ylabel('phonon')
plt.legend(loc='best',shadow=1)
plt.title('Diabatic transport of 88Sr at 1.2MHz, 120μm')



Text(0.5, 1.0, 'Diabatic transport of 88Sr at 1.2MHz, 120μm')

In [6]:
#### Transport protocol comparison ####
t_anal = np.linspace(0, 7, 100)*1e-6
q_lin = np.array([120*(min(max(1e-6, t), 6e-6)-1e-6)/5e-6 for t in t_anal])
q_sin = np.array([qsin(t, (1E-6, 6E-6)) for t in t_anal])
q_tanh = np.array([qtanhN(t, (1E-6, 6E-6), N=5) for t in t_anal])
q_sta = np.array([qsta(t, (1E-6, 6E-6), f=5e-6*1.2e6) for t in t_anal])
q_sta2 = np.array([qsta2(t, (1E-6, 6E-6), f=5e-6*1.2e6) for t in t_anal])
q_trig = np.array([qtrig(t, (1E-6, 6E-6), f=1.2e6, mu=1) for t in t_anal])

fig, ax = plt.subplots()
ax.plot(t_anal/t_anal[-1], q_sin, color='k', ls='dashdot', lw=2, label='sin')
# ax.plot(t_anal/t_anal[-1], q_trig, color='k', ls='dashdot', lw=2, label='STA trig')
ax.plot(t_anal/t_anal[-1], q_tanh, 'k:', lw=2, label='tanh N=5')
ax.plot(t_anal/t_anal[-1], q_sta2, 'k--', lw=2, label='STA 9th-order')
# ax.plot(t_anal/t_anal[-1], q_sta, color='k', lw=2, label='STA 5th-order')
ax.plot(t_anal/t_anal[-1], q_lin, 'k', lw=2, label='linear')
ax.set_ylabel('Potential minimum displacement (μm)')
ax.set_xlabel('t/T')
plt.legend(loc='best',shadow=1)

dx = t_anal[1]-t_anal[0]
dqdt = np.gradient(q_sin, dx)
dsin = np.gradient(dqdt, dx)

dqdt = np.gradient(q_tanh, dx)
dtanh = np.gradient(dqdt, dx)

dqdt = np.gradient(q_sta, dx)
dsta = np.gradient(dqdt, dx)

dqdt = np.gradient(q_sta2, dx)
dsta2 = np.gradient(dqdt, dx)

dqdt = np.gradient(q_lin, dx)
dlin= np.gradient(dqdt, dx)

dqdt = np.gradient(q_trig, dx)
dtrig= np.gradient(dqdt, dx)

ax2 = ax.twinx()
ax2.plot(t_anal/t_anal[-1], dsin, color='r', ls='dashdot', lw=2, label='sin')
# ax2.plot(t_anal/t_anal[-1], dtrig, color='r', ls='dashdot', lw=2, label='STA trig')
ax2.plot(t_anal/t_anal[-1], dtanh, 'r:', lw=2, label='tanh N=5')
ax2.plot(t_anal/t_anal[-1], dsta2, 'r--', lw=2, label='STA 9th-order')
# ax2.plot(t_anal/t_anal[-1], dsta, color='r', lw=2, label='STA 5th-order')
ax2.plot(t_anal/t_anal[-1], dlin, 'r', lw=2, label='linear')
ax2.set_ylabel('Acceleration', color='red')
ax2.tick_params(axis='y', colors='red')
plt.title('Comparison of transport protocol & acceleration')

Text(0.5, 1.0, 'Comparison of transport protocol & acceleration')

In [ ]:
def sinc_interp(x, s, u):
    """
    Interpolates x, sampled at "s" instants
    Output y is sampled at "u" instants ("u" for "upsampled")
    
    from Matlab:
    http://phaseportrait.blogspot.com/2008/06/sinc-interpolation-in-matlab.html        
    """
    
    if len(x) != len(s):
        raise Exception('x and s must be the same length')
    
    # Find the period    
    T = s[1] - s[0]
    
    sincM = np.tile(u, (len(s), 1)) - np.tile(s[:, np.newaxis], (1, len(u)))
    y = np.dot(x, np.sinc(sincM/T))
    return y

## Splitting

In [ ]:
%matplotlib inline
t_anal = np.linspace(0, 20, 400)*1e-6
t_crit = 2e-6
a_final = np.array([1e-12*get_alpha_from_freq(1.2E6, 40*amu)*(1-t/(t_crit)) for t in t_anal])
b_final = np.array([2e-7*min(1, t/t_crit) for t in t_anal])

fig, ax = plt.subplots()

y_eval = np.linspace(-60, 60, 1000)        # x-array
line, = ax.plot(y_eval, [a_final[0]*y**2 + b_final[0]*y**4 for y in y_eval])

def animate(i):
    line.set_ydata([a_final[i]*y**2 + b_final[i]*y**4 for y in y_eval])  # update the data
    return line,

#Init only required for blitting to give a clean slate.
def init():
    line.set_ydata(np.ma.array(y_eval, mask=True))
    return line,

ani = animation.FuncAnimation(fig, animate, np.arange(1, len(t_anal)), init_func=init,
    interval=25, blit=True)
HTML(ani.to_jshtml())


In [ ]:
h_CP = []
f_CP = []
for bmax in 2*np.logspace(-12, -7, 60):
    t_crit = 15e-6
    t_anal = np.linspace(0, 2e6*t_crit, 1000)*1e-6
    q_final = np.array([0]*len(t_anal))
    a_final = np.array([1e-12*get_alpha_from_freq(1.2E6, 40*amu)*max(0, 1-t/(t_crit)) for t in t_anal])
    b_final = np.array([bmax*min(1, t/t_crit) for t in t_anal])

    Nion = 2
    q_ions = elem_charge*   np.array([1, 1])
    m = amu*                np.array([40, 40])
    x0dot = um*             np.array([0, 0])
    x0guess = um*           np.array([q_final[0]-1, q_final[0]+1])
    x0 = find_ic(q_final[0]/1e6, a_final[0]*1e12, Nion, x0guess, b0=b_final[0]*1e24)

    heat_start = t_anal[-1]
    heat_end = heat_start+42e-6
    t_eval = np.linspace(heat_start, heat_end, 4096)
    outsol, n, y_eval, q_eval = heat_sim(t_eval[-1], q_final, t_eval, a=a_final, b=b_final, deg=1)
    
    N = 2**16
    dx = t_eval[1]-t_eval[0]
    ion_w = np.fft.rfftfreq(N, dx)
    ion_fft = np.fft.rfft((y_eval[0]-q_eval) - np.average((y_eval[0]-q_eval)), N)
    peaks, _ = find_peaks(np.abs(ion_fft), np.max(np.abs(ion_fft))/4)
    f_peak = ion_w[peaks[-1]]
    print('end freq: ', f_peak)
    f_CP.append(f_peak)
#     window = gaussian(ion_w, f_peak, 0.1E6)
    
#     plt.plot(ion_w, np.abs(ion_fft), marker='.')
#     plt.plot(ion_w, window)

#     h_CP.append(get_heat(np.fft.irfft(ion_fft*window, N)[:len(t_eval)], 2*pi*f_peak))
print([float('{:0.2f}'.format(i)) for i in f_CP])

In [ ]:
beta_vals = [2e-6, 2e-7, 2e-8, 2e-9]
t_crit_vals = np.array([1e-6, 2e-6, 5e-6, 10e-6])
heat_CP = [[12.354, 7.652, 0.241, 0.424], [49.637, 7.278, 2.080, 0.595], [6569.1, 908.3, 23.3, 14.3], [578193.9, 174325.8, 35175.8, 7031.0]]

for i, b in enumerate(beta_vals):
    plt.semilogy(t_crit_vals*1e6, heat_CP[i], marker='.', label=b)
plt.axhline(11, ls='--', color='r', label='Lamb-Dicke Regime', lw=3)
plt.legend(loc='best', shadow=1)
plt.ylabel('phonon')
plt.xlabel('critical time (us)')
plt.title('Ca splitting heat beta dependence')

In [ ]:
beta_vals = 2*np.logspace(-10, -7, 60)
heatCP_10 = [459768.74, 388241.21, 327286.49, 275405.45, 231168.22, 193988.97, 161864.82, 134541.53, 111333.85, 91664.83, 75044.36, 61049.87, 49318.83, 39535.03, 31424.2, 24799.53, 19337.95, 14916.49, 11372.53, 8563.32, 6363.83, 4664.72, 3371.17, 2401.94, 1687.56, 1170.2, 802.14, 544.97, 368.42, 249.29, 170.25, 118.94, 85.74, 64.5, 50.45, 40.1, 31.15, 22.86, 16.36, 13.87, 16.74, 22.48, 24.81, 19.87, 12.04, 8.74, 8.09, 5.77, 8.47, 11.93, 4.61, 3.56, 4.76, 5.44, 2.5, 3.59, 2.72, 1.85, 2.93, 0.59]
heatCP_4 = [1910410.59, 1656378.54, 1437064.08, 1242970.99, 1073155.02, 924558.37, 794584.02, 682504.05, 583337.51, 496724.66, 421663.44, 357140.69, 300529.98, 251614.75, 209973.89, 173906.24, 143105.92, 117212.86, 95146.25, 76623.14, 61310.97, 48543.47, 38063.04, 29535.25, 22722.88, 17248.16, 12931.23, 9571.08, 7007.79, 5055.14, 3601.61, 2538.73, 1775.09, 1236.24, 864.03, 606.44, 429.67, 306.98, 220.23, 156.34, 108.99, 75.17, 53.36, 43.3, 43.39, 50.33, 58.86, 63.21, 59.88, 50.33, 40.46, 35.88, 35.09, 29.42, 15.56, 5.8, 13.53, 26.57, 21.22, 6.47]
plt.loglog(beta_vals, heatCP_10, marker='.', label='tcrit = 10us')
plt.loglog(beta_vals, heatCP_4, marker='.', label='tcrit = 4us')
plt.xlabel('beta')
plt.ylabel('phonon')
plt.title('CP @ half transport time')
plt.legend(loc='best', shadow=1)


In [ ]:
beta_vals = 2*np.logspace(-12, -7, 60)
f_CP4 = [81825.26, 86288.45, 89263.92, 95214.84, 99678.04, 104141.24, 108604.43, 113067.63, 120506.29, 124969.48, 130920.41, 136871.34, 142822.27, 150260.93, 157699.58, 165138.24, 172576.9, 181503.3, 188941.96, 197868.35, 208282.47, 218696.59, 229110.72, 239524.84, 251426.7, 263328.55, 276718.14, 290107.73, 304985.05, 319862.37, 336227.42, 354080.2, 371932.98, 391273.5, 410614.01, 432929.99, 455245.97, 480537.41, 505828.86, 534095.76, 563850.4, 595092.77, 627822.88, 663528.44, 702209.47, 743865.97, 787010.19, 833129.88, 882225.04, 935783.39, 990829.47, 1050338.75, 1114311.22, 1181259.16, 1252670.29, 1327056.88, 1407394.41, 1492195.13, 1582946.78, 1678161.62]
f_CP15 = [74386.6, 78849.79, 81825.26, 86288.45, 90751.65, 95214.84, 99678.04, 104141.24, 108604.43, 114555.36, 119018.55, 124969.48, 130920.41, 136871.34, 142822.27, 150260.93, 159187.32, 166625.98, 174064.64, 182991.03, 191917.42, 200843.81, 211257.93, 221672.06, 233573.91, 245475.77, 258865.36, 272254.94, 285644.53, 300521.85, 316886.9, 334739.69, 354080.2, 373420.72, 394248.96, 416564.94, 440368.65, 465660.1, 493927.0, 522193.91, 553436.28, 586166.38, 620384.22, 657577.51, 697746.28, 739402.77, 784034.73, 831642.15, 880737.3, 934295.65, 990829.47, 1050338.75, 1114311.22, 1181259.16, 1252670.29, 1327056.88, 1407394.41, 1492195.13, 1582946.78, 1678161.62]
plt.loglog(beta_vals, f_CP4, marker='.', label='tcrit = 4us')
plt.loglog(beta_vals, f_CP15, marker='.', label='tcrit = 15us')
plt.loglog(beta_vals, 8.37830*(1e24*beta_vals)**0.3*(5/3)**0.5, 'k--', label='Home equation')
plt.xlabel('beta (V um^-4)')
plt.ylabel('frequency at CP (Hz)')
plt.title('CP @ half transport time')
plt.legend(loc='best', shadow=1)

### Kaufmann Splitting

In [ ]:
eps0 = ct.epsilon_0
bmax = 2e-10
K = ct.e/(4*pi*eps0)
Ca = 40*ct.m_u
Sr = 88*ct.m_u
Ba = 133*ct.m_u
d_CP = 0.5*(2*K/(bmax*1e24))**0.2
mass = Sr

t_anal, dx = np.linspace(0, 50, 5000, retstep=True) ## us
d_final = np.array([d_t(t, (t_anal[0], t_anal[-1]), qspan=(2.1542814, 60)) for t in t_anal]) ## um (half the separation)
t_CP = interp1d(d_final, t_anal).__call__(d_CP*1e6)
print('d_CP: ', d_CP*1e6, 't_CP: ', t_CP)

b_final = np.array([bmax*qsin(t, (0, t_CP), qspan=(0, 1)) if t<t_CP else bmax*(1-0.9*qsin(t-t_CP, (t_CP*0.5, t_CP*1.5), qspan=(0, 1))) for t in t_anal]) ## V/um^4
# b_final = np.array([0.5*Ca/ct.e*(2*pi*1.2e6)**2*d**(-2) - K/d**(-5) for d in d_final*1e-6])/1e24
a_final = 1e-12*np.array([K*d**(-3) - 0.5*b*d**2 for b, d in zip(b_final*1e24, 2*d_final*1e-6)]) ## V/um^2

w_final = np.array([(p_derivative(phi, point=[t, d, a, b, mass], order=2)*ct.e/mass)**0.5 for t, d, a, b in zip(t_anal*1e-6, d_final*1e-6, a_final*1e12, b_final*1e24)])
w2_final = np.array([(ct.e/mass*(2*a + 3*b*d**2))**0.5 for a, b, d in zip(a_final*1e12, b_final*1e24, d_final*2e-6)])
w2_stretch_final = w2_final*(1+4*ct.e*K/(mass*(w2_final)**2*(2e-6*d_final)**3))**0.5

fig, ax = plt.subplots()
plt.axvline(t_CP/t_anal[-1], color='r', ls='--', label='critical point')
ax.plot(t_anal/t_anal[-1], d_final, 'k--', lw=3, label='ion position')
ax2 = ax.twinx()
ax2.plot(t_anal/t_anal[-1], a_final/a_final[0], label='a(t)')
ax2.plot(t_anal/t_anal[-1], b_final/bmax, label='b(t)')
# ax2.plot(t_anal/t_anal[-1], w_final/(2*pi*1e6), label='w, numerical')
# ax2.plot(t_anal/t_anal[-1], w2_final/(2*pi*1e6), ls='--', label='w, Home (2006)')
# ax2.plot(t_anal/t_anal[-1], w2_stretch_final/(2*pi*1e6), ls='--', label='w+, Home (2006)')
ax.legend(loc='best', shadow=1)
ax2.legend(loc='best', shadow=1)
ax.set_xlabel('t/T')
ax.set_ylabel('um')
ax2.set_ylabel('MHz')

# t_anal *= 1e-6
# q_final = np.array([0]*len(t_anal))

# heat_start = t_anal[-1]
# heat_end = t_anal[-1] + 25e-6

# t_eval, dx = np.linspace(heat_start, heat_end, 4096, retstep=True)
# outsol, n, y_eval, q_eval = heat_sim(t_eval[-1], q_final, t_eval, showplot=False, a=a_final, b=b_final, deg=1)

# N = 2**16
# ion_w = np.fft.rfftfreq(N, dx)
# ion_fft = np.fft.rfft((y_eval[0]-q_eval) - np.average((y_eval[0]-q_eval)), N)
# peaks, _ = find_peaks(np.abs(ion_fft), np.max(np.abs(ion_fft))/4)
# f_peak = ion_w[peaks[-1]]
# print('end freq: ', f_peak)
# window = gaussian(ion_w, f_peak, 0.1E6)

# n = get_heat(np.fft.irfft(ion_fft*window, N)[:len(t_eval)], 2*pi*f_peak, m_=mass)
# print('heat: ', n, '\n')


In [2]:
Tftot = np.linspace(5, 100, 96)
h_T_Catot = [16.185987879921658, 1.0214616823252434, 0.16551334119871794, 0.3455686513194434, 0.08971978886884781, 0.003968426055437185, 0.05031408001256366, 0.021148400542029062, 0.006871196905863802, 0.001215221717728232, 0.0029480580800028567, 0.003520545106521591, 0.0018741061825209673, 0.00029132815783059596, 0.0006433449116579165, 0.0010484178999359609, 0.0007845918873466076, 0.0005450327457897577, 0.00011210424772274242, 0.0003974874635024491, 0.0002814803529113055, 2.2602715304068003e-05, 0.00013498181160561797, 0.00015631487197869173, 0.00014111452266787924, 5.525346889278119e-05, 6.235255737864706e-05, 3.1550536477511206e-05, 0.00011366073984084505, 6.367105470847399e-05, 1.4766153009992504e-05, 0.0001026471479574828, 9.728359623286573e-05, 0.00011476518384072897, 2.1595402447752856e-05, 5.852913852670814e-05, 9.202298207957093e-05, 0.00010110655507847178, 9.289407033709327e-06, 7.838557470957584e-05, 5.414574536178698e-05, 2.3106262052002756e-05, 1.6487448167521993e-06, 5.05912307954243e-05, 3.523799800046869e-05, 2.4142934083315145e-05, 2.540439356243646e-05, 7.363256642137426e-05, 0.07072680207316834, 0.6338710424719675, 0.4843594082374225, 0.38517747821462184, 0.30264058708829533, 0.24777326195051091, 0.19744275553361298, 0.15719586171974284, 0.12028172619715005, 0.09467539144182577, 0.06793249661675771, 0.05185078497052458, 0.03706382821624687, 0.024007055768693577, 0.016953834510846876, 0.011274559684027668, 0.005127816812425065, 0.0032290539482161757, 0.0009650927644726439, 7.296152994878973e-05, 0.00023877604218484834, 0.0010230130167420903, 0.0031657580602470784, 0.005501192614125068, 0.009441970787949531, 0.011810989150008517, 0.01461373294658669, 0.018252366207163643, 0.02203337864122257, 0.023223934558233895, 0.02820743714141526, 0.03262383711979542, 0.035898469939122185, 0.03980028251304958, 0.04338831687496749, 0.047297208904352586, 0.05112096575139969, 0.05584791370489891, 0.056739173573565185, 0.06045617046615881, 0.0640265626753818, 0.0679321275105229, 0.07066638392558956, 0.07312691328621734, 0.0771730121282248, 0.08201909023832674, 0.08336125282446558, 0.0863440050970002]
h_T_Srtot = [73.59028672135285, 30.856633644195764, 13.226685626200236, 3.5936697666309, 0.37284732984315966, 0.021005796688595833, 0.16570715410822318, 0.15944320121335637, 0.07066075962176926, 0.013243962252939391, 0.002103504267233568, 0.009466089931808122, 0.01362670866509031, 0.009488447502967486, 0.004759949495924997, 0.0013323841281448715, 0.0005562514217962066, 0.0010836244123294632, 0.001692200647185596, 0.0015822392334407477, 0.0009845672513797345, 0.0003844692936882051, 0.00014300752081259014, 0.00018885186809801055, 0.0004220225334035303, 0.0004171086687019293, 0.00025798544258633, 0.0001577435175446184, 6.535694881952436e-06, 5.893193340897832e-05, 0.00011482439653978909, 0.00015887561804930607, 8.31350020933703e-05, 4.892362692088923e-05, 2.5555521949835615e-05, 1.7853658898927315e-05, 2.1489248465407862e-05, 5.837500914348931e-05, 4.3854514528709106e-05, 2.0358660901435142e-05, 1.0561698544249413e-05, 3.7743616093842994e-07, 3.596678793910042e-05, 1.0954060004810071e-05, 3.6307069230211004e-05, 1.4753256972289379e-05, 9.833765748968052e-06, 7.305699526256261e-06, 1.1467758565939277e-06, 1.4900638947474939e-05, 1.5125700429441127e-05, 1.2942318867930198e-05, 2.2008338390453923e-06, 1.495447215191625e-06, 8.035708390691867e-06, 9.298570199300155e-06, 7.705033390839433e-06, 1.0329160011184787e-05, 2.9798486740388284e-06, 3.974967630535793e-06, 6.44506498881954e-06, 1.4573724715162538e-05, 1.487276674129844e-05, 5.404042667538533e-06, 2.237836763185851e-06, 1.9276825094360036e-06, 1.9197005164598197e-06, 3.6635051648415516e-06, 2.973655541221378e-06, 1.1170403615727672e-05, 7.967050520916684e-06, 4.5180030415198446e-06, 2.5944166119015022e-05, 0.0006488731133306599, 0.17309637331456867, 0.2779582935083059, 0.23217125958552495, 0.18528591901012237, 0.15176223351487292, 0.12667402353024898, 0.10352867763925556, 0.08830499578978633, 0.07344930604088604, 0.06043645308057191, 0.05151170330550344, 0.042050640870477114, 0.035636414569607845, 0.02958222398097943, 0.024168409299286573, 0.019608734518994347, 0.015328345936563969, 0.012270880120227291, 0.009975966436077417, 0.007788777592490783, 0.005405548942287695, 0.004074699916290544]
h_T_Batot = [324.44646393354145, 71.39296249320611, 31.81090837009645, 17.325308844760215, 7.655640898690108, 2.3683260320651547, 0.3658768870439228, 0.004548456253096214, 0.09299346606363504, 0.1522638356572347, 0.11851476272324145, 0.057195341859994614, 0.01600614977194423, 0.001946958271878572, 0.003916173141739661, 0.008854329257460698, 0.011036328515886974, 0.008762092028005517, 0.005474803078303233, 0.0022663318667095556, 0.0006629489191536444, 0.00047974615478823315, 0.0009379510385374646, 0.0014197535194536612, 0.0013390832516639148, 0.001169968819617034, 0.0006017135214857696, 0.00031061905144896134, 0.0001065584127095185, 0.00010771788843996398, 0.00024971794819971205, 0.0003312008743555955, 0.00035025946268838257, 0.00032552917889445076, 0.0001725214012371399, 5.29967798530069e-05, 4.260633104911853e-05, 2.7930665227538926e-05, 7.188106730636964e-05, 0.00014575183469311373, 0.00011347431234908371, 7.974455914481941e-05, 3.774481579442432e-05, 1.6485043555955916e-05, 8.67165114062526e-06, 2.849224518055213e-05, 4.763354438216879e-05, 4.62679707887374e-05, 4.143296764607723e-05, 2.48350044314328e-05, 1.0165962974397692e-05, 4.103965477130147e-06, 1.2373383231568168e-05, 1.831011471134222e-05, 1.9873795382298877e-05, 2.2239467270705275e-05, 1.8614506667783302e-05, 7.33136112389106e-06, 1.1460438091138341e-05, 5.961669502280354e-06, 7.37187442387621e-06, 1.5721785867674752e-05, 2.20747890099713e-05, 4.880328521174434e-06, 9.756314761204124e-06, 3.508980916225064e-06, 3.877220411798649e-06, 4.819364795929359e-06, 4.098820690844012e-06, 8.562371183439422e-06, 1.3367441469266907e-05, 1.1836038803651022e-05, 5.5010217146675654e-06, 6.067229442348151e-07, 8.920477643556414e-07, 2.8994446238331945e-06, 8.804477966070057e-06, 8.824622388386294e-06, 1.0002304526217946e-05, 2.9069834607869394e-06, 1.125365213462017e-06, 4.409889235921348e-06, 2.1298773711409652e-06, 6.973972209263285e-06, 2.144361569336489e-06, 7.57140389119972e-06, 6.5310861255661174e-06, 3.498778868756514e-06, 7.5221338636962756e-06, 6.482632110358317e-06, 6.655339174267692e-05, 0.000610329819812449, 0.0955937211690734, 0.2987929856828753, 0.21130875807026192, 0.17900645791585848]

plt.loglog(Tftot, h_T_Catot, marker='.', label='40Ca')
plt.loglog(Tftot, h_T_Srtot, marker='.', label='88Sr')
plt.loglog(Tftot, h_T_Batot, marker='.', label='133Ba')
plt.ylabel('phonon')
plt.xlabel('transport time (us)')
plt.legend(loc='best', shadow=1)
plt.title('heat vs T - Jules beta')



Text(0.5, 1.0, 'heat vs T - Jules beta')

In [3]:
h_T_Ca = [31030.349287188157, 12413.953044381038, 7298.630575924747, 3510.939305084948, 2017.6395305992164, 1188.1566023494618, 652.5621539153399, 432.28053075078225, 242.35202843560162, 164.710222772729, 100.02532657896555, 64.33398191630303, 44.54637186926699, 26.206335163365335, 19.902966473902673, 11.9039528227267, 8.488162222889649, 5.875432026113415, 3.6523823546285006, 2.8647580339933043, 1.6857211014753803, 1.3438614076825774, 0.8421910103461344, 0.5971852957461884, 0.4511561248284081, 0.2543939148988819, 0.2354175120713044, 0.12319685576662551, 0.10397938823383517, 0.07194474127947866, 0.04111442693840466, 0.03986220179835899, 0.017847800013275014, 0.018847638422114666, 0.009529827408798956, 0.0076113041460743385, 0.005814048837632633, 0.002126649217739157, 0.003559517839324575, 0.00071191941070115, 0.0013182726683338307, 0.000693682765060617, 0.00019371129132167628, 0.0005442366148113113, 8.916997524530875e-06, 0.00015765289030307756, 9.031575979334318e-05, 5.0392346928219295e-05, 7.807709050182959e-05, 1.7550782147612258e-05, 2.5127122311996352e-05, 0.00010561365974232202, 1.675559271791951e-06, 5.713281019054253e-05, 4.9977255181902484e-05, 4.5653680924709024e-05]
h_T_Sr = [156560.08543657127, 94623.46337404409, 59498.514347750024, 31096.53338858783, 17626.193784094085, 12451.92156721494, 8202.624973575392, 4857.822731281415, 3297.804812710546, 2452.8358048159953, 1629.3602513822314, 1068.3896768256272, 806.5552806631606, 595.6337476705219, 398.7223079396819, 290.2524449682287, 229.50636739461004, 163.3243769702086, 113.12434727032243, 90.53268267399145, 70.92516668345178, 48.690536797768566, 36.253779231012416, 30.62532161914686, 22.83795824618117, 15.687459613243076, 12.809242449059177, 10.70572801948134, 7.5239911955891685, 5.516751358940619, 4.8047677893873635, 3.7295560357392357, 2.5583736762843494, 2.117031087101105, 1.8244364137318207, 1.2713216526260405, 0.9259332702678801, 0.8583135487938812, 0.6713365702613325, 0.4298068936573552, 0.3660067216963548, 0.346966476751382, 0.23242075538241708, 0.15247805617273746, 0.15422302208555047, 0.1314432208591102, 0.07621122539652246, 0.06115932892012312, 0.06430208532000749, 0.04407513091519742, 0.025512634918352955, 0.027409395102505188, 0.024148013333332077, 0.012568925842176204, 0.010090997499812196, 0.01228265186084874]
h_T_Ba = [298921.21350067336, 139900.51177480214, 86659.55501854324, 62348.3354447187, 39794.96608509279, 23146.131366340385, 14680.471956629346, 10964.096505697587, 8149.805266103653, 5451.97226829091, 3597.038257490508, 2673.3194291018417, 2112.644675904647, 1557.3360975353637, 1079.7613936907442, 797.3426685278724, 643.1062353619161, 503.66646979636516, 365.33619643846686, 269.44909115768417, 218.88275744304866, 179.193764067663, 134.96070615262119, 98.94766933431559, 79.7031399279353, 67.94626203390715, 53.650498981946335, 39.201700842873834, 30.502333632781884, 26.500652324637876, 22.245359268286915, 16.647621104003203, 12.409004469088991, 10.55642708033582, 9.307481485713133, 7.334214158381726, 5.36814159645461, 4.365172066249209, 3.9254658907338342, 3.25962463991007, 2.4038174599407327, 1.8670593032161178, 1.6857074140808648, 1.468634746430704, 1.0990206742402395, 0.8089374157043434, 0.7210818625802311, 0.6713314953501047, 0.5218855200660304, 0.3604172074895525, 0.2993338512944708, 0.29818979420461533, 0.2546371963745708, 0.17181234603458206, 0.12431561966672774, 0.12416917878387784]
h_T_Ca2 = [16.185987879921658, 1.0214616823252434, 0.16551334119871794, 0.3455686513194434, 0.08971978886884781, 0.003968426055437185, 0.05031408001256366, 0.021148400542029062, 0.006871196905863802, 0.001215221717728232, 0.0029480580800028567, 0.003520545106521591, 0.0018741061825209673, 0.00029132815783059596, 0.0006433449116579165, 0.0010484178999359609, 0.0007845918873466076, 0.0005450327457897577, 0.00011210424772274242, 0.0003974874635024491, 0.0002814803529113055, 2.2602715304068003e-05, 0.00013498181160561797, 0.00015631487197869173, 0.00014111452266787924, 5.525346889278119e-05, 6.235255737864706e-05, 3.1550536477511206e-05, 0.00011366073984084505, 6.367105470847399e-05, 1.4766153009992504e-05, 0.0001026471479574828, 9.728359623286573e-05, 0.00011476518384072897, 2.1595402447752856e-05, 5.852913852670814e-05, 9.202298207957093e-05, 0.00010110655507847178, 9.289407033709327e-06, 7.838557470957584e-05, 5.414574536178698e-05, 2.3106262052002756e-05, 1.6487448167521993e-06, 5.05912307954243e-05, 3.523799800046869e-05, 2.4142934083315145e-05, 2.540439356243646e-05, 7.363256642137426e-05, 0.07072680207316834, 0.6338710424719675, 0.4843594082374225, 0.38517747821462184, 0.30264058708829533, 0.24777326195051091, 0.19744275553361298, 0.15719586171974284]
h_T_Sr2 = [73.59028672135285, 30.856633644195764, 13.226685626200236, 3.5936697666309, 0.37284732984315966, 0.021005796688595833, 0.16570715410822318, 0.15944320121335637, 0.07066075962176926, 0.013243962252939391, 0.002103504267233568, 0.009466089931808122, 0.01362670866509031, 0.009488447502967486, 0.004759949495924997, 0.0013323841281448715, 0.0005562514217962066, 0.0010836244123294632, 0.001692200647185596, 0.0015822392334407477, 0.0009845672513797345, 0.0003844692936882051, 0.00014300752081259014, 0.00018885186809801055, 0.0004220225334035303, 0.0004171086687019293, 0.00025798544258633, 0.0001577435175446184, 6.535694881952436e-06, 5.893193340897832e-05, 0.00011482439653978909, 0.00015887561804930607, 8.31350020933703e-05, 4.892362692088923e-05, 2.5555521949835615e-05, 1.7853658898927315e-05, 2.1489248465407862e-05, 5.837500914348931e-05, 4.3854514528709106e-05, 2.0358660901435142e-05, 1.0561698544249413e-05, 3.7743616093842994e-07, 3.596678793910042e-05, 1.0954060004810071e-05, 3.6307069230211004e-05, 1.4753256972289379e-05, 9.833765748968052e-06, 7.305699526256261e-06, 1.1467758565939277e-06, 1.4900638947474939e-05, 1.5125700429441127e-05, 1.2942318867930198e-05, 2.2008338390453923e-06, 1.495447215191625e-06, 8.035708390691867e-06, 9.298570199300155e-06]
h_T_Ba2 = [324.44646393354145, 71.39296249320611, 31.81090837009645, 17.325308844760215, 7.655640898690108, 2.3683260320651547, 0.3658768870439228, 0.004548456253096214, 0.09299346606363504, 0.1522638356572347, 0.11851476272324145, 0.057195341859994614, 0.01600614977194423, 0.001946958271878572, 0.003916173141739661, 0.008854329257460698, 0.011036328515886974, 0.008762092028005517, 0.005474803078303233, 0.0022663318667095556, 0.0006629489191536444, 0.00047974615478823315, 0.0009379510385374646, 0.0014197535194536612, 0.0013390832516639148, 0.001169968819617034, 0.0006017135214857696, 0.00031061905144896134, 0.0001065584127095185, 0.00010771788843996398, 0.00024971794819971205, 0.0003312008743555955, 0.00035025946268838257, 0.00032552917889445076, 0.0001725214012371399, 5.29967798530069e-05, 4.260633104911853e-05, 2.7930665227538926e-05, 7.188106730636964e-05, 0.00014575183469311373, 0.00011347431234908371, 7.974455914481941e-05, 3.774481579442432e-05, 1.6485043555955916e-05, 8.67165114062526e-06, 2.849224518055213e-05, 4.763354438216879e-05, 4.62679707887374e-05, 4.143296764607723e-05, 2.48350044314328e-05, 1.0165962974397692e-05, 4.103965477130147e-06, 1.2373383231568168e-05, 1.831011471134222e-05, 1.9873795382298877e-05, 2.2239467270705275e-05]

Tf = np.linspace(5, 60, 56)
plt.ylabel('phonon')
plt.xlabel('transport time (us)')
plt.loglog(Tf, h_T_Ca, marker='.', color='tab:blue', label='40Ca')
plt.loglog(Tf, h_T_Sr, marker='.', color='tab:orange', label='88Sr')
plt.loglog(Tf, h_T_Ba, marker='.', color='tab:green', label='133Ba')
# plt.semilogy(Tf, h_T_Ca2, marker='.', ls='--', color='tab:blue', label='40Ca')
# plt.semilogy(Tf, h_T_Sr2, marker='.', ls='--', color='tab:orange', label='88Sr')
# plt.semilogy(Tf, h_T_Ba2, marker='.', ls='--', color='tab:green', label='133Ba')
plt.legend(loc='best', shadow=1)
plt.title('heat vs T (Kaufmann beta vs Jules beta)')


Text(0.5, 1.0, 'heat vs T (Kaufmann beta vs Jules beta)')

### STA splitting

In [10]:
#### STA Splitting ####
bmax = 2e-7
eps0 = ct.epsilon_0
K = ct.e/(4*pi*eps0)
Ca = 40*ct.m_u
Sr = 88*ct.m_u
Ba = 133*ct.m_u
mass = Sr

_a10, _a11 = -780, 127
a0 = get_alpha_from_freq(1e6, m_=mass)
l0 = 2*a0/mass*np.array([1, 3]) ## (-, +)
lf = 2*a0/mass*np.array([1, 1.002])
gn, gp = (l0/lf)**0.25
rho_p_t = lambda s, a10, a11: 1-(126*(1-gp) + a10 + 5*a11)*s**5 + (420*(1-gp) + 5*a10 + 24*a11)*s**6 - (540*(1-gp) + 10*a10 + 45*a11)*s**7 + (315*(1-gp) + 10*a10 + 40*a11)*s**8 - (70*(1-gp) + 5*a10 + 15*a11)*s**9 + a10*s**10 + a11*s**11

t_anal, dx = np.linspace(0, 50, 5000, retstep=True) ## us
s_view = t_anal/t_anal[-1]
rho_p = rho_p_t(s_view, _a10, _a11)
d2rho_pdt2 = np.array([p_derivative(rho_p_t, ms=1e-5, var=0, point=[s, _a10, _a11], order=2) for s in s_view])
lp_t = l0[1]*rho_p**(-4) - d2rho_pdt2/rho_p
ln_t = np.full(len(s_view), l0[0])
d_final = (4*K/(mass*(lp_t-ln_t)))**(1/3) ## SI units [m]
a_final = mass/8*(3*lp_t - 5*ln_t) ## SI units [V/m^2]
b_final = 2*K*(d_final)**(-5) - 2*a_final*(d_final)**(-2) ## SI units [V/m^4]
bmax = np.max(b_final)
print(bmax)
print(b_final[-1]/bmax)
print(a_final[-1]/a_final[0])

fig, ax = plt.subplots()
t_CP = interp1d(a_final, t_anal).__call__(0)
plt.axvline(t_CP/t_anal[-1], color='r', ls='--', label='critical point')
ax.plot(s_view, d_final, 'k--', lw=3, label='d(t)')
ax2 = ax.twinx()
# ax2.plot(s_view, (ct.e*lp_t)**0.5/(2*pi), label='λ+(t)')
# ax2.plot(s_view, (ct.e*ln_t)**0.5/(2*pi), label='λ-(t)')
# w_final = np.array([(p_derivative(phi, ms=1e-8, point=[t, d, a, b, mass], order=2)*ct.e/mass)**0.5 for t, d, a, b in zip(t_anal*1e-6, d_final/2, a_final, b_final)])
w2_final1 = (ct.e/mass*(2*a_final + 3*b_final*d_final**2))**0.5
# ax2.plot(s_view, w_final/(2*pi), label='ω, numerical')
ax2.plot(s_view, w2_final1/(2*pi), ls='--', label='ω, Home (2006)')
ax.legend(loc='best', shadow=1)
ax2.legend(loc='best', shadow=1)
ax.set_xlabel('t/T')
ax.set_ylabel('distance [um]')
ax2.set_ylabel('Hz')

# t_anal *= 1e-6
# q_final = np.array([0]*len(t_anal))
# hbar = ct.hbar
# eps0 = ct.epsilon_0
# k0 = 1/(4*pi*eps0)
# amu = ct.m_u
# elem_charge = ct.e
# um = 1E-6 

# Nion = 2
# q_ions = elem_charge*   np.full(Nion, 1)
# m =                     np.array([mass, mass])
# x0dot = um*             np.full(Nion, 0)
# x0guess = um*           np.array([q_final[0]+1, q_final[0]-1])
# x0 = find_ic(q_final[0]/1e6, a_final[0], Nion, x0guess)
# print(x0*1e6)

# heat_start = t_anal[-1]
# heat_end = t_anal[-1] + 25e-6
# t_eval, dx = np.linspace(heat_start, heat_end, 4096, retstep=True)
# outsol, n, y_eval, q_eval = heat_sim(t_eval[-1], q_final, t_eval, showplot=False, a=a_final/1e12, b=b_final/1e24, deg=1)

# N = 2**16
# ion_w = np.fft.rfftfreq(N, dx)
# ion_fft = np.fft.rfft((y_eval[0]-q_eval) - np.average((y_eval[0]-q_eval)), N)
# peaks, _ = find_peaks(np.abs(ion_fft), np.max(np.abs(ion_fft))/4)
# f_peak = ion_w[peaks[-1]]
# print('end freq: ', f_peak)
# window = gaussian(ion_w, f_peak, 0.1E6)

# n = get_heat(np.fft.irfft(ion_fft*window, N)[:len(t_eval)], 2*pi*f_peak, m_=mass)
# print('heat: ', n, '\n')

w_STA = w2_final1
w_stretch_STA = w_STA*(1+4*ct.e*K/(mass*(w_STA)**2*(d_final)**3))**0.5



3.1589438242309965e+17
0.030669562469513682
-0.4985000000005462


In [11]:
from matplotlib.lines import Line2D

fig, ax = plt.subplots()
plt.plot(w_STA/(2*pi*1e6), color='k')
plt.plot(w_stretch_STA/(2*pi*1e6), ls='--', color='k')
plt.plot(w2_final/(2*pi*1e6), color='r')
plt.plot(w2_stretch_final/(2*pi*1e6), ls='--', color='r')
plt.title('Axial frequency of 88Sr during splitting')
plt.xlabel('Step')
plt.ylabel('Axial Frequency (MHz)')

custom_lines = [Line2D([0], [0], color='k'), Line2D([0], [0], color='r')]
custom_lines2 = [Line2D([0], [0], color='tab:grey', ls='--', lw=2), Line2D([0], [0], color='tab:grey', ls='solid', lw=2)]

ax.legend(custom_lines, ['Inner Seg', 'Outer Seg'])
ax2 = ax.twinx()
ax2.legend(custom_lines2, ['ω (stretch)', 'ω (COM)'], loc=(0.815, 0.68))


NameError: name 'w2_final' is not defined

In [4]:
from matplotlib.lines import Line2D

h_T_Ca = [31030.349287188157, 12413.953044381038, 7298.630575924747, 3510.939305084948, 2017.6395305992164, 1188.1566023494618, 652.5621539153399, 432.28053075078225, 242.35202843560162, 164.710222772729, 100.02532657896555, 64.33398191630303, 44.54637186926699, 26.206335163365335, 19.902966473902673, 11.9039528227267, 8.488162222889649, 5.875432026113415, 3.6523823546285006, 2.8647580339933043, 1.6857211014753803, 1.3438614076825774, 0.8421910103461344, 0.5971852957461884, 0.4511561248284081, 0.2543939148988819, 0.2354175120713044, 0.12319685576662551, 0.10397938823383517, 0.07194474127947866, 0.04111442693840466, 0.03986220179835899, 0.017847800013275014, 0.018847638422114666, 0.009529827408798956, 0.0076113041460743385, 0.005814048837632633, 0.002126649217739157, 0.003559517839324575, 0.00071191941070115, 0.0013182726683338307, 0.000693682765060617, 0.00019371129132167628, 0.0005442366148113113, 8.916997524530875e-06, 0.00015765289030307756, 9.031575979334318e-05, 5.0392346928219295e-05, 7.807709050182959e-05, 1.7550782147612258e-05, 2.5127122311996352e-05, 0.00010561365974232202, 1.675559271791951e-06, 5.713281019054253e-05, 4.9977255181902484e-05, 4.5653680924709024e-05]
h_T_Sr = [156560.08543657127, 94623.46337404409, 59498.514347750024, 31096.53338858783, 17626.193784094085, 12451.92156721494, 8202.624973575392, 4857.822731281415, 3297.804812710546, 2452.8358048159953, 1629.3602513822314, 1068.3896768256272, 806.5552806631606, 595.6337476705219, 398.7223079396819, 290.2524449682287, 229.50636739461004, 163.3243769702086, 113.12434727032243, 90.53268267399145, 70.92516668345178, 48.690536797768566, 36.253779231012416, 30.62532161914686, 22.83795824618117, 15.687459613243076, 12.809242449059177, 10.70572801948134, 7.5239911955891685, 5.516751358940619, 4.8047677893873635, 3.7295560357392357, 2.5583736762843494, 2.117031087101105, 1.8244364137318207, 1.2713216526260405, 0.9259332702678801, 0.8583135487938812, 0.6713365702613325, 0.4298068936573552, 0.3660067216963548, 0.346966476751382, 0.23242075538241708, 0.15247805617273746, 0.15422302208555047, 0.1314432208591102, 0.07621122539652246, 0.06115932892012312, 0.06430208532000749, 0.04407513091519742, 0.025512634918352955, 0.027409395102505188, 0.024148013333332077, 0.012568925842176204, 0.010090997499812196, 0.01228265186084874]
h_T_Ba = [298921.21350067336, 139900.51177480214, 86659.55501854324, 62348.3354447187, 39794.96608509279, 23146.131366340385, 14680.471956629346, 10964.096505697587, 8149.805266103653, 5451.97226829091, 3597.038257490508, 2673.3194291018417, 2112.644675904647, 1557.3360975353637, 1079.7613936907442, 797.3426685278724, 643.1062353619161, 503.66646979636516, 365.33619643846686, 269.44909115768417, 218.88275744304866, 179.193764067663, 134.96070615262119, 98.94766933431559, 79.7031399279353, 67.94626203390715, 53.650498981946335, 39.201700842873834, 30.502333632781884, 26.500652324637876, 22.245359268286915, 16.647621104003203, 12.409004469088991, 10.55642708033582, 9.307481485713133, 7.334214158381726, 5.36814159645461, 4.365172066249209, 3.9254658907338342, 3.25962463991007, 2.4038174599407327, 1.8670593032161178, 1.6857074140808648, 1.468634746430704, 1.0990206742402395, 0.8089374157043434, 0.7210818625802311, 0.6713314953501047, 0.5218855200660304, 0.3604172074895525, 0.2993338512944708, 0.29818979420461533, 0.2546371963745708, 0.17181234603458206, 0.12431561966672774, 0.12416917878387784]
hSTA_Ca = [2840.095448046702, 783.3115536623667, 197.1308132894436, 43.74034633878774, 8.050216318866267, 1.046348598018772, 0.05410107276067466, 0.006831022912639495, 0.02242486389052651, 0.01930010196519903, 0.010765761136369926, 0.005163487849388397, 0.0022984295421921703, 0.001191267317844142, 0.0005992663605350542, 0.0004326209532734918, 0.00021607324468043908, 0.00019125197679992543, 7.876168539681115e-05, 8.828703613975048e-05, 3.004997469192702e-05, 4.483183425722448e-05, 1.2858744938534772e-05, 2.2517898849465254e-05, 6.834031816294669e-06, 1.1587811719928914e-05, 5.4602408881373455e-06, 5.9615945429194624e-06, 4.533759989070519e-06, 2.4953906297629417e-06, 4.602892182893598e-06, 1.112701477422366e-06, 4.200982359735643e-06, 5.546494783743594e-07, 3.6010951104685015e-06, 3.755725812082357e-07, 3.4413164230091212e-06, 5.207453112964182e-07, 2.7278510913988978e-06, 9.47769050816452e-07, 2.2551945357779364e-06, 1.1616711016004658e-06, 2.0855235122224918e-06, 1.354643967894689e-06, 1.5983747397987967e-06, 1.765444109198865e-06, 1.4095897175881486e-06, 1.7279317140575153e-06, 1.449785474492822e-06, 1.765224217455701e-06, 1.2321697129484758e-06, 1.9700285409827212e-06, 1.2550039212204632e-06, 1.7372074875928125e-06, 1.4454579063144976e-06, 1.7113626865339463e-06]
hSTA_Sr = [29432.624483748285, 13353.259078516283, 5951.824454522821, 2567.514927012301, 1063.1987401310357, 421.8409577762273, 159.03457039111365, 56.17791810076062, 18.171341177121942, 5.166606229006716, 1.1865180839783833, 0.1749432050895355, 0.005981435899831694, 0.014833866937032762, 0.03239755963539833, 0.03434485144540851, 0.027291313832892863, 0.01859294628158023, 0.011610942136174533, 0.006957703200806158, 0.004152248714722771, 0.0025382940645445264, 0.0016225791920779177, 0.0010890571995492676, 0.000759219589092473, 0.0005382575300630169, 0.00037939039651416537, 0.0002636979670620392, 0.00018144842843921703, 0.0001242931874868779, 8.406619410203045e-05, 5.502830224990328e-05, 3.49128621684375e-05, 2.2092097895480304e-05, 1.4715235573941155e-05, 1.0786975975891272e-05, 8.818923512623307e-06, 8.098393585963174e-06, 8.15162776663435e-06, 8.892603535772417e-06, 1.032772432217692e-05, 1.1946320841467074e-05, 1.308431845454934e-05, 1.3517844199378415e-05, 1.3808941509962294e-05, 1.452673440217328e-05, 1.5101089560407937e-05, 1.4925192559987416e-05, 1.4024645029618592e-05, 1.3223812615906297e-05, 1.2964227945174487e-05, 1.253618565228984e-05, 1.1462327617524668e-05, 1.0082127710727479e-05, 9.178507921659704e-06, 8.767163737919888e-06]
hSTA_Ba = [76315.492666811, 39812.823975959924, 20972.234062628817, 10966.61133496681, 5611.050583885593, 2806.4400430841347, 1373.3999555879798, 650.9990713614202, 298.8783901601414, 131.6123786055753, 55.454123941098125, 21.942052943637734, 7.97463638944068, 2.562326927199085, 0.6721629184123306, 0.11268957320653297, 0.005107103263774923, 0.01516074661433383, 0.03472535886802001, 0.041413366501391415, 0.03847894352554621, 0.03136357597631728, 0.023056768652735152, 0.015468866464832145, 0.009813806912019423, 0.006205962897109747, 0.004179487384958529, 0.002979570869486426, 0.002133697694085241, 0.0014838665348794356, 0.000994021071223441, 0.0006969716729318375, 0.0005788679949094692, 0.0005274229338406443, 0.0004549801603531247, 0.00034238057719563914, 0.0002160815940633957, 0.00013574305640947905, 0.00012090368929921652, 0.000131618015750155, 0.00013063112865313673, 0.00010247300353148635, 5.69145905082356e-05, 2.651401080441898e-05, 2.5154490203254006e-05, 3.81090787775531e-05, 4.761881133449785e-05, 4.100661348361088e-05, 2.099912570213713e-05, 5.5897132764080246e-06, 4.4879273963176235e-06, 1.2885579018056264e-05, 2.1629091614659617e-05, 2.101851413172076e-05, 1.1113472451049938e-05, 2.0840032381794676e-06]
# hSTA_Ca_max = [1633.8120435224737, 387.8529043084843, 81.24376629636441, 14.14080234471729, 1.7884888118768347, 0.09786647557293263, 0.004905136063326431, 0.02174397470481738, 0.01846923593926806, 0.009025020987942858, 0.004214989065958764, 0.001832304844746274, 0.000761716267616462, 0.0005457511848226249, 0.0003087271913066726, 0.0001357091773741934, 0.00014907040797816225, 7.355976429486818e-05, 2.6890008737611432e-05, 5.252664131709673e-05, 1.971562925187808e-05, 6.733654932232196e-06, 2.413699283507758e-05, 6.2354114884766235e-06, 3.5133302642567797e-06, 1.3531252012323954e-05, 2.718721435606241e-06, 3.464623550269301e-06, 8.809973423666781e-06, 1.99070840615098e-06, 3.814233876351511e-06, 6.427122641578043e-06, 2.086051407311164e-06, 4.037709698961971e-06, 5.137092492516481e-06, 2.4022626967936322e-06, 4.102148636962241e-06, 4.4235891270398185e-06, 2.7363852207942098e-06, 4.046728788133454e-06, 4.035906867972713e-06, 3.015648284393307e-06, 3.939445262710491e-06, 3.844121367349546e-06, 3.2123107145487412e-06, 3.824943619047306e-06, 3.7627239328789326e-06, 3.352476497712126e-06, 3.724868214825406e-06, 3.7395980965208037e-06, 3.4163684420281783e-06, 3.6729277929524797e-06, 3.739143875679019e-06, 3.4250781404337833e-06, 3.670877086001163e-06, 3.7360763391741834e-06]

Tf = np.linspace(5, 60, 56)
fig, ax = plt.subplots()
plt.ylabel('Phonon')
plt.xlabel('Split Time (μs)')
plt.semilogy(Tf, hSTA_Ca, marker='x', ls='--', color='tab:blue', label='40Ca STA')
plt.semilogy(Tf, hSTA_Sr, marker='x', ls='--', color='tab:orange', label='88Sr STA')
plt.semilogy(Tf, hSTA_Ba, marker='x', ls='--', color='tab:green', label='133Ba STA')
# plt.semilogy(Tf, hSTA_Ca_max, marker='x', lw=2, color='tab:red', label='40Ca MAX')
plt.semilogy(Tf, h_T_Ca, marker='.', color='tab:blue', label='40Ca')
plt.semilogy(Tf, h_T_Sr, marker='.', color='tab:orange', label='88Sr')
plt.semilogy(Tf, h_T_Ba, marker='.', color='tab:green', label='133Ba')

custom_lines = [Line2D([0], [0], color='tab:blue'), Line2D([0], [0], color='tab:orange'), Line2D([0], [0], color='tab:green')]
custom_lines2 = [Line2D([0], [0], color='tab:grey', ls='--', marker='x', lw=2), Line2D([0], [0], color='tab:grey', ls='solid', marker='.', lw=2)]

ax.legend(custom_lines, ['40Ca', '88Sr', '133Ba'])
ax2 = ax.twinx()
ax2.legend(custom_lines2, ['Inner Seg', 'Outer Seg'], loc=(0.815, 0.68))

# plt.legend(loc='best', shadow=1)
# plt.title('Kaufmann splitting vs STA splitting')
plt.title('Ion Splitting: Inner vs Outer Segmentation')


Text(0.5, 1.0, 'Ion Splitting: Inner vs Outer Segmentation')

## Spectrograms

In [58]:
N = 2**16 ## whether or not you zero pad makes little difference
t_fft, dx = np.linspace(0, 100e-6, 4096, retstep=1)
q_w = np.fft.rfftfreq(N, dx)
q_w_func = interp1d(range(len(q_w)), q_w)
q_w_ifunc = interp1d(q_w, range(len(q_w)))

Tvec = np.array([n/10*1E-6 for n in range(10, 150)])
array = []
for T in Tvec:
#     q_t = np.array([qtrig(t, (40e-6, 40e-6+T), f=1.2e6, mu=1) for t in t_fft])
    q_t = np.array([qsta2(t, (40e-6, 40e-6+T), f=1.2e6*T) for t in t_fft])
#     q_t = np.array([120*(min(max(40e-6, t), 40e-6+T)-40e-6)/T for t in t_fft])
#     q_t = np.array([qsin(t, (40e-6, 40e-6+T)) for t in t_fft])
#     q_t = np.array([qtanhN(t, (40e-6, 40e-6+T), N=5) for t in t_fft])
    q_t = np.gradient(q_t, dx)
    q_t = np.gradient(q_t, dx)*np.hanning(len(q_t)) ## whether or not you use a hanning window makes little difference
    q_fft = np.fft.rfft(q_t, N)
    q_fftmag = np.abs(q_fft)
    q_fftphs = np.angle(q_fft)
    array.append(np.log(q_fftmag+1e-20))
    
spec = np.array(array).T
title = 'Spectrogram of STA 9th order transport'
plt.figure(title.replace(' ', '_'))
plt.imshow(spec, aspect = 'auto', ## Prevent the image from getting stretched out
    interpolation = 'nearest', 
    origin = 'lower', ## Important to do this so that the first row goes on the bottom
    cmap = 'viridis',
    vmin = 0)
plt.axis([0, len(Tvec), 0, N/20])
plt.colorbar()
plt.show()
ticks = ticker.FuncFormatter(lambda x, pos: '{0:.1f}'.format(x/10+1))
plt.axes().xaxis.set_major_formatter(ticks)
ticks = ticker.FuncFormatter(lambda y, pos: '{0:.0f}'.format(q_w_func.__call__(y)))
plt.axes().yaxis.set_major_formatter(ticks)
plt.xlabel('Transport Time (us)')
plt.ylabel('Frequency Content (Hz)')
plt.title(title)

## Horizontal Slice
title2 = 'Slice of Spectrogram at 1.2MHz'
plt.figure(title2)
plt.plot(Tvec*1e6, spec[int(q_w_ifunc.__call__(1.2e6))], marker='.', label='STA 9th order')
plt.title(title2)
plt.legend(loc='best', shadow=1)
plt.xlabel('Transport Time (us)')
plt.ylabel('FFT Magnitude (log)')

## Vertical Slice
# plt.figure('vertical slice')
# plt.plot(q_w, array[10]) ## T=2us
# plt.plot(q_w, array[130]) ## T=14us



/Users/lukeqi/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:34: MatplotlibDeprecationWarning: Adding an axes using the same arguments as a previous axes currently reuses the earlier instance.  In a future version, a new instance will always be created and returned.  Meanwhile, this warning can be suppressed, and the future behavior ensured, by passing a unique label to each axes instance.
/Users/lukeqi/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:36: MatplotlibDeprecationWarning: Adding an axes using the same arguments as a previous axes currently reuses the earlier instance.  In a future version, a new instance will always be created and returned.  Meanwhile, this warning can be suppressed, and the future behavior ensured, by passing a unique label to each axes instance.


Text(29.222222222222214, 0.5, 'FFT Magnitude (log)')

In [13]:
#### Implementing Artificial Notching ####
N = 2**12
t_fft, dx = np.linspace(0, 100e-6, 4096, retstep=1)
q_w = np.fft.rfftfreq(N, dx)
q_w_func = interp1d(range(len(q_w)), q_w)
q_w_ifunc = interp1d(q_w, range(len(q_w)))

T = 5e-6
q_t = np.array([qsta2(t, (40e-6, 40e-6+T), f=T*1.2e6) for t in t_fft])
# plt.plot(t_fft*1e6, q_t, 'k--', label='original') ## plot original protocol
q_t = np.gradient(q_t, dx)
q_t = np.gradient(q_t, dx)
q_fft = np.fft.rfft(q_t, N)
q_fftmag = np.abs(q_fft)
q_fftphs = np.angle(q_fft)
window = 1-gaussian(q_w, 1.2e6, 0.3e6) #-gaussian(q_w, 2.4e6, 0.3e6)-gaussian(q_w, 3.6e6, 0.3e6)

# plt.plot(q_w, q_fftmag/max(q_fftmag), color='tab:blue', marker='.', label='original FFT')
# plt.plot(q_w, np.abs(q_fft*window)/max(np.abs(q_fft*window)), color='tab:orange', marker='.', label='notched FFT')
# plt.plot(q_w, window, 'k--', label='Gaussian window')
# plt.legend(loc='best', shadow=1)

q_t2 = np.fft.irfft(q_fft*window, N)[:len(t_fft)]
q_t2 = 2*np.c_[np.r_[0, q_t2[1:-1:2].cumsum()], q_t2[::2].cumsum() - q_t2[0] / 2].ravel()[:len(q_t2)]
q_t2 = dx**2*2*np.c_[np.r_[0, q_t2[1:-1:2].cumsum()], q_t2[::2].cumsum() - q_t2[0] / 2].ravel()[:len(q_t2)]

# plt.plot(t_fft*1e6, q_t2, label='notched') ## plot notched protocol
# plt.legend(loc='best', shadow=1)

qtanh_notch = q_t2[int(40e-6/dx):int(45e-6/dx)]

#### Ion Simulation ####
hbar = ct.hbar
eps0 = ct.epsilon_0
k0 = 1/(4*pi*eps0)
amu = ct.m_u
elem_charge = ct.e
um = 1E-6

t_anal = np.linspace(0, 5e-6, len(qtanh_notch))
q_final = np.full(len(t_anal), 0)
a_final = np.full(len(t_anal), get_alpha_from_freq(1.2e6, m_=88*amu)*1e-12) ## V/um^2
tstart = 0

Nion = 1
q_ions = elem_charge*   np.full(Nion, 1)
m = amu*                np.array([88])
x0dot = um*             np.full(Nion, 0)
x0guess = um*           np.array([q_final[0]-1])
x0 = find_ic(q_final[0]/1e6, a_final[0]*1e12, Nion, x0guess) ## must be in SI units
print(x0*1e6)

heat_start = t_anal[-1]
heat_end = t_anal[-1]+25e-6
t_eval = np.linspace(heat_start, heat_end, 4096)
q_final = np.array(qtanh_notch)
# q_final = np.array([qsta2(t, (tstart, tstart+T), f=T*1.2e6) for t in t_anal])
outsol, n, y_eval, q_eval = heat_sim(t_eval[-1], q_final, t_eval, showplot=True, ms=1E-8, a=a_final, deg=1)  

if Nion > 1:
    freq_modes = [1.2e6, 2.08e6, 1060266.7621143984]
    N = 2**16
    dx = t_eval[1]-t_eval[0]
    ion_w = [np.fft.rfftfreq(N, dx)]*Nion
    ion_fft = [np.fft.rfft((y_eval[i]-q_eval) - np.average((y_eval[i]-q_eval)), N) for i in range(Nion)]
    windows = [gaussian(ion_w[0], freq_modes[i], 0.01e6) for i in range(Nion)]

    ion_heat = []
    for ion in range(Nion):
        ion_heat.append([get_heat(np.fft.irfft(ion_fft[ion]*windows[j], N)[:len(t_eval)], 2*pi*freq_modes[j], m_=m[ion]) for j in range(Nion)])

#         plt.figure('freq domain')
#         plt.plot(ion_w[0], np.abs(ion_fft[0]))
#         plt.plot(ion_w[1], np.abs(ion_fft[1]))
    print(T*1e6, *ion_heat, sep=': ')
else:
    print(T*1e6, n, sep=': ')



[-3.0730563e-07]
5.0: 0.04108892995253041


In [ ]:
#### Gaussian acceleration ####
def antigradient(array):
    return 2*np.c_[np.r_[0, array[1:-1:2].cumsum()], array[::2].cumsum() - array[0] / 2].ravel()[:len(array)]

N = 2**16
T = 3e-6
tbuff = 3e-6
tbuff2 = 3e-6
t_anal, dx = np.linspace(0, tbuff+T+tbuff2, 4096, retstep=1)
q_w = np.fft.rfftfreq(N, dx)
# qdd_fft = gaussian(q_w, 1/T, 10e3)
# plt.plot(q_w, qdd_fft)

qdd_t = np.array([np.sin(2*pi/T*(t-tbuff)) if t>tbuff and t<tbuff+T else 0 for t in t_anal])
# window = np.sinc((t_anal-tbuff-T/2)*pi/2/t_anal[-1])
# qdd_filtered_t = np.convolve(qdd_t, window)[::2]
# qdd_filtered_t *= 1/max(qdd_filtered_t)

qdd_fft = np.fft.rfft(qdd_t, N)
# qdd_filtered_fft = np.fft.rfft(qdd_filtered_t, N)
qdd_filtered_fft = qdd_fft*gaussian(q_w, 1/T, 200e3)

# plt.figure('freq')
# plt.semilogy(q_w, np.abs(qdd_fft), marker='.')
# plt.semilogy(q_w, np.abs(qdd_filtered_fft), marker='.')

qdd_filtered_t = np.fft.irfft(qdd_filtered_fft, N)[:len(t_anal)]
# plt.figure('time')
# plt.plot(t_anal, qdd_t)
# plt.plot(t_anal, qdd_filtered_t)

q_filtered_t = antigradient(antigradient(qdd_filtered_t))
q_filtered_t = q_filtered_t/q_filtered_t[-1]*120
q_filtered_t_func = interp1d(t_anal, q_filtered_t)
plt.plot(q_filtered_t)


In [ ]:
#### My own protocol ####
def qluke(t, tspan, qspan=(0, 120)):
    ''' t is a specific time
        tspan is a tuple with the start and end time'''
    t0, tf = tspan
    y0, yf = qspan
    T = tf-t0
    s = (t-t0)/T
    if t < t0:
        return y0
    elif (t-t0) < T:
        return (yf-y0)*(s - np.sin(2*np.pi*s)/(2*np.pi))
    else:
        return yf
    

#### Ion Simulation ####
T = 3e-6
t_anal = np.linspace(0, 9e-6, 1000)
q_final = np.full(len(t_anal), 0)
a_final = np.full(len(t_anal), get_alpha_from_freq(1.2e6, m_=88*amu)*1e-12) ## V/um^2
tstart = 0

hbar = ct.hbar
eps0 = ct.epsilon_0
k0 = 1/(4*pi*eps0)
amu = ct.m_u
elem_charge = ct.e
um = 1E-6

Nion = 1
q_ions = elem_charge*   np.full(Nion, 1)
m = amu*                np.array([88])
x0dot = um*             np.full(Nion, 0)
x0guess = um*           np.array([q_final[0]-1])
x0 = find_ic(q_final[0]/1e6, a_final[0]*1e12, Nion, x0guess) ## must be in SI units
print(x0*1e6)

heat_start = t_anal[-1]
heat_end = t_anal[-1]+25e-6
t_eval = np.linspace(heat_start, heat_end, 4096)
# q_final = np.array([qluke(t, (tstart, tstart+T)) for t in t_anal])
# q_final_accel = np.gradient(np.gradient(q_final, t_anal[1]), t_anal[1])
# plt.plot(t_anal*1e6, q_final_accel)
q_final = q_filtered_t_func.__call__(t_anal)

outsol, n, y_eval, q_eval = heat_sim(t_eval[-1], q_final, t_eval, showplot=True, ms=1E-8, a=a_final, deg=1)  

if Nion > 1:
    freq_modes = [1.2e6, 2.08e6, 1060266.7621143984]
    N = 2**16
    dx = t_eval[1]-t_eval[0]
    ion_w = [np.fft.rfftfreq(N, dx)]*Nion
    ion_fft = [np.fft.rfft((y_eval[i]-q_eval) - np.average((y_eval[i]-q_eval)), N) for i in range(Nion)]
    windows = [gaussian(ion_w[0], freq_modes[i], 0.01e6) for i in range(Nion)]

    ion_heat = []
    for ion in range(Nion):
        ion_heat.append([get_heat(np.fft.irfft(ion_fft[ion]*windows[j], N)[:len(t_eval)], 2*pi*freq_modes[j], m_=m[ion]) for j in range(Nion)])

#         plt.figure('freq domain')
#         plt.plot(ion_w[0], np.abs(ion_fft[0]))
#         plt.plot(ion_w[1], np.abs(ion_fft[1]))
    print(T*1e6, *ion_heat, sep=': ')
else:
    print(T*1e6, n, sep=': ')

In [ ]:
#### Spectrogram of multi-ion motion ####

t_anal = np.linspace(0, 20e-6, 10000)
a_final = np.full(len(t_anal), get_alpha_from_freq(1.2e6, m_=88*amu)*1e-12) ## V/um^2
q_final = np.full(len(t_anal), 0)
tstart = 1E-6

hbar = ct.hbar
eps0 = ct.epsilon_0
k0 = 1/(4*pi*eps0)
amu = ct.m_u
elem_charge = ct.e
um = 1E-6

Nion = 2
q_ions = elem_charge*   np.full(Nion, 1)
m = amu*                np.array([88, 133])
x0dot = um*             np.full(Nion, 0)
x0guess = um*           np.array([q_final[0]-1, q_final[0]+1])
x0 = find_ic(q_final[0]/1e6, a_final[0]*1e12, Nion, x0guess) ## must be in SI units
print(x0*1e6)

T=5e-6
t_anal = np.linspace(0, T + tstart, 10000)
heat_start = t_anal[-1]
heat_end = t_anal[-1]+25e-6
t_eval = np.linspace(heat_start, heat_end, 4096)
q_final = np.array([qsta(t, (tstart, tstart+T), f=T*1060266.7621143984) for t in t_anal])
outsol, n, y_eval, q_eval = heat_sim(t_eval[-1], q_final, t_eval, showplot=False, ms=1E-8, a=a_final, deg=1)  

t_tot = np.append(t_anal, t_eval)
y_vals = outsol.sol.__call__(t_tot)
y_SR = y_vals[0]
y_Ba = y_vals[1]
plt.plot(t_tot*1e6, y_SR, marker='.')
plt.plot(t_tot*1e6, y_vals[2], marker='.')

# if Nion > 1:
#     freq_modes = [1.056e6, 1.9e6]
#     N = 2**16
#     dx = t_eval[1]-t_eval[0]
#     ion_w = [np.fft.rfftfreq(N, dx)]*Nion
#     ion_fft = [np.fft.rfft((y_eval[i]-q_eval) - np.average((y_eval[i]-q_eval)), N) for i in range(Nion)]
#     windows = [gaussian(ion_w[0], freq_modes[i], 0.1E6) for i in range(Nion)]

#     ion_heat = []
#     for ion in range(Nion):
#         ion_heat.append([get_heat(np.fft.irfft(ion_fft[ion]*windows[j], N)[:len(t_eval)], 2*pi*freq_modes[j], m_=m[ion]) for j in range(Nion)])

# #         plt.figure('freq domain')
# #         plt.plot(ion_w[0], np.abs(ion_fft[0]))
# #         plt.plot(ion_w[1], np.abs(ion_fft[1]))
#     print(T*1e6, *ion_heat, sep=': ')
# else:
#     print(T*1e6, n, sep=': ')
    

In [ ]:
#### Spectrogram of STA splitting ####
N = 2**16 ## whether or not you zero pad makes little difference
t_fft, dx = np.linspace(0, 100e-6, 4096, retstep=1)
q_w = np.fft.rfftfreq(N, dx)
q_w_func = interp1d(range(len(q_w)), q_w)
q_w_ifunc = interp1d(q_w, range(len(q_w)))

bmax = 2e-7
eps0 = ct.epsilon_0
K = ct.e/(4*pi*eps0)
Ca = 40*ct.m_u
Sr = 88*ct.m_u
Ba = 133*ct.m_u
mass = Ca

_a10, _a11 = -780, 127
a0 = get_alpha_from_freq(1e6, m_=mass)
l0 = 2*a0/mass*np.array([1, 3]) ## (-, +)
lf = 2*a0/mass*np.array([1, 1.002])
gn, gp = (l0/lf)**0.25 
rho_p_t = lambda s, a10, a11: 1-(126*(1-gp) + a10 + 5*a11)*s**5 + (420*(1-gp) + 5*a10 + 24*a11)*s**6 - (540*(1-gp) + 10*a10 + 45*a11)*s**7 + (315*(1-gp) + 10*a10 + 40*a11)*s**8 - (70*(1-gp) + 5*a10 + 15*a11)*s**9 + a10*s**10 + a11*s**11

Tvec = np.array([n/2*1e-6 for n in range(10, 120)])
array = []
for T in Tvec:
    s_view = np.minimum(t_fft/T, 1)
    rho_p = rho_p_t(s_view, _a10, _a11)
    d2rho_pdt2 = np.array([p_derivative(rho_p_t, ms=1e-5, var=0, point=[s, _a10, _a11], order=2) for s in s_view])
    lp_t = l0[1]*rho_p**(-4) - d2rho_pdt2/rho_p
    ln_t = np.full(len(s_view), l0[0])
    q_t = (4*K/(mass*(lp_t-ln_t)))**(1/3) ## SI units [m]
    q_t = np.gradient(q_t, dx)
    q_t = np.gradient(q_t, dx)*np.hanning(len(q_t)) ## whether or not you use a hanning window makes little difference
    q_fft = np.fft.rfft(q_t, N)
    q_fftmag = np.abs(q_fft)
    q_fftphs = np.angle(q_fft)
    array.append(np.log(q_fftmag+1e-20))
    
spec = np.array(array).T
title = 'Spectrogram of STA splitting'
plt.figure(title.replace(' ', '_'))
plt.imshow(spec, aspect = 'auto', ## Prevent the image from getting stretched out
    interpolation = 'nearest', 
    origin = 'lower', ## Important to do this so that the first row goes on the bottom
    cmap = 'viridis',
    vmin = 0)
plt.axis([0, len(Tvec), 0, N/20])
plt.colorbar()
plt.show()
ticks = ticker.FuncFormatter(lambda x, pos: '{0:.1f}'.format(x/2+1))
plt.axes().xaxis.set_major_formatter(ticks)
ticks = ticker.FuncFormatter(lambda y, pos: '{0:.0f}'.format(q_w_func.__call__(y)))
plt.axes().yaxis.set_major_formatter(ticks)
plt.xlabel('Transport Time (us)')
plt.ylabel('Frequency Content (Hz)')
plt.title(title)

title2 = 'Slice of Spectrogram at various frequencies'
plt.figure(title2)
plt.plot(Tvec*1e6, spec[int(q_w_ifunc.__call__(0.658e6))], marker='.', label='658 kHz')
plt.title(title2)
plt.legend(loc='best', shadow=1)
plt.xlabel('Transport Time (us)')
plt.ylabel('FFT Magnitude (log)')


## Vibrational Modes

In [ ]:
mu = 88/133
wi = 2*pi*1.2e6
w_IP = wi*(1 + mu - (1 + - mu + mu**2)**0.5)**0.5
w_OOP = wi*(1 + mu + (1 + - mu + mu**2)**0.5)**0.5
r = ((1+mu+(1-mu+mu**2)**0.5)/(1+mu-(1-mu+mu**2)**0.5))**0.5

print(w_OOP/w_IP, r) ## shows agreement between Colin's paper and 'Two-species Coulomb chains for quantum information'
print(w_IP/(2*pi))

**These are just trapping voltages for MaxBeta**

In [ ]:
t_transport = np.linspace(0, 20, 10)*1e-6
q_ideal = np.array([186.5 for t in t_transport])
a_ideal = np.array([get_alpha_from_freq(1.2e6, m_=Ca)*1e-12 for t in t_transport])

#### Call Backwards ITVG ####
R = [[0, 0.0, 50.0]]
tilt_angle = np.radians(15) ## convert from deg to rad
volt_gen = vg3
fix_volt = [(i, 0.0) for i, v in enumerate(artiq_electrode_ordering) if v not in [8,9,10,11,12,13,14,29,30,31,32]]
print(fix_volt)
bounds = (-10.0, 10.0)
a = a_ideal[-1]
norm = np.full((9, 1), 1)
# norm[1] = 1e3
# norm[6] = 1e3
volt_gen.set_electrode_grouping(electrode_grouping3) ## sets previous voltages to None. MAKE SURE ELECTRODE GROUPING IS CORRECT
for q, i in zip(q_ideal, 1+np.arange(len(t_transport))):
    trap_center = [0, q, 50.0]
    R = [trap_center]
    xx, xz, zz = tiltWithPseudo(volt_gen, R, tilt_angle, a, volt_gen.rf_electrodes, debug=False)
    print(i, 'solving at (q, a):', q, a)
    cons = [('x', 0), ('y', 0), ('z', 0), ('xx', xx), ('xz', xz), ('zz', zz), ('yy', a), ('yz', 0), ('xy', 0)]
    nom_v = volt_gen.findControlVoltages(R, cons=[cons], tol=1e-15, fixed_voltages=fix_volt, bnds=bounds, normvec=norm, epss=1e-9, independent=False)
    save_voltage_config(volt_gen, nom_v, os.path.join('/Users/lukeqi/Desktop/School/MIT/UROP/SuperUROP/IonMotion/backwards_ITVG_output/maxbeta', "NominalVolts_{}.npy".format(i)))
